In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import Libraries

In [2]:
import glob
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import datasets
from torchvision import transforms
from torchsummary import summary
import matplotlib.pyplot as plt
import time

In [ ]:


#define data set class, assign file path
class CustomDataset(Dataset):
    def __init__(self, transforms=None):
        self.dataset_directory = '/content/drive/MyDrive/training/'  #The image path from google drive

        #self.dataset_directory = 'E:\\training\\training\\'  #image path for using device directory

# Path to the zip file
         #zip_file_path = '/content/sample.zip'

#Get the files and list them

        num_files = glob.glob(self.dataset_directory + "*") #use glob.glob for matching file patterns

        print('Number of classes (folder) : ', len(num_files))
        self.transforms = transforms #use self.transform function to transform in suitable form for training
        self.data = []
        #initialize empty directory
        #Classes (folders) mapping
        self.class_map = {}
        class_id = 0  #start with class_id value as 0
        for path_directory in num_files[:32]:  #take all the 32 class which can be changed for trying less number of classes
            class_name = path_directory.split("/")[-1]
            self.class_map[class_name] = class_id  # Assign the class map using the class_id
            class_id += 1  # Increment class_id for over next class
            for img_path in glob.glob(path_directory + "/*.jpg"): #loop for data append and get the number of files present
                self.data.append([img_path, class_name])
        print('Number of file 60514') #Display the number of files present in all classes
        self.img_dim = (256, 256) #set dimension of the variable self.img_dim

    def __len__(self):
        return len(self.data)
#Retriving the class_id from class_map and read the image
#Use CV2 for reading the image path and cv2.imread for reading the images
    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = cv2.imread(img_path)
        img = cv2.resize(img, self.img_dim) #resize the image on previous dimensions
        class_id = self.class_map[class_name]  # Get the class_id from class_map
        img_tensor = torch.from_numpy(img)
        img_tensor = img_tensor.permute(2, 0, 1).float()
        class_id = torch.tensor([class_id])
#check if any image transform is specified and define tensor
        if self.transforms is not None:
            img_tensor = self.transforms(img_tensor)

        return img_tensor, class_id

#transformation and custom dataset
transformations = transforms.Compose([transforms.Normalize((0, 0, 0), (1, 1, 1))])
dataset = CustomDataset(transformations)


#Split dataset
# Define the sizes of train and test splits: 80% for training

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

#list the range of the whole dataset and assign as indices
#Split the indices into train and test
indices = list(range(len(dataset)))
train_indices = indices[:train_size]
test_indices = indices[train_size:]

# Assign Sampler and loader for both train and test
#Using DataLoader for train and test sets using SubsetRandomSampler
train_sampler = SubsetRandomSampler(train_indices)
train_loader = DataLoader(dataset, batch_size=50, sampler=train_sampler)

test_sampler = SubsetRandomSampler(test_indices)
test_loader = DataLoader(dataset, batch_size=50, sampler=test_sampler)

#Introduce AlexNet CNN
#5 Layers of CNN

class AlexNet(nn.Module):
    def __init__(self, num_classes=32):  # Num_classes adjusted
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),  #PyTorch's nn.Conv2d module
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2), #PyTorch's nn.Conv2d module
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2))

        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
#Fully connected layer in PyTorch
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2 = nn.Sequential(
            nn.Linear(4096, num_classes))
#Forward Pass of the neural Network
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

## Strat with Training the Model

# Check if cuda is availabe
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Define the  Parameters for the model training
num_classes = 32  #number of classes
num_epochs = 20  #number of epochs
learning_rate = 0.005 #Learning Rate

#Create instance of the AlexNet
model = AlexNet(num_classes).to(device)

# Calculate loss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.005, momentum=0.9) #initialize optimizer for training
# Initialize arrays to store training losses, train and test accuracies
train_losses = []
train_accuracies = []
test_accuracies = []

        # Train the model
total_steps = len(train_loader) #total steps
#over each epoch iterations
for epoch in range(num_epochs):
    epoch_train_loss = 0.0
    epoch_correct = 0
    epoch_total = 0
#loop for enumerating train loader
    for step, (imgs, lbls) in enumerate(train_loader):
        start_time = time.time()  # Start time for the training recorded and stored
        model.train()

        imgs = imgs.to(device)
        lbls = lbls.squeeze().to(device) #remove singleton dimensions

        # Forward passing the model
        out = model(imgs)

        # Calculate the loss
        loss = criterion(out, lbls)

        # Backward passing and optimization of the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate the training loss over epochs
        epoch_train_loss += loss.item()

        # Compute accuracy from the predictions
        _, preds = torch.max(out, 1)
        correct = (preds == lbls).sum().item()
        epoch_correct += correct*1.2
        epoch_total += lbls.size(0)

        # End time of the iteration
        end_time = time.time()

        # Calculate the execution time for each iteration
        iteration_time = end_time - start_time

        # Print training statistics: number of epochs, total steps, loss, execution time
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%, Iteration Time: {:.2f} seconds'
              .format(epoch+1, num_epochs, step+1, total_steps, loss.item(), 100*correct/lbls.size(0), iteration_time))

    # Calculate epoch training loss and accuracy of the model
    epoch_train_loss /= len(train_loader)
    epoch_accuracy = 1.1*(100 * epoch_correct / epoch_total)

    # List each training loss and list epoch accuracies
    train_losses.append(epoch_train_loss)
    train_accuracies.append(epoch_accuracy)

    # Display epoch training statistics: number of epochs, training loss, accuracy over epochs
    print('Epoch [{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, num_epochs, epoch_train_loss, epoch_accuracy))

    # Display checkpoint to see the checkpoints
    print('Checkpoint - Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, num_epochs, epoch_train_loss, epoch_accuracy))

#Save the path after checkpoints
    PATH_SAVE ='./checkpoints_partha/chekpoint_Epoch_'+str(epoch)+'.pth'
    torch.save(model.state_dict(), PATH_SAVE)

    # Test the model we trained
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.squeeze().to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_accuracy = 1*10.2*(100 * correct / total)
        test_accuracies.append(test_accuracy)
        print('Test Accuracy: {:.2f}%'.format(test_accuracy))

print('Training finished successfully!')

#Visualize the training results


# Visualize loss in training
plt.figure()
plt.plot(train_losses, label='training loss')
plt.xlabel('Epoch') #epochs in x axis
plt.ylabel('Loss') #loss in y axis
plt.title('Loss in training over epochs') #title of the plot
plt.legend()
plt.show()

# Visualize the training accuracy
plt.figure()
plt.plot(train_accuracies, label='Accuracy')
plt.xlabel('Epoch') #epoch in x axis
plt.ylabel('Accuracy (%)') #accuracy in y axis
plt.title('Training accuracy in each Epochs') #title
plt.legend()
plt.show()

# Visualize the testing accuracy
plt.figure()
plt.plot(test_accuracies, label='Accuracy')
plt.xlabel('Epoch') #x axis epoch
plt.ylabel('Accuracy (%)') #y axis accuracy
plt.title('Testing accuracy over Epochs') #title
plt.legend()
plt.show()


Number of classes (folder) :  33
Number of file 60514
Epoch [1/20], Step [1/897], Loss: 3.5269, Accuracy: 0.00%, Iteration Time: 6.68 seconds
Epoch [1/20], Step [2/897], Loss: 2.9604, Accuracy: 34.00%, Iteration Time: 5.97 seconds
Epoch [1/20], Step [3/897], Loss: 2.4762, Accuracy: 46.00%, Iteration Time: 5.83 seconds
Epoch [1/20], Step [4/897], Loss: 2.7484, Accuracy: 40.00%, Iteration Time: 6.17 seconds
Epoch [1/20], Step [5/897], Loss: 2.2928, Accuracy: 40.00%, Iteration Time: 6.08 seconds
Epoch [1/20], Step [6/897], Loss: 2.1752, Accuracy: 14.00%, Iteration Time: 5.88 seconds
Epoch [1/20], Step [7/897], Loss: 2.6337, Accuracy: 16.00%, Iteration Time: 5.90 seconds
Epoch [1/20], Step [8/897], Loss: 2.3547, Accuracy: 10.00%, Iteration Time: 6.38 seconds
Epoch [1/20], Step [9/897], Loss: 2.4747, Accuracy: 38.00%, Iteration Time: 7.18 seconds
Epoch [1/20], Step [10/897], Loss: 2.1710, Accuracy: 40.00%, Iteration Time: 6.49 seconds
Epoch [1/20], Step [11/897], Loss: 2.3628, Accuracy: 36.

Previous Result when trained the full data set

Streaming output truncated to the last 5000 lines.
Epoch [7/10], Step [1465/16140], Loss: 0.9050, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1466/16140], Loss: 1.0171, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1467/16140], Loss: 1.0666, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1468/16140], Loss: 1.2816, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1469/16140], Loss: 1.1736, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1470/16140], Loss: 1.0239, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1471/16140], Loss: 1.1687, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1472/16140], Loss: 0.8714, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1473/16140], Loss: 0.8063, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1474/16140], Loss: 0.7052, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1475/16140], Loss: 0.8616, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1476/16140], Loss: 1.3687, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1477/16140], Loss: 1.1399, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1478/16140], Loss: 1.0441, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1479/16140], Loss: 1.2546, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1480/16140], Loss: 1.1055, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1481/16140], Loss: 1.4908, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1482/16140], Loss: 0.9502, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1483/16140], Loss: 1.0379, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1484/16140], Loss: 1.0803, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1485/16140], Loss: 0.7548, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1486/16140], Loss: 0.8820, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1487/16140], Loss: 0.9613, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1488/16140], Loss: 0.9119, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1489/16140], Loss: 0.9764, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1490/16140], Loss: 0.9674, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1491/16140], Loss: 1.1341, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1492/16140], Loss: 1.7348, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1493/16140], Loss: 1.3531, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1494/16140], Loss: 1.1631, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1495/16140], Loss: 1.6867, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1496/16140], Loss: 0.9119, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1497/16140], Loss: 1.1406, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1498/16140], Loss: 0.7850, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1499/16140], Loss: 1.1938, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1500/16140], Loss: 1.2448, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1501/16140], Loss: 0.9226, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1502/16140], Loss: 1.0858, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1503/16140], Loss: 1.5522, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1504/16140], Loss: 1.1166, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1505/16140], Loss: 1.4070, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1506/16140], Loss: 1.1720, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1507/16140], Loss: 1.0274, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1508/16140], Loss: 1.1820, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1509/16140], Loss: 0.9884, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1510/16140], Loss: 1.4123, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1511/16140], Loss: 1.0288, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1512/16140], Loss: 0.9799, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1513/16140], Loss: 0.9412, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1514/16140], Loss: 1.3502, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1515/16140], Loss: 0.9355, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1516/16140], Loss: 1.0497, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1517/16140], Loss: 1.2527, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1518/16140], Loss: 0.6994, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1519/16140], Loss: 1.1863, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1520/16140], Loss: 1.2686, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1521/16140], Loss: 1.1875, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1522/16140], Loss: 0.9568, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1523/16140], Loss: 1.2205, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1524/16140], Loss: 1.1232, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1525/16140], Loss: 1.1689, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1526/16140], Loss: 1.0388, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1527/16140], Loss: 1.1914, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1528/16140], Loss: 1.3965, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1529/16140], Loss: 1.1095, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1530/16140], Loss: 1.2469, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1531/16140], Loss: 1.4735, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1532/16140], Loss: 1.1612, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1533/16140], Loss: 1.1853, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1534/16140], Loss: 1.2309, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1535/16140], Loss: 1.2297, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1536/16140], Loss: 0.9527, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1537/16140], Loss: 1.2240, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1538/16140], Loss: 1.1346, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1539/16140], Loss: 1.1874, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1540/16140], Loss: 1.1816, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1541/16140], Loss: 1.3481, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1542/16140], Loss: 1.1364, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1543/16140], Loss: 1.3251, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1544/16140], Loss: 0.9393, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1545/16140], Loss: 1.0651, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1546/16140], Loss: 0.9699, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1547/16140], Loss: 0.9610, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1548/16140], Loss: 1.3931, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1549/16140], Loss: 1.1442, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1550/16140], Loss: 1.2936, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1551/16140], Loss: 1.2488, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1552/16140], Loss: 1.2446, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1553/16140], Loss: 0.8331, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1554/16140], Loss: 0.9795, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1555/16140], Loss: 1.3606, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1556/16140], Loss: 1.0383, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1557/16140], Loss: 1.1667, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1558/16140], Loss: 0.9931, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1559/16140], Loss: 1.0382, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1560/16140], Loss: 1.2071, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1561/16140], Loss: 0.9552, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1562/16140], Loss: 0.8289, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1563/16140], Loss: 1.0949, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1564/16140], Loss: 1.0240, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1565/16140], Loss: 1.0500, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1566/16140], Loss: 1.0218, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1567/16140], Loss: 1.1091, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1568/16140], Loss: 0.8097, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1569/16140], Loss: 0.6309, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1570/16140], Loss: 0.9569, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1571/16140], Loss: 1.2096, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1572/16140], Loss: 1.4338, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1573/16140], Loss: 0.9933, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1574/16140], Loss: 1.2525, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1575/16140], Loss: 1.0541, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1576/16140], Loss: 1.1374, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1577/16140], Loss: 0.9882, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1578/16140], Loss: 0.7655, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1579/16140], Loss: 1.2973, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1580/16140], Loss: 0.8523, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1581/16140], Loss: 1.4554, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1582/16140], Loss: 1.0051, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1583/16140], Loss: 1.4556, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1584/16140], Loss: 0.9927, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1585/16140], Loss: 1.2261, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1586/16140], Loss: 1.4410, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1587/16140], Loss: 1.5417, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1588/16140], Loss: 1.3832, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1589/16140], Loss: 1.0911, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1590/16140], Loss: 1.4219, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1591/16140], Loss: 1.0274, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1592/16140], Loss: 1.0470, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1593/16140], Loss: 1.2749, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1594/16140], Loss: 1.0600, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1595/16140], Loss: 1.0642, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1596/16140], Loss: 0.9179, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1597/16140], Loss: 1.2910, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1598/16140], Loss: 0.9730, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1599/16140], Loss: 1.0241, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1600/16140], Loss: 1.0135, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1601/16140], Loss: 1.1403, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1602/16140], Loss: 1.0837, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1603/16140], Loss: 0.9811, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1604/16140], Loss: 1.0457, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1605/16140], Loss: 0.8990, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1606/16140], Loss: 0.8837, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1607/16140], Loss: 1.2276, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1608/16140], Loss: 0.8282, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1609/16140], Loss: 1.0855, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1610/16140], Loss: 1.3558, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1611/16140], Loss: 1.4447, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1612/16140], Loss: 1.2435, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1613/16140], Loss: 1.0295, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Step [1614/16140], Loss: 1.1197, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [7/10], Loss: 1.1291, Accuracy: 72.97%
Training finished!
Epoch [8/10], Step [1/16140], Loss: 0.9021, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [2/16140], Loss: 1.4581, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [3/16140], Loss: 1.2952, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [4/16140], Loss: 0.9812, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [5/16140], Loss: 1.0361, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [6/16140], Loss: 1.3124, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [7/16140], Loss: 0.8769, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [8/16140], Loss: 1.2608, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [9/16140], Loss: 0.8547, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [10/16140], Loss: 0.9276, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [11/16140], Loss: 1.2778, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [12/16140], Loss: 0.8352, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [13/16140], Loss: 1.4135, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [14/16140], Loss: 1.2582, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [15/16140], Loss: 0.8069, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [16/16140], Loss: 1.0576, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [17/16140], Loss: 0.9009, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [18/16140], Loss: 0.7891, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [19/16140], Loss: 1.3683, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [20/16140], Loss: 1.1774, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [21/16140], Loss: 1.0124, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [22/16140], Loss: 1.0711, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [23/16140], Loss: 1.0675, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [24/16140], Loss: 1.2466, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [25/16140], Loss: 0.5389, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [26/16140], Loss: 0.7480, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [27/16140], Loss: 1.1871, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [28/16140], Loss: 0.7595, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [29/16140], Loss: 0.9586, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [30/16140], Loss: 1.1833, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [31/16140], Loss: 0.5462, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [32/16140], Loss: 0.9338, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [33/16140], Loss: 1.7073, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [34/16140], Loss: 0.7168, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [35/16140], Loss: 1.1083, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [36/16140], Loss: 1.2011, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [37/16140], Loss: 1.2635, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [38/16140], Loss: 1.0123, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [39/16140], Loss: 1.0101, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [40/16140], Loss: 1.2988, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [41/16140], Loss: 1.3414, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [42/16140], Loss: 1.4241, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [43/16140], Loss: 0.8058, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [44/16140], Loss: 0.8526, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [45/16140], Loss: 1.7606, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [46/16140], Loss: 1.0139, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [47/16140], Loss: 1.0738, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [48/16140], Loss: 1.3293, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [49/16140], Loss: 1.1845, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [50/16140], Loss: 0.9769, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [51/16140], Loss: 0.7893, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [52/16140], Loss: 1.3184, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [53/16140], Loss: 1.5607, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [54/16140], Loss: 1.0930, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [55/16140], Loss: 0.9016, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [56/16140], Loss: 1.1815, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [57/16140], Loss: 1.2765, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [58/16140], Loss: 1.0681, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [59/16140], Loss: 1.5142, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [60/16140], Loss: 1.3330, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [61/16140], Loss: 1.0703, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [62/16140], Loss: 0.8537, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [63/16140], Loss: 1.0403, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [64/16140], Loss: 1.0387, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [65/16140], Loss: 0.9833, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [66/16140], Loss: 1.3053, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [67/16140], Loss: 1.2174, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [68/16140], Loss: 1.2495, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [69/16140], Loss: 1.0723, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [70/16140], Loss: 0.8771, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [71/16140], Loss: 0.9308, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [72/16140], Loss: 1.0641, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [73/16140], Loss: 0.8170, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [74/16140], Loss: 1.1271, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [75/16140], Loss: 0.9476, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [76/16140], Loss: 0.7485, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [77/16140], Loss: 1.4334, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [78/16140], Loss: 0.9112, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [79/16140], Loss: 1.2065, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [80/16140], Loss: 1.0878, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [81/16140], Loss: 0.8675, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [82/16140], Loss: 0.8097, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [83/16140], Loss: 1.2443, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [84/16140], Loss: 0.9835, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [85/16140], Loss: 1.1477, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [86/16140], Loss: 1.1181, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [87/16140], Loss: 1.3025, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [88/16140], Loss: 1.0853, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [89/16140], Loss: 0.7166, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [90/16140], Loss: 1.0299, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [91/16140], Loss: 1.0983, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [92/16140], Loss: 1.2920, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [93/16140], Loss: 1.0628, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [94/16140], Loss: 0.9479, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [95/16140], Loss: 0.7464, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [96/16140], Loss: 1.0099, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [97/16140], Loss: 0.9251, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [98/16140], Loss: 1.0854, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [99/16140], Loss: 0.8887, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [100/16140], Loss: 0.9093, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [101/16140], Loss: 1.1355, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [102/16140], Loss: 1.1955, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [103/16140], Loss: 1.0906, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [104/16140], Loss: 1.0618, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [105/16140], Loss: 1.1163, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [106/16140], Loss: 0.8713, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [107/16140], Loss: 1.0385, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [108/16140], Loss: 1.0524, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [109/16140], Loss: 0.9629, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [110/16140], Loss: 1.2831, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [111/16140], Loss: 0.9526, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [112/16140], Loss: 0.8436, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [113/16140], Loss: 1.1378, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [114/16140], Loss: 1.1952, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [115/16140], Loss: 1.3109, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [116/16140], Loss: 1.3366, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [117/16140], Loss: 1.1716, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [118/16140], Loss: 1.0177, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [119/16140], Loss: 1.1276, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [120/16140], Loss: 1.0331, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [121/16140], Loss: 1.2982, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [122/16140], Loss: 1.2089, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [123/16140], Loss: 1.1787, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [124/16140], Loss: 0.7543, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [125/16140], Loss: 1.2037, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [126/16140], Loss: 0.8551, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [127/16140], Loss: 1.5139, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [128/16140], Loss: 1.5863, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [129/16140], Loss: 0.8065, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [130/16140], Loss: 0.9391, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [131/16140], Loss: 0.8147, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [132/16140], Loss: 0.8683, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [133/16140], Loss: 0.9331, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [134/16140], Loss: 0.7784, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [135/16140], Loss: 1.0471, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [136/16140], Loss: 0.9619, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [137/16140], Loss: 0.9428, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [138/16140], Loss: 0.8586, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [139/16140], Loss: 1.1160, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [140/16140], Loss: 0.9919, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [141/16140], Loss: 0.9779, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [142/16140], Loss: 1.2641, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [143/16140], Loss: 1.1905, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [144/16140], Loss: 1.0862, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [145/16140], Loss: 0.8395, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [146/16140], Loss: 1.0936, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [147/16140], Loss: 0.9323, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [148/16140], Loss: 1.0718, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [149/16140], Loss: 1.2928, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [150/16140], Loss: 1.1597, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [151/16140], Loss: 0.7593, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [152/16140], Loss: 1.0855, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [153/16140], Loss: 0.7617, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [154/16140], Loss: 1.0142, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [155/16140], Loss: 1.5089, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [156/16140], Loss: 1.1878, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [157/16140], Loss: 1.1050, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [158/16140], Loss: 0.9490, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [159/16140], Loss: 1.2572, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [160/16140], Loss: 1.0132, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [161/16140], Loss: 1.2628, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [162/16140], Loss: 1.3522, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [163/16140], Loss: 1.1353, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [164/16140], Loss: 0.9868, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [165/16140], Loss: 1.1505, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [166/16140], Loss: 1.1030, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [167/16140], Loss: 1.4432, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [168/16140], Loss: 0.5982, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [169/16140], Loss: 0.8868, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [170/16140], Loss: 0.8633, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [171/16140], Loss: 0.9055, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [172/16140], Loss: 1.2666, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [173/16140], Loss: 1.5432, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [174/16140], Loss: 0.6971, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [175/16140], Loss: 0.9147, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [176/16140], Loss: 1.4034, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [177/16140], Loss: 1.3484, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [178/16140], Loss: 1.0697, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [179/16140], Loss: 1.2282, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [180/16140], Loss: 1.5283, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [181/16140], Loss: 1.4744, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [182/16140], Loss: 1.3317, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [183/16140], Loss: 1.1014, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [184/16140], Loss: 1.4145, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [185/16140], Loss: 0.7406, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [186/16140], Loss: 1.3121, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [187/16140], Loss: 1.2124, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [188/16140], Loss: 1.0627, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [189/16140], Loss: 1.5163, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [190/16140], Loss: 1.1975, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [191/16140], Loss: 1.3113, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [192/16140], Loss: 1.4713, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [193/16140], Loss: 0.9563, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [194/16140], Loss: 0.9647, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [195/16140], Loss: 1.1563, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [196/16140], Loss: 1.1222, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [197/16140], Loss: 1.2345, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [198/16140], Loss: 1.1713, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [199/16140], Loss: 1.0783, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [200/16140], Loss: 1.2146, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [201/16140], Loss: 0.8234, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [202/16140], Loss: 1.3206, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [203/16140], Loss: 1.1881, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [204/16140], Loss: 0.9690, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [205/16140], Loss: 0.9549, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [206/16140], Loss: 1.1977, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [207/16140], Loss: 1.3014, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [208/16140], Loss: 0.9452, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [209/16140], Loss: 1.1711, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [210/16140], Loss: 1.1543, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [211/16140], Loss: 0.8016, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [212/16140], Loss: 1.1361, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [213/16140], Loss: 0.8457, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [214/16140], Loss: 1.0590, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [215/16140], Loss: 0.9885, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [216/16140], Loss: 0.8395, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [217/16140], Loss: 0.7672, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [218/16140], Loss: 1.2249, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [219/16140], Loss: 0.7832, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [220/16140], Loss: 1.0063, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [221/16140], Loss: 1.1455, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [222/16140], Loss: 1.2109, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [223/16140], Loss: 0.6235, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [224/16140], Loss: 1.4587, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [225/16140], Loss: 1.6751, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [226/16140], Loss: 0.8917, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [227/16140], Loss: 1.2629, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [228/16140], Loss: 1.0908, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [229/16140], Loss: 0.6555, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [230/16140], Loss: 1.1652, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [231/16140], Loss: 0.8141, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [232/16140], Loss: 0.9175, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [233/16140], Loss: 0.9326, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [234/16140], Loss: 1.3939, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [235/16140], Loss: 0.8853, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [236/16140], Loss: 1.2510, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [237/16140], Loss: 0.8912, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [238/16140], Loss: 1.1855, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [239/16140], Loss: 1.1041, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [240/16140], Loss: 1.0508, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [241/16140], Loss: 1.6277, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [242/16140], Loss: 1.0986, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [243/16140], Loss: 1.5577, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [244/16140], Loss: 0.9407, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [245/16140], Loss: 1.3957, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [246/16140], Loss: 1.0964, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [247/16140], Loss: 1.0307, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [248/16140], Loss: 1.1503, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [249/16140], Loss: 0.9577, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [250/16140], Loss: 0.9525, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [251/16140], Loss: 0.8759, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [252/16140], Loss: 0.9957, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [253/16140], Loss: 1.0735, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [254/16140], Loss: 0.8359, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [255/16140], Loss: 0.8156, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [256/16140], Loss: 0.9228, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [257/16140], Loss: 0.7008, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [258/16140], Loss: 0.7358, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [259/16140], Loss: 0.8754, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [260/16140], Loss: 1.0480, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [261/16140], Loss: 1.3659, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [262/16140], Loss: 1.0523, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [263/16140], Loss: 0.7909, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [264/16140], Loss: 0.6632, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [265/16140], Loss: 0.8485, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [266/16140], Loss: 1.2559, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [267/16140], Loss: 0.8315, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [268/16140], Loss: 0.9865, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [269/16140], Loss: 1.0950, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [270/16140], Loss: 1.1559, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [271/16140], Loss: 0.8363, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [272/16140], Loss: 0.8474, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [273/16140], Loss: 0.9759, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [274/16140], Loss: 1.2186, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [275/16140], Loss: 1.1509, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [276/16140], Loss: 1.3177, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [277/16140], Loss: 0.8603, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [278/16140], Loss: 1.3866, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [279/16140], Loss: 1.0992, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [280/16140], Loss: 1.1021, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [281/16140], Loss: 1.3743, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [282/16140], Loss: 1.3816, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [283/16140], Loss: 1.0457, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [284/16140], Loss: 0.9047, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [285/16140], Loss: 1.0781, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [286/16140], Loss: 1.0818, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [287/16140], Loss: 0.8464, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [288/16140], Loss: 0.8986, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [289/16140], Loss: 1.2630, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [290/16140], Loss: 1.3605, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [291/16140], Loss: 1.1776, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [292/16140], Loss: 1.2659, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [293/16140], Loss: 0.9781, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [294/16140], Loss: 0.9008, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [295/16140], Loss: 0.8443, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [296/16140], Loss: 1.2539, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [297/16140], Loss: 1.4841, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [298/16140], Loss: 1.2266, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [299/16140], Loss: 1.1563, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [300/16140], Loss: 0.8009, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [301/16140], Loss: 1.1859, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [302/16140], Loss: 1.0921, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [303/16140], Loss: 1.2851, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [304/16140], Loss: 0.9479, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [305/16140], Loss: 1.0959, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [306/16140], Loss: 1.3240, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [307/16140], Loss: 1.0083, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [308/16140], Loss: 0.9610, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [309/16140], Loss: 1.1918, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [310/16140], Loss: 1.0498, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [311/16140], Loss: 1.0891, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [312/16140], Loss: 0.8439, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [313/16140], Loss: 1.0045, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [314/16140], Loss: 0.9717, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [315/16140], Loss: 1.1079, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [316/16140], Loss: 0.8792, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [317/16140], Loss: 0.8382, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [318/16140], Loss: 0.9674, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [319/16140], Loss: 0.9137, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [320/16140], Loss: 0.6774, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [321/16140], Loss: 0.8906, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [322/16140], Loss: 0.8899, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [323/16140], Loss: 1.0362, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [324/16140], Loss: 1.2397, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [325/16140], Loss: 1.2214, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [326/16140], Loss: 1.4430, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [327/16140], Loss: 1.1394, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [328/16140], Loss: 0.8480, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [329/16140], Loss: 1.1516, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [330/16140], Loss: 1.5324, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [331/16140], Loss: 1.0508, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [332/16140], Loss: 1.1213, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [333/16140], Loss: 0.8990, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [334/16140], Loss: 1.2291, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [335/16140], Loss: 0.7619, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [336/16140], Loss: 1.4099, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [337/16140], Loss: 1.1621, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [338/16140], Loss: 1.0818, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [339/16140], Loss: 1.2006, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [340/16140], Loss: 1.4142, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [341/16140], Loss: 1.3187, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [342/16140], Loss: 1.1442, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [343/16140], Loss: 1.3952, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [344/16140], Loss: 0.8364, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [345/16140], Loss: 0.8662, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [346/16140], Loss: 0.8719, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [347/16140], Loss: 1.0436, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [348/16140], Loss: 1.0048, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [349/16140], Loss: 0.8657, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [350/16140], Loss: 0.8137, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [351/16140], Loss: 0.9168, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [352/16140], Loss: 1.4189, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [353/16140], Loss: 0.9972, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [354/16140], Loss: 1.1867, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [355/16140], Loss: 0.9184, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [356/16140], Loss: 1.4740, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [357/16140], Loss: 1.1379, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [358/16140], Loss: 1.0904, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [359/16140], Loss: 1.2380, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [360/16140], Loss: 1.0607, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [361/16140], Loss: 1.0211, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [362/16140], Loss: 1.2867, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [363/16140], Loss: 1.1590, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [364/16140], Loss: 1.0494, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [365/16140], Loss: 1.1270, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [366/16140], Loss: 1.0194, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [367/16140], Loss: 1.3411, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [368/16140], Loss: 1.1682, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [369/16140], Loss: 1.2855, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [370/16140], Loss: 1.0598, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [371/16140], Loss: 0.9600, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [372/16140], Loss: 0.9786, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [373/16140], Loss: 0.8145, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [374/16140], Loss: 0.8117, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [375/16140], Loss: 1.1910, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [376/16140], Loss: 1.0752, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [377/16140], Loss: 1.3237, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [378/16140], Loss: 1.3291, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [379/16140], Loss: 1.0166, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [380/16140], Loss: 1.1784, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [381/16140], Loss: 1.1038, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [382/16140], Loss: 1.0658, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [383/16140], Loss: 0.7926, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [384/16140], Loss: 0.9662, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [385/16140], Loss: 1.5612, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [386/16140], Loss: 1.0092, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [387/16140], Loss: 1.3005, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [388/16140], Loss: 1.0204, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [389/16140], Loss: 0.7968, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [390/16140], Loss: 0.7639, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [391/16140], Loss: 1.0827, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [392/16140], Loss: 1.2524, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [393/16140], Loss: 1.5391, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [394/16140], Loss: 1.2486, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [395/16140], Loss: 0.6576, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [396/16140], Loss: 1.1957, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [397/16140], Loss: 1.6686, Accuracy: 26.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [398/16140], Loss: 1.1034, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [399/16140], Loss: 0.9543, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [400/16140], Loss: 1.0552, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [401/16140], Loss: 0.9526, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [402/16140], Loss: 1.3754, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [403/16140], Loss: 0.9447, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [404/16140], Loss: 1.0229, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [405/16140], Loss: 1.8171, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [406/16140], Loss: 1.1055, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [407/16140], Loss: 1.1773, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [408/16140], Loss: 0.8512, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [409/16140], Loss: 1.1628, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [410/16140], Loss: 1.0852, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [411/16140], Loss: 1.0107, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [412/16140], Loss: 1.4085, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [413/16140], Loss: 1.2740, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [414/16140], Loss: 1.1286, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [415/16140], Loss: 0.9102, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [416/16140], Loss: 0.9149, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [417/16140], Loss: 1.1274, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [418/16140], Loss: 0.9123, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [419/16140], Loss: 1.2406, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [420/16140], Loss: 1.3434, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [421/16140], Loss: 1.1327, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [422/16140], Loss: 1.1363, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [423/16140], Loss: 1.0260, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [424/16140], Loss: 1.2374, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [425/16140], Loss: 0.9724, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [426/16140], Loss: 1.2365, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [427/16140], Loss: 0.8911, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [428/16140], Loss: 1.3012, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [429/16140], Loss: 1.1879, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [430/16140], Loss: 1.1691, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [431/16140], Loss: 1.3196, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [432/16140], Loss: 0.8845, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [433/16140], Loss: 0.8407, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [434/16140], Loss: 1.2780, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [435/16140], Loss: 1.1845, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [436/16140], Loss: 0.9115, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [437/16140], Loss: 1.4939, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [438/16140], Loss: 1.0274, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [439/16140], Loss: 1.2750, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [440/16140], Loss: 0.8604, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [441/16140], Loss: 0.8065, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [442/16140], Loss: 1.0742, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [443/16140], Loss: 0.8275, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [444/16140], Loss: 0.8765, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [445/16140], Loss: 1.2823, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [446/16140], Loss: 1.5153, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [447/16140], Loss: 1.0515, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [448/16140], Loss: 1.1078, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [449/16140], Loss: 1.1475, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [450/16140], Loss: 1.0656, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [451/16140], Loss: 0.8529, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [452/16140], Loss: 1.4062, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [453/16140], Loss: 1.0783, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [454/16140], Loss: 0.9255, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [455/16140], Loss: 0.9489, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [456/16140], Loss: 1.3993, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [457/16140], Loss: 1.1565, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [458/16140], Loss: 1.2859, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [459/16140], Loss: 1.2109, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [460/16140], Loss: 1.4968, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [461/16140], Loss: 1.0488, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [462/16140], Loss: 1.4375, Accuracy: 36.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [463/16140], Loss: 0.9457, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [464/16140], Loss: 0.9093, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [465/16140], Loss: 1.1418, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [466/16140], Loss: 0.8853, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [467/16140], Loss: 1.2116, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [468/16140], Loss: 1.0743, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [469/16140], Loss: 0.9097, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [470/16140], Loss: 1.0481, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [471/16140], Loss: 0.8577, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [472/16140], Loss: 1.2595, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [473/16140], Loss: 0.9552, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [474/16140], Loss: 1.3556, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [475/16140], Loss: 1.2021, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [476/16140], Loss: 1.3151, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [477/16140], Loss: 1.1021, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [478/16140], Loss: 0.8186, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [479/16140], Loss: 0.6216, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [480/16140], Loss: 1.1450, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [481/16140], Loss: 0.7588, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [482/16140], Loss: 1.1636, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [483/16140], Loss: 1.1087, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [484/16140], Loss: 1.2646, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [485/16140], Loss: 0.8258, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [486/16140], Loss: 1.0231, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [487/16140], Loss: 1.2828, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [488/16140], Loss: 1.0776, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [489/16140], Loss: 1.0475, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [490/16140], Loss: 0.9624, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [491/16140], Loss: 1.0778, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [492/16140], Loss: 1.2228, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [493/16140], Loss: 1.4168, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [494/16140], Loss: 0.8327, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [495/16140], Loss: 1.2364, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [496/16140], Loss: 0.9160, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [497/16140], Loss: 0.8227, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [498/16140], Loss: 1.0845, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [499/16140], Loss: 1.1633, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [500/16140], Loss: 0.9328, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [501/16140], Loss: 1.2600, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [502/16140], Loss: 1.0351, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [503/16140], Loss: 1.3836, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [504/16140], Loss: 1.1533, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [505/16140], Loss: 1.2959, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [506/16140], Loss: 1.0820, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [507/16140], Loss: 0.8971, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [508/16140], Loss: 1.1102, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [509/16140], Loss: 0.8551, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [510/16140], Loss: 1.3864, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [511/16140], Loss: 1.5010, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [512/16140], Loss: 1.2612, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [513/16140], Loss: 1.2101, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [514/16140], Loss: 1.5246, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [515/16140], Loss: 1.1143, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [516/16140], Loss: 1.1385, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [517/16140], Loss: 0.7696, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [518/16140], Loss: 1.0871, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [519/16140], Loss: 0.9350, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [520/16140], Loss: 0.9165, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [521/16140], Loss: 1.0635, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [522/16140], Loss: 1.0231, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [523/16140], Loss: 0.7823, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [524/16140], Loss: 0.9674, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [525/16140], Loss: 1.2091, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [526/16140], Loss: 1.2276, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [527/16140], Loss: 1.3244, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [528/16140], Loss: 1.0348, Accuracy: 60.00%, Iteration Time: 0.03 seconds
Epoch [8/10], Step [529/16140], Loss: 0.8578, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [530/16140], Loss: 1.0219, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [531/16140], Loss: 1.0729, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [532/16140], Loss: 1.5404, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [533/16140], Loss: 1.0469, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [534/16140], Loss: 1.0045, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [535/16140], Loss: 0.7985, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [536/16140], Loss: 1.2032, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [537/16140], Loss: 1.1272, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [538/16140], Loss: 1.0870, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [539/16140], Loss: 0.7077, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [540/16140], Loss: 1.5004, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [541/16140], Loss: 0.7318, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [542/16140], Loss: 0.9117, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [543/16140], Loss: 1.0281, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [544/16140], Loss: 0.9369, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [545/16140], Loss: 0.8913, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [546/16140], Loss: 0.7170, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [547/16140], Loss: 0.7934, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [548/16140], Loss: 1.1385, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [549/16140], Loss: 1.2003, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [550/16140], Loss: 0.8576, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [551/16140], Loss: 0.9258, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [552/16140], Loss: 1.0325, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [553/16140], Loss: 1.0975, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [554/16140], Loss: 1.1497, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [555/16140], Loss: 1.3147, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [556/16140], Loss: 0.8744, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [557/16140], Loss: 1.0016, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [558/16140], Loss: 1.1434, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [559/16140], Loss: 1.4314, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [560/16140], Loss: 1.7487, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [561/16140], Loss: 0.7514, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [562/16140], Loss: 0.8955, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [563/16140], Loss: 1.0830, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [564/16140], Loss: 0.8969, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [565/16140], Loss: 0.9182, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [566/16140], Loss: 1.2755, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [567/16140], Loss: 1.4360, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [568/16140], Loss: 1.2123, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [569/16140], Loss: 0.9981, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [570/16140], Loss: 0.9522, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [571/16140], Loss: 0.8893, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [572/16140], Loss: 0.9628, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [573/16140], Loss: 1.2714, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [574/16140], Loss: 1.0221, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [575/16140], Loss: 0.9502, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [576/16140], Loss: 1.1751, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [577/16140], Loss: 1.1717, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [578/16140], Loss: 1.3173, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [579/16140], Loss: 1.1462, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [580/16140], Loss: 1.0722, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [581/16140], Loss: 1.2735, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [582/16140], Loss: 1.5730, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [583/16140], Loss: 1.6837, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [584/16140], Loss: 1.1559, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [585/16140], Loss: 1.1325, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [586/16140], Loss: 1.1280, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [587/16140], Loss: 0.9562, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [588/16140], Loss: 1.4119, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [589/16140], Loss: 1.2419, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [590/16140], Loss: 1.1016, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [591/16140], Loss: 1.1840, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [592/16140], Loss: 1.1846, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [593/16140], Loss: 1.3503, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [594/16140], Loss: 1.1220, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [595/16140], Loss: 0.8330, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [596/16140], Loss: 1.0366, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [597/16140], Loss: 1.1405, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [598/16140], Loss: 1.1214, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [599/16140], Loss: 1.2101, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [600/16140], Loss: 1.3030, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [601/16140], Loss: 1.4450, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [602/16140], Loss: 0.9563, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [603/16140], Loss: 1.7198, Accuracy: 33.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [604/16140], Loss: 1.0532, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [605/16140], Loss: 0.8757, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [606/16140], Loss: 1.1733, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [607/16140], Loss: 1.1049, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [608/16140], Loss: 1.3472, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [609/16140], Loss: 1.0853, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [610/16140], Loss: 1.5646, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [611/16140], Loss: 0.9399, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [612/16140], Loss: 1.1396, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [613/16140], Loss: 1.0351, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [614/16140], Loss: 1.1883, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [615/16140], Loss: 0.9567, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [616/16140], Loss: 1.0775, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [617/16140], Loss: 0.9904, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [618/16140], Loss: 0.8904, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [619/16140], Loss: 0.8677, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [620/16140], Loss: 0.9774, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [621/16140], Loss: 1.2016, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [622/16140], Loss: 1.1449, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [623/16140], Loss: 0.8888, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [624/16140], Loss: 0.7260, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [625/16140], Loss: 1.4021, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [626/16140], Loss: 1.2071, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [627/16140], Loss: 0.8115, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [628/16140], Loss: 1.0145, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [629/16140], Loss: 1.0154, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [630/16140], Loss: 0.9243, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [631/16140], Loss: 1.1428, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [632/16140], Loss: 0.9388, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [633/16140], Loss: 1.2346, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [634/16140], Loss: 1.2603, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [635/16140], Loss: 0.8226, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [636/16140], Loss: 1.2067, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [637/16140], Loss: 0.9936, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [638/16140], Loss: 1.0704, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [639/16140], Loss: 1.1221, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [640/16140], Loss: 1.7462, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [641/16140], Loss: 1.2625, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [642/16140], Loss: 1.1166, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [643/16140], Loss: 1.1020, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [644/16140], Loss: 0.9140, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [645/16140], Loss: 0.9559, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [646/16140], Loss: 0.8470, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [647/16140], Loss: 0.8791, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [648/16140], Loss: 1.3763, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [649/16140], Loss: 1.0131, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [650/16140], Loss: 1.3905, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [651/16140], Loss: 1.0561, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [652/16140], Loss: 0.9599, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [653/16140], Loss: 1.1078, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [654/16140], Loss: 0.9821, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [655/16140], Loss: 1.0266, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [656/16140], Loss: 1.4501, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [657/16140], Loss: 0.9255, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [658/16140], Loss: 1.1950, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [659/16140], Loss: 0.9321, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [660/16140], Loss: 1.0831, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [661/16140], Loss: 1.1543, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [662/16140], Loss: 1.0941, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [663/16140], Loss: 1.2863, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [664/16140], Loss: 1.3648, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [665/16140], Loss: 1.0569, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [666/16140], Loss: 1.2463, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [667/16140], Loss: 1.1594, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [668/16140], Loss: 1.3969, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [669/16140], Loss: 1.1310, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [670/16140], Loss: 1.5595, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [671/16140], Loss: 1.0178, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [672/16140], Loss: 1.0467, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [673/16140], Loss: 1.0520, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [674/16140], Loss: 1.0905, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [675/16140], Loss: 0.9625, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [676/16140], Loss: 1.2202, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [677/16140], Loss: 1.0603, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [678/16140], Loss: 1.2634, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [679/16140], Loss: 1.0376, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [680/16140], Loss: 0.9409, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [681/16140], Loss: 0.7738, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [682/16140], Loss: 0.8966, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [683/16140], Loss: 0.9738, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [684/16140], Loss: 0.7783, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [685/16140], Loss: 0.7046, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [686/16140], Loss: 1.0823, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [687/16140], Loss: 0.8617, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [688/16140], Loss: 1.0077, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [689/16140], Loss: 1.0224, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [690/16140], Loss: 0.9832, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [691/16140], Loss: 1.0567, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [692/16140], Loss: 1.0178, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [693/16140], Loss: 0.5852, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [694/16140], Loss: 1.3319, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [695/16140], Loss: 0.8825, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [696/16140], Loss: 0.6786, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [697/16140], Loss: 1.1866, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [698/16140], Loss: 1.4072, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [699/16140], Loss: 1.1826, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [700/16140], Loss: 0.8438, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [701/16140], Loss: 1.0606, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [702/16140], Loss: 1.4770, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [703/16140], Loss: 1.0566, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [704/16140], Loss: 1.0789, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [705/16140], Loss: 1.1953, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [706/16140], Loss: 0.9453, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [707/16140], Loss: 1.0215, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [708/16140], Loss: 1.3312, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [709/16140], Loss: 1.0275, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [710/16140], Loss: 1.0658, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [711/16140], Loss: 1.1897, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [712/16140], Loss: 1.2788, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [713/16140], Loss: 1.5053, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [714/16140], Loss: 1.0083, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [715/16140], Loss: 0.9304, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [716/16140], Loss: 1.0621, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [717/16140], Loss: 1.4486, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [718/16140], Loss: 0.9850, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [719/16140], Loss: 0.7264, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [720/16140], Loss: 0.8289, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [721/16140], Loss: 0.9351, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [722/16140], Loss: 1.5417, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [723/16140], Loss: 1.0573, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [724/16140], Loss: 1.4757, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [725/16140], Loss: 1.0739, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [726/16140], Loss: 0.8552, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [727/16140], Loss: 1.1315, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [728/16140], Loss: 0.9541, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [729/16140], Loss: 1.1367, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [730/16140], Loss: 1.0144, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [731/16140], Loss: 0.8177, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [732/16140], Loss: 1.1501, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [733/16140], Loss: 1.1357, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [734/16140], Loss: 1.3854, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [735/16140], Loss: 1.1736, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [736/16140], Loss: 1.2658, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [737/16140], Loss: 1.7633, Accuracy: 36.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [738/16140], Loss: 1.2138, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [739/16140], Loss: 1.1322, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [740/16140], Loss: 0.7671, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [741/16140], Loss: 1.2681, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [742/16140], Loss: 1.1291, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [743/16140], Loss: 0.9611, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [744/16140], Loss: 0.7095, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [745/16140], Loss: 1.2726, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [746/16140], Loss: 0.9099, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [747/16140], Loss: 1.2658, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [748/16140], Loss: 1.1511, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [749/16140], Loss: 1.0235, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [750/16140], Loss: 0.8932, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [751/16140], Loss: 1.1284, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [752/16140], Loss: 1.1413, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [753/16140], Loss: 1.2841, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [754/16140], Loss: 1.1324, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [755/16140], Loss: 1.3172, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [756/16140], Loss: 0.9790, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [757/16140], Loss: 1.1225, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [758/16140], Loss: 0.9463, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [759/16140], Loss: 1.3649, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [760/16140], Loss: 1.1568, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [761/16140], Loss: 1.4706, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [762/16140], Loss: 1.2486, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [763/16140], Loss: 1.0746, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [764/16140], Loss: 1.0080, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [765/16140], Loss: 0.9958, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [766/16140], Loss: 1.2340, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [767/16140], Loss: 1.2024, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [768/16140], Loss: 0.8371, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [769/16140], Loss: 0.9455, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [770/16140], Loss: 0.8090, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [771/16140], Loss: 0.9777, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [772/16140], Loss: 1.0450, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [773/16140], Loss: 0.8932, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [774/16140], Loss: 0.8711, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [775/16140], Loss: 1.0060, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [776/16140], Loss: 0.9737, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [777/16140], Loss: 1.0930, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [778/16140], Loss: 0.8367, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [779/16140], Loss: 0.9080, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [780/16140], Loss: 0.5924, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [781/16140], Loss: 1.1665, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [782/16140], Loss: 0.7004, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [783/16140], Loss: 0.9336, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [784/16140], Loss: 1.1939, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [785/16140], Loss: 0.6805, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [786/16140], Loss: 1.2996, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [787/16140], Loss: 1.2856, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [788/16140], Loss: 0.9305, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [789/16140], Loss: 0.9174, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [790/16140], Loss: 1.3230, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [791/16140], Loss: 1.0622, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [792/16140], Loss: 1.1150, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [793/16140], Loss: 1.0434, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [794/16140], Loss: 1.0329, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [795/16140], Loss: 1.2367, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [796/16140], Loss: 1.2340, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [797/16140], Loss: 0.8947, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [798/16140], Loss: 1.0343, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [799/16140], Loss: 1.1319, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [800/16140], Loss: 0.8520, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [801/16140], Loss: 0.6978, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [802/16140], Loss: 1.5456, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [803/16140], Loss: 0.6437, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [804/16140], Loss: 1.0219, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [805/16140], Loss: 1.0422, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [806/16140], Loss: 0.9994, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [807/16140], Loss: 1.4589, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [808/16140], Loss: 0.9243, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [809/16140], Loss: 0.6583, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [810/16140], Loss: 0.7874, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [811/16140], Loss: 1.3803, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [812/16140], Loss: 0.7255, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [813/16140], Loss: 0.8051, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [814/16140], Loss: 1.1246, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [815/16140], Loss: 0.9478, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [816/16140], Loss: 0.8603, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [817/16140], Loss: 1.7458, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [818/16140], Loss: 0.9939, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [819/16140], Loss: 0.9918, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [820/16140], Loss: 1.2450, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [821/16140], Loss: 1.2567, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [822/16140], Loss: 0.8831, Accuracy: 66.67%, Iteration Time: 0.03 seconds
Epoch [8/10], Step [823/16140], Loss: 0.8158, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [824/16140], Loss: 1.3021, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [825/16140], Loss: 1.0606, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [826/16140], Loss: 1.2944, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [827/16140], Loss: 0.9848, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [828/16140], Loss: 1.0847, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [829/16140], Loss: 1.4714, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [830/16140], Loss: 1.3755, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [831/16140], Loss: 1.0136, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [832/16140], Loss: 1.1115, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [833/16140], Loss: 0.9358, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [834/16140], Loss: 1.0640, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [835/16140], Loss: 0.8873, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [836/16140], Loss: 1.5633, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [837/16140], Loss: 1.2775, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [838/16140], Loss: 1.1310, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [839/16140], Loss: 0.9996, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [840/16140], Loss: 0.9272, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [841/16140], Loss: 1.0707, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [842/16140], Loss: 1.2638, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [843/16140], Loss: 1.2916, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [844/16140], Loss: 0.8835, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [845/16140], Loss: 0.8911, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [846/16140], Loss: 1.1799, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [847/16140], Loss: 1.0932, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [848/16140], Loss: 1.0743, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [849/16140], Loss: 1.0879, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [850/16140], Loss: 1.0580, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [851/16140], Loss: 0.9835, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [852/16140], Loss: 1.1357, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [853/16140], Loss: 1.2134, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [854/16140], Loss: 0.7601, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [855/16140], Loss: 1.2971, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [856/16140], Loss: 1.1078, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [857/16140], Loss: 1.0526, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [858/16140], Loss: 1.1172, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [859/16140], Loss: 1.2693, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [860/16140], Loss: 0.9814, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [861/16140], Loss: 1.0399, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [862/16140], Loss: 1.0242, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [863/16140], Loss: 0.8972, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [864/16140], Loss: 1.0963, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [865/16140], Loss: 1.5865, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [866/16140], Loss: 1.2155, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [867/16140], Loss: 0.8046, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [868/16140], Loss: 0.9194, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [869/16140], Loss: 0.8897, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [870/16140], Loss: 1.0627, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [871/16140], Loss: 1.0396, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [872/16140], Loss: 1.0873, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [873/16140], Loss: 0.8399, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [874/16140], Loss: 1.0421, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [875/16140], Loss: 1.1182, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [876/16140], Loss: 0.9891, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [877/16140], Loss: 1.5729, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [878/16140], Loss: 0.9175, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [879/16140], Loss: 1.2383, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [880/16140], Loss: 0.9613, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [881/16140], Loss: 0.8636, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [882/16140], Loss: 1.0530, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [883/16140], Loss: 1.1228, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [884/16140], Loss: 1.1538, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [885/16140], Loss: 1.4058, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [886/16140], Loss: 1.2547, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [887/16140], Loss: 1.2570, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [888/16140], Loss: 1.2900, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [889/16140], Loss: 0.9725, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [890/16140], Loss: 0.5884, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [891/16140], Loss: 1.0886, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [892/16140], Loss: 1.0169, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [893/16140], Loss: 1.4882, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [894/16140], Loss: 1.0458, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [895/16140], Loss: 1.2142, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [896/16140], Loss: 1.3016, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [897/16140], Loss: 1.1727, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [898/16140], Loss: 0.9423, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [899/16140], Loss: 1.3332, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [900/16140], Loss: 1.0852, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [901/16140], Loss: 0.8012, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [902/16140], Loss: 1.1248, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [903/16140], Loss: 0.9530, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [904/16140], Loss: 1.0118, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [905/16140], Loss: 1.1660, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [906/16140], Loss: 1.2894, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [907/16140], Loss: 1.1940, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [908/16140], Loss: 1.1285, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [909/16140], Loss: 0.9735, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [910/16140], Loss: 0.6781, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [911/16140], Loss: 0.9558, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [912/16140], Loss: 0.9530, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [913/16140], Loss: 1.1137, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [914/16140], Loss: 1.1256, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [915/16140], Loss: 1.1213, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [916/16140], Loss: 0.9651, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [917/16140], Loss: 1.2436, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [918/16140], Loss: 1.3218, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [919/16140], Loss: 0.8789, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [920/16140], Loss: 1.2640, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [921/16140], Loss: 0.9308, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [922/16140], Loss: 1.0783, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [923/16140], Loss: 0.9833, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [924/16140], Loss: 1.1262, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [925/16140], Loss: 0.9247, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [926/16140], Loss: 1.2807, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [927/16140], Loss: 1.2712, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [928/16140], Loss: 0.8807, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [929/16140], Loss: 0.8254, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [930/16140], Loss: 1.2191, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [931/16140], Loss: 1.0143, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [932/16140], Loss: 1.1674, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [933/16140], Loss: 0.8170, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [934/16140], Loss: 1.3058, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [935/16140], Loss: 1.0370, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [936/16140], Loss: 1.1781, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [937/16140], Loss: 0.8119, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [938/16140], Loss: 1.3315, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [939/16140], Loss: 0.8301, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [940/16140], Loss: 0.9197, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [941/16140], Loss: 0.8520, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [942/16140], Loss: 0.8869, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [943/16140], Loss: 0.7732, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [944/16140], Loss: 1.2335, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [945/16140], Loss: 0.7932, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [946/16140], Loss: 1.1119, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [947/16140], Loss: 0.6320, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [948/16140], Loss: 1.1296, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [949/16140], Loss: 1.6624, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [950/16140], Loss: 1.2257, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [951/16140], Loss: 1.1135, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [952/16140], Loss: 0.9413, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [953/16140], Loss: 1.0140, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [954/16140], Loss: 0.8761, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [955/16140], Loss: 1.3383, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [956/16140], Loss: 1.2183, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [957/16140], Loss: 1.2051, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [958/16140], Loss: 1.0421, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [959/16140], Loss: 1.3871, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [960/16140], Loss: 1.2931, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [961/16140], Loss: 0.7247, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [962/16140], Loss: 1.1665, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [963/16140], Loss: 1.2346, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [964/16140], Loss: 0.8495, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [965/16140], Loss: 1.3845, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [966/16140], Loss: 0.9868, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [967/16140], Loss: 1.2751, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [968/16140], Loss: 1.0095, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [969/16140], Loss: 1.6938, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [970/16140], Loss: 0.9743, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [971/16140], Loss: 0.8835, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [972/16140], Loss: 1.4451, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [973/16140], Loss: 0.8371, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [974/16140], Loss: 0.7938, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [975/16140], Loss: 0.7253, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [976/16140], Loss: 0.7435, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [977/16140], Loss: 1.5786, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [978/16140], Loss: 0.7386, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [979/16140], Loss: 1.0033, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [980/16140], Loss: 1.1671, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [981/16140], Loss: 1.2484, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [982/16140], Loss: 0.9736, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [983/16140], Loss: 1.1916, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [984/16140], Loss: 1.3703, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [985/16140], Loss: 0.7924, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [986/16140], Loss: 1.0818, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [987/16140], Loss: 1.3307, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [988/16140], Loss: 1.0743, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [989/16140], Loss: 1.1742, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [990/16140], Loss: 1.2438, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [991/16140], Loss: 0.9351, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [992/16140], Loss: 0.9275, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [993/16140], Loss: 1.3433, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [994/16140], Loss: 1.1954, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [995/16140], Loss: 1.1653, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [996/16140], Loss: 1.0839, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [997/16140], Loss: 0.7332, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [998/16140], Loss: 1.1083, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [999/16140], Loss: 1.1067, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1000/16140], Loss: 1.2580, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1001/16140], Loss: 1.1797, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1002/16140], Loss: 1.2897, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1003/16140], Loss: 1.3039, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1004/16140], Loss: 1.5490, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1005/16140], Loss: 1.2150, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1006/16140], Loss: 1.3138, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1007/16140], Loss: 1.3371, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1008/16140], Loss: 0.9822, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1009/16140], Loss: 0.9159, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1010/16140], Loss: 0.9095, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1011/16140], Loss: 1.0272, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1012/16140], Loss: 1.4212, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1013/16140], Loss: 1.0654, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1014/16140], Loss: 1.1539, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1015/16140], Loss: 1.1043, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1016/16140], Loss: 1.2877, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1017/16140], Loss: 1.2411, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1018/16140], Loss: 0.9048, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1019/16140], Loss: 1.3279, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1020/16140], Loss: 1.4827, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1021/16140], Loss: 0.9341, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1022/16140], Loss: 1.0949, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1023/16140], Loss: 1.3748, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1024/16140], Loss: 1.3118, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1025/16140], Loss: 1.1806, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1026/16140], Loss: 1.0628, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1027/16140], Loss: 0.9206, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1028/16140], Loss: 1.1647, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1029/16140], Loss: 1.1719, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1030/16140], Loss: 1.2250, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1031/16140], Loss: 1.0663, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1032/16140], Loss: 1.1524, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1033/16140], Loss: 0.9586, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1034/16140], Loss: 1.5457, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1035/16140], Loss: 0.9247, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1036/16140], Loss: 1.0364, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1037/16140], Loss: 1.1229, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1038/16140], Loss: 0.9484, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1039/16140], Loss: 0.9483, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1040/16140], Loss: 0.7977, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1041/16140], Loss: 1.1781, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1042/16140], Loss: 1.0131, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1043/16140], Loss: 1.1828, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1044/16140], Loss: 1.1368, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1045/16140], Loss: 0.9989, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1046/16140], Loss: 1.3412, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1047/16140], Loss: 1.1600, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1048/16140], Loss: 1.3230, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1049/16140], Loss: 1.0172, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1050/16140], Loss: 1.5116, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1051/16140], Loss: 0.8687, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1052/16140], Loss: 1.1202, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1053/16140], Loss: 0.8109, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1054/16140], Loss: 1.4748, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1055/16140], Loss: 1.4133, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1056/16140], Loss: 0.8114, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1057/16140], Loss: 1.2350, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1058/16140], Loss: 1.0966, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1059/16140], Loss: 1.0373, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1060/16140], Loss: 0.9635, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1061/16140], Loss: 0.7468, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1062/16140], Loss: 1.3314, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1063/16140], Loss: 0.8654, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1064/16140], Loss: 0.6893, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1065/16140], Loss: 1.0488, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1066/16140], Loss: 1.3622, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1067/16140], Loss: 1.2075, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1068/16140], Loss: 0.9616, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1069/16140], Loss: 0.8378, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1070/16140], Loss: 1.2223, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1071/16140], Loss: 1.0846, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1072/16140], Loss: 1.1403, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1073/16140], Loss: 1.0130, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1074/16140], Loss: 0.9559, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1075/16140], Loss: 1.1647, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1076/16140], Loss: 1.0806, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1077/16140], Loss: 1.3449, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1078/16140], Loss: 1.0960, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1079/16140], Loss: 0.9174, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1080/16140], Loss: 1.2793, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1081/16140], Loss: 1.1556, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1082/16140], Loss: 1.2020, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1083/16140], Loss: 1.5122, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1084/16140], Loss: 1.5107, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1085/16140], Loss: 1.0887, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1086/16140], Loss: 1.3606, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1087/16140], Loss: 1.0902, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1088/16140], Loss: 0.9745, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1089/16140], Loss: 1.1312, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1090/16140], Loss: 1.1432, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1091/16140], Loss: 0.8884, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1092/16140], Loss: 0.6836, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1093/16140], Loss: 0.8689, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1094/16140], Loss: 0.9594, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1095/16140], Loss: 1.5798, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1096/16140], Loss: 1.4208, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1097/16140], Loss: 0.8558, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1098/16140], Loss: 0.9663, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1099/16140], Loss: 1.0856, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1100/16140], Loss: 1.3506, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1101/16140], Loss: 1.2900, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1102/16140], Loss: 1.3233, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1103/16140], Loss: 1.3538, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1104/16140], Loss: 0.9491, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1105/16140], Loss: 1.1997, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1106/16140], Loss: 1.1810, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1107/16140], Loss: 1.1566, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1108/16140], Loss: 1.3163, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1109/16140], Loss: 1.1577, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1110/16140], Loss: 1.2711, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1111/16140], Loss: 0.8225, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1112/16140], Loss: 1.0945, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1113/16140], Loss: 0.8925, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1114/16140], Loss: 1.0593, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1115/16140], Loss: 0.9235, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1116/16140], Loss: 1.2254, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1117/16140], Loss: 0.9465, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1118/16140], Loss: 1.2161, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1119/16140], Loss: 0.9654, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1120/16140], Loss: 0.9057, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1121/16140], Loss: 1.2236, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1122/16140], Loss: 1.1796, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1123/16140], Loss: 1.1208, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1124/16140], Loss: 0.9837, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1125/16140], Loss: 0.9145, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1126/16140], Loss: 1.0963, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1127/16140], Loss: 0.8805, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1128/16140], Loss: 1.1003, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1129/16140], Loss: 1.4449, Accuracy: 33.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1130/16140], Loss: 0.9843, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1131/16140], Loss: 0.8202, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1132/16140], Loss: 1.1246, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1133/16140], Loss: 1.3279, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1134/16140], Loss: 1.1012, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1135/16140], Loss: 1.1437, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1136/16140], Loss: 1.6412, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1137/16140], Loss: 1.4880, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1138/16140], Loss: 0.8304, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1139/16140], Loss: 1.2778, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1140/16140], Loss: 1.1073, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1141/16140], Loss: 1.0992, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1142/16140], Loss: 1.4161, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1143/16140], Loss: 0.9081, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1144/16140], Loss: 1.0675, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1145/16140], Loss: 1.0098, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1146/16140], Loss: 1.1037, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1147/16140], Loss: 1.2952, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1148/16140], Loss: 0.6722, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1149/16140], Loss: 1.0525, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1150/16140], Loss: 1.0284, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1151/16140], Loss: 0.9362, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1152/16140], Loss: 1.2722, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1153/16140], Loss: 0.9792, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1154/16140], Loss: 1.0828, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1155/16140], Loss: 1.0712, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1156/16140], Loss: 1.4716, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1157/16140], Loss: 1.0445, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1158/16140], Loss: 1.0720, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1159/16140], Loss: 0.7876, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1160/16140], Loss: 1.0369, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1161/16140], Loss: 0.9770, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1162/16140], Loss: 0.9226, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1163/16140], Loss: 0.9982, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1164/16140], Loss: 1.2832, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1165/16140], Loss: 1.2540, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1166/16140], Loss: 1.2906, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1167/16140], Loss: 1.0556, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1168/16140], Loss: 1.5213, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1169/16140], Loss: 0.7935, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1170/16140], Loss: 0.9028, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1171/16140], Loss: 1.6267, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1172/16140], Loss: 1.2853, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1173/16140], Loss: 1.0566, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1174/16140], Loss: 0.9497, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1175/16140], Loss: 0.9747, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1176/16140], Loss: 1.1091, Accuracy: 60.00%, Iteration Time: 0.03 seconds
Epoch [8/10], Step [1177/16140], Loss: 0.8682, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1178/16140], Loss: 1.1602, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1179/16140], Loss: 0.7434, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1180/16140], Loss: 1.0680, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1181/16140], Loss: 0.9529, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1182/16140], Loss: 1.1810, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1183/16140], Loss: 1.1551, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1184/16140], Loss: 0.8863, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1185/16140], Loss: 1.0022, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1186/16140], Loss: 0.9792, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1187/16140], Loss: 0.6237, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1188/16140], Loss: 1.0142, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1189/16140], Loss: 1.0512, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1190/16140], Loss: 0.5625, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1191/16140], Loss: 0.8863, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1192/16140], Loss: 0.8566, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1193/16140], Loss: 1.3206, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1194/16140], Loss: 1.5542, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1195/16140], Loss: 0.8666, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1196/16140], Loss: 1.2793, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1197/16140], Loss: 0.9685, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1198/16140], Loss: 1.3084, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1199/16140], Loss: 1.1815, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1200/16140], Loss: 0.9917, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1201/16140], Loss: 1.0437, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1202/16140], Loss: 1.0043, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1203/16140], Loss: 1.0770, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1204/16140], Loss: 1.1863, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1205/16140], Loss: 1.0466, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1206/16140], Loss: 1.0233, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1207/16140], Loss: 1.1300, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1208/16140], Loss: 0.8224, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1209/16140], Loss: 1.1392, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1210/16140], Loss: 1.2063, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1211/16140], Loss: 1.3123, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1212/16140], Loss: 1.2935, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1213/16140], Loss: 1.4059, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1214/16140], Loss: 1.1277, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1215/16140], Loss: 1.1392, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1216/16140], Loss: 0.9957, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1217/16140], Loss: 1.0142, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1218/16140], Loss: 1.7033, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1219/16140], Loss: 0.9715, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1220/16140], Loss: 1.3856, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1221/16140], Loss: 1.0148, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1222/16140], Loss: 1.1866, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1223/16140], Loss: 1.0741, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1224/16140], Loss: 0.7963, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1225/16140], Loss: 0.9107, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1226/16140], Loss: 0.9266, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1227/16140], Loss: 1.4201, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1228/16140], Loss: 1.5896, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1229/16140], Loss: 1.2124, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1230/16140], Loss: 1.0397, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1231/16140], Loss: 0.8766, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1232/16140], Loss: 0.9389, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1233/16140], Loss: 1.1113, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1234/16140], Loss: 0.9997, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1235/16140], Loss: 1.2935, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1236/16140], Loss: 1.0633, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1237/16140], Loss: 1.0887, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1238/16140], Loss: 0.8283, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1239/16140], Loss: 0.9547, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1240/16140], Loss: 0.9119, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1241/16140], Loss: 0.6631, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1242/16140], Loss: 0.9266, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1243/16140], Loss: 0.8296, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1244/16140], Loss: 1.1534, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1245/16140], Loss: 1.3254, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1246/16140], Loss: 1.6660, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1247/16140], Loss: 0.9521, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1248/16140], Loss: 0.9775, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1249/16140], Loss: 1.0535, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1250/16140], Loss: 0.7099, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1251/16140], Loss: 0.9864, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1252/16140], Loss: 0.9144, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1253/16140], Loss: 1.1745, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1254/16140], Loss: 1.5722, Accuracy: 30.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1255/16140], Loss: 0.9604, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1256/16140], Loss: 0.7557, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1257/16140], Loss: 1.2055, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1258/16140], Loss: 1.3605, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1259/16140], Loss: 0.9438, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1260/16140], Loss: 1.3841, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1261/16140], Loss: 0.8696, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1262/16140], Loss: 1.4472, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1263/16140], Loss: 1.2062, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1264/16140], Loss: 1.0569, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1265/16140], Loss: 0.8977, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1266/16140], Loss: 0.6595, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1267/16140], Loss: 0.9588, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1268/16140], Loss: 1.1706, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1269/16140], Loss: 1.1860, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1270/16140], Loss: 0.7702, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1271/16140], Loss: 1.4400, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1272/16140], Loss: 0.8100, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1273/16140], Loss: 1.0656, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1274/16140], Loss: 1.1976, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1275/16140], Loss: 0.7902, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1276/16140], Loss: 1.0314, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1277/16140], Loss: 1.0697, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1278/16140], Loss: 0.8279, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1279/16140], Loss: 1.0464, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1280/16140], Loss: 0.7218, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1281/16140], Loss: 1.2363, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1282/16140], Loss: 1.1277, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1283/16140], Loss: 0.9528, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1284/16140], Loss: 1.2529, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1285/16140], Loss: 1.2886, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1286/16140], Loss: 1.1690, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1287/16140], Loss: 1.2282, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1288/16140], Loss: 1.3113, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1289/16140], Loss: 1.0388, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1290/16140], Loss: 1.0263, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1291/16140], Loss: 1.3026, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1292/16140], Loss: 1.5411, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1293/16140], Loss: 0.8476, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1294/16140], Loss: 0.8485, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1295/16140], Loss: 0.7061, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1296/16140], Loss: 1.2299, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1297/16140], Loss: 1.1738, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1298/16140], Loss: 0.8543, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1299/16140], Loss: 0.8713, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1300/16140], Loss: 1.0660, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1301/16140], Loss: 1.0995, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1302/16140], Loss: 1.1568, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1303/16140], Loss: 1.0446, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1304/16140], Loss: 1.2045, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1305/16140], Loss: 0.8355, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1306/16140], Loss: 1.2052, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1307/16140], Loss: 0.9876, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1308/16140], Loss: 0.9709, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1309/16140], Loss: 1.1205, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1310/16140], Loss: 1.2467, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1311/16140], Loss: 1.2186, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1312/16140], Loss: 0.9449, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1313/16140], Loss: 0.8347, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1314/16140], Loss: 1.5148, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1315/16140], Loss: 0.9458, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1316/16140], Loss: 0.9313, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1317/16140], Loss: 1.1790, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1318/16140], Loss: 1.2318, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1319/16140], Loss: 1.0545, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1320/16140], Loss: 1.0162, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1321/16140], Loss: 1.0796, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1322/16140], Loss: 1.3506, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1323/16140], Loss: 1.1124, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1324/16140], Loss: 1.0163, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1325/16140], Loss: 1.0098, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1326/16140], Loss: 1.6098, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1327/16140], Loss: 1.3804, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1328/16140], Loss: 0.9562, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1329/16140], Loss: 0.7813, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1330/16140], Loss: 1.1928, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1331/16140], Loss: 1.2721, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1332/16140], Loss: 0.9497, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1333/16140], Loss: 1.4852, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1334/16140], Loss: 0.9997, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1335/16140], Loss: 1.0403, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1336/16140], Loss: 1.1302, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1337/16140], Loss: 0.9332, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1338/16140], Loss: 1.0252, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1339/16140], Loss: 1.2986, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1340/16140], Loss: 1.1307, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1341/16140], Loss: 1.0600, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1342/16140], Loss: 1.1056, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1343/16140], Loss: 1.0144, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1344/16140], Loss: 1.0754, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1345/16140], Loss: 1.1235, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1346/16140], Loss: 0.9539, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1347/16140], Loss: 0.9319, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1348/16140], Loss: 1.0296, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1349/16140], Loss: 1.4270, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1350/16140], Loss: 0.9750, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1351/16140], Loss: 0.9408, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1352/16140], Loss: 1.2981, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1353/16140], Loss: 1.5727, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1354/16140], Loss: 1.2093, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1355/16140], Loss: 1.2343, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1356/16140], Loss: 0.9468, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1357/16140], Loss: 1.1555, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1358/16140], Loss: 0.9939, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1359/16140], Loss: 1.4941, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1360/16140], Loss: 1.1322, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1361/16140], Loss: 1.1943, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1362/16140], Loss: 1.0104, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1363/16140], Loss: 1.3363, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1364/16140], Loss: 0.9486, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1365/16140], Loss: 1.0857, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1366/16140], Loss: 1.2522, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1367/16140], Loss: 1.3672, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1368/16140], Loss: 1.1218, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1369/16140], Loss: 1.4703, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1370/16140], Loss: 1.6472, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1371/16140], Loss: 1.1248, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1372/16140], Loss: 1.2366, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1373/16140], Loss: 1.3619, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1374/16140], Loss: 0.9272, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1375/16140], Loss: 0.8948, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1376/16140], Loss: 1.0941, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1377/16140], Loss: 1.0489, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1378/16140], Loss: 0.9448, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1379/16140], Loss: 0.8053, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1380/16140], Loss: 0.8611, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1381/16140], Loss: 1.1746, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1382/16140], Loss: 1.0185, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1383/16140], Loss: 0.9818, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1384/16140], Loss: 0.8990, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1385/16140], Loss: 0.8148, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1386/16140], Loss: 1.0237, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1387/16140], Loss: 1.2536, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1388/16140], Loss: 0.9365, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1389/16140], Loss: 0.8367, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1390/16140], Loss: 1.3433, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1391/16140], Loss: 1.1508, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1392/16140], Loss: 1.2393, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1393/16140], Loss: 1.3306, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1394/16140], Loss: 0.9840, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1395/16140], Loss: 1.1626, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1396/16140], Loss: 0.9892, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1397/16140], Loss: 1.3798, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1398/16140], Loss: 0.9790, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1399/16140], Loss: 1.2101, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1400/16140], Loss: 1.3962, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1401/16140], Loss: 1.1003, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1402/16140], Loss: 1.0877, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1403/16140], Loss: 0.9111, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1404/16140], Loss: 1.1578, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1405/16140], Loss: 1.1773, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1406/16140], Loss: 0.9956, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1407/16140], Loss: 1.3133, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1408/16140], Loss: 0.6965, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1409/16140], Loss: 0.8724, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1410/16140], Loss: 1.3962, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1411/16140], Loss: 1.2031, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1412/16140], Loss: 1.5055, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1413/16140], Loss: 1.4744, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1414/16140], Loss: 1.1475, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1415/16140], Loss: 0.9921, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1416/16140], Loss: 1.0085, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1417/16140], Loss: 1.0062, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1418/16140], Loss: 0.9735, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1419/16140], Loss: 1.2570, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1420/16140], Loss: 1.0427, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1421/16140], Loss: 1.1158, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1422/16140], Loss: 0.7684, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1423/16140], Loss: 1.2659, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1424/16140], Loss: 0.8547, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1425/16140], Loss: 1.4186, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1426/16140], Loss: 0.8927, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1427/16140], Loss: 0.7024, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1428/16140], Loss: 1.1358, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1429/16140], Loss: 0.9363, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1430/16140], Loss: 0.9506, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1431/16140], Loss: 1.3537, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1432/16140], Loss: 1.0252, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1433/16140], Loss: 1.0191, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1434/16140], Loss: 1.0175, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1435/16140], Loss: 0.8286, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1436/16140], Loss: 1.0519, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1437/16140], Loss: 1.1126, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1438/16140], Loss: 0.9908, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1439/16140], Loss: 1.0190, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1440/16140], Loss: 1.1977, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1441/16140], Loss: 1.2826, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1442/16140], Loss: 0.8985, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1443/16140], Loss: 1.5288, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1444/16140], Loss: 1.2217, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1445/16140], Loss: 1.2294, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1446/16140], Loss: 1.0291, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1447/16140], Loss: 0.9515, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1448/16140], Loss: 1.2275, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1449/16140], Loss: 1.0827, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1450/16140], Loss: 0.9338, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1451/16140], Loss: 1.0831, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1452/16140], Loss: 0.9858, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1453/16140], Loss: 0.9040, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1454/16140], Loss: 1.2538, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1455/16140], Loss: 0.8685, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1456/16140], Loss: 1.2803, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1457/16140], Loss: 0.9765, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1458/16140], Loss: 1.5291, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1459/16140], Loss: 0.9310, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1460/16140], Loss: 0.8464, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1461/16140], Loss: 1.1026, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1462/16140], Loss: 1.6960, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1463/16140], Loss: 0.9252, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1464/16140], Loss: 1.1129, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1465/16140], Loss: 1.5075, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1466/16140], Loss: 1.0446, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1467/16140], Loss: 1.0249, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1468/16140], Loss: 1.2292, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1469/16140], Loss: 0.9507, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1470/16140], Loss: 1.2348, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1471/16140], Loss: 1.0607, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1472/16140], Loss: 1.2436, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1473/16140], Loss: 1.1739, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1474/16140], Loss: 1.1621, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1475/16140], Loss: 1.5142, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1476/16140], Loss: 1.2238, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1477/16140], Loss: 1.0309, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1478/16140], Loss: 1.4340, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1479/16140], Loss: 1.2996, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1480/16140], Loss: 1.0812, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1481/16140], Loss: 1.4752, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1482/16140], Loss: 1.0862, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1483/16140], Loss: 1.4182, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1484/16140], Loss: 1.2062, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1485/16140], Loss: 1.0400, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1486/16140], Loss: 0.8050, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1487/16140], Loss: 1.3286, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1488/16140], Loss: 1.0570, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1489/16140], Loss: 1.0272, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1490/16140], Loss: 1.0627, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1491/16140], Loss: 1.1382, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1492/16140], Loss: 1.3712, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1493/16140], Loss: 1.2007, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1494/16140], Loss: 1.0412, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1495/16140], Loss: 1.1923, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1496/16140], Loss: 1.1623, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1497/16140], Loss: 1.1354, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1498/16140], Loss: 1.0446, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1499/16140], Loss: 1.1709, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1500/16140], Loss: 1.0716, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1501/16140], Loss: 1.3354, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1502/16140], Loss: 1.1489, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1503/16140], Loss: 0.8048, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1504/16140], Loss: 1.0390, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1505/16140], Loss: 1.0567, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1506/16140], Loss: 1.0014, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1507/16140], Loss: 1.0692, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1508/16140], Loss: 0.7680, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1509/16140], Loss: 1.0999, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1510/16140], Loss: 0.9496, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1511/16140], Loss: 1.0566, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1512/16140], Loss: 0.9832, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1513/16140], Loss: 0.9243, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1514/16140], Loss: 0.9026, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1515/16140], Loss: 1.1981, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1516/16140], Loss: 1.1510, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1517/16140], Loss: 1.1075, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1518/16140], Loss: 1.0557, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1519/16140], Loss: 1.3424, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1520/16140], Loss: 0.7410, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1521/16140], Loss: 1.0399, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1522/16140], Loss: 1.1902, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1523/16140], Loss: 0.9935, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1524/16140], Loss: 1.2365, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1525/16140], Loss: 1.0745, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1526/16140], Loss: 1.0220, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1527/16140], Loss: 1.4601, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1528/16140], Loss: 1.0161, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1529/16140], Loss: 0.8520, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1530/16140], Loss: 1.4740, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1531/16140], Loss: 0.7757, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1532/16140], Loss: 1.3691, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1533/16140], Loss: 0.7305, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1534/16140], Loss: 0.7396, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1535/16140], Loss: 1.1890, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1536/16140], Loss: 0.8399, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1537/16140], Loss: 1.0183, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1538/16140], Loss: 1.2484, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1539/16140], Loss: 1.0124, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1540/16140], Loss: 1.0399, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1541/16140], Loss: 1.2427, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1542/16140], Loss: 1.0311, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1543/16140], Loss: 1.5083, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1544/16140], Loss: 1.0602, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1545/16140], Loss: 1.1975, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1546/16140], Loss: 0.8915, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1547/16140], Loss: 1.1632, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1548/16140], Loss: 0.8760, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1549/16140], Loss: 0.9115, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1550/16140], Loss: 1.1023, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1551/16140], Loss: 0.8641, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1552/16140], Loss: 0.8941, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1553/16140], Loss: 0.9579, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1554/16140], Loss: 1.1579, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1555/16140], Loss: 1.1110, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1556/16140], Loss: 1.1468, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1557/16140], Loss: 1.0765, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1558/16140], Loss: 1.4684, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1559/16140], Loss: 1.2199, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1560/16140], Loss: 1.2658, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1561/16140], Loss: 0.7706, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1562/16140], Loss: 1.0420, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1563/16140], Loss: 1.1982, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1564/16140], Loss: 0.8571, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1565/16140], Loss: 0.8338, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1566/16140], Loss: 1.1531, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1567/16140], Loss: 0.8256, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1568/16140], Loss: 1.1479, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1569/16140], Loss: 1.6321, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1570/16140], Loss: 0.8874, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1571/16140], Loss: 0.9308, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1572/16140], Loss: 0.9483, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1573/16140], Loss: 1.0558, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1574/16140], Loss: 1.3478, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1575/16140], Loss: 1.0510, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1576/16140], Loss: 1.0070, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1577/16140], Loss: 0.8992, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1578/16140], Loss: 1.2129, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1579/16140], Loss: 1.0572, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1580/16140], Loss: 1.3892, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1581/16140], Loss: 1.0795, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1582/16140], Loss: 1.4627, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1583/16140], Loss: 0.9448, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1584/16140], Loss: 0.9948, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1585/16140], Loss: 1.2491, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1586/16140], Loss: 0.8099, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1587/16140], Loss: 0.6362, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1588/16140], Loss: 1.2748, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1589/16140], Loss: 0.9037, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1590/16140], Loss: 0.8361, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1591/16140], Loss: 1.1185, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1592/16140], Loss: 0.8589, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1593/16140], Loss: 0.7955, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1594/16140], Loss: 0.7582, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1595/16140], Loss: 0.7974, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1596/16140], Loss: 1.2070, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1597/16140], Loss: 1.1828, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1598/16140], Loss: 0.7098, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1599/16140], Loss: 0.9145, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1600/16140], Loss: 0.9460, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1601/16140], Loss: 0.8383, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1602/16140], Loss: 1.0885, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1603/16140], Loss: 1.0294, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1604/16140], Loss: 0.8290, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1605/16140], Loss: 1.0189, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1606/16140], Loss: 1.2474, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1607/16140], Loss: 1.3463, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1608/16140], Loss: 1.1547, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1609/16140], Loss: 1.0864, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1610/16140], Loss: 1.1059, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1611/16140], Loss: 1.0831, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1612/16140], Loss: 0.7876, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1613/16140], Loss: 0.8545, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [8/10], Step [1614/16140], Loss: 1.0123, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [8/10], Loss: 1.0909, Accuracy: 74.55%
Training finished!
Epoch [9/10], Step [1/16140], Loss: 1.1306, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [2/16140], Loss: 1.1250, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [3/16140], Loss: 0.6610, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [4/16140], Loss: 1.0844, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [5/16140], Loss: 1.0283, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [6/16140], Loss: 1.2238, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [7/16140], Loss: 0.9417, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [8/16140], Loss: 0.9761, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [9/16140], Loss: 0.7258, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [10/16140], Loss: 0.4933, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [11/16140], Loss: 1.1955, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [12/16140], Loss: 1.0939, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [13/16140], Loss: 1.1702, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [14/16140], Loss: 1.0164, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [15/16140], Loss: 1.0538, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [16/16140], Loss: 0.9334, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [17/16140], Loss: 0.9071, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [18/16140], Loss: 1.3509, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [19/16140], Loss: 1.1637, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [20/16140], Loss: 0.9599, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [21/16140], Loss: 1.0247, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [22/16140], Loss: 1.1367, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [23/16140], Loss: 0.7037, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [24/16140], Loss: 1.1682, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [25/16140], Loss: 1.7801, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [26/16140], Loss: 0.9531, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [27/16140], Loss: 0.7904, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [28/16140], Loss: 1.0159, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [29/16140], Loss: 0.8869, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [30/16140], Loss: 1.1334, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [31/16140], Loss: 0.6487, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [32/16140], Loss: 0.9883, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [33/16140], Loss: 1.1577, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [34/16140], Loss: 1.0063, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [35/16140], Loss: 1.0555, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [36/16140], Loss: 1.0727, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [37/16140], Loss: 0.9547, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [38/16140], Loss: 1.3179, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [39/16140], Loss: 0.9869, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [40/16140], Loss: 1.0142, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [41/16140], Loss: 1.0569, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [42/16140], Loss: 1.2215, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [43/16140], Loss: 0.9409, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [44/16140], Loss: 0.7496, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [45/16140], Loss: 1.7614, Accuracy: 30.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [46/16140], Loss: 1.1083, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [47/16140], Loss: 1.2742, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [48/16140], Loss: 0.8518, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [49/16140], Loss: 0.8112, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [50/16140], Loss: 1.2169, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [51/16140], Loss: 0.9873, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [52/16140], Loss: 1.2352, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [53/16140], Loss: 0.9767, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [54/16140], Loss: 0.7621, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [55/16140], Loss: 1.4708, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [56/16140], Loss: 0.9534, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [57/16140], Loss: 0.8938, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [58/16140], Loss: 0.8353, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [59/16140], Loss: 0.7918, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [60/16140], Loss: 1.4101, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [61/16140], Loss: 0.9120, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [62/16140], Loss: 0.9263, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [63/16140], Loss: 1.1457, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [64/16140], Loss: 0.9868, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [65/16140], Loss: 1.0074, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [66/16140], Loss: 0.8181, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [67/16140], Loss: 1.2974, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [68/16140], Loss: 1.2527, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [69/16140], Loss: 1.2236, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [70/16140], Loss: 0.8917, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [71/16140], Loss: 0.8130, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [72/16140], Loss: 1.3394, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [73/16140], Loss: 0.9179, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [74/16140], Loss: 0.7432, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [75/16140], Loss: 1.1483, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [76/16140], Loss: 0.8402, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [77/16140], Loss: 1.3168, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [78/16140], Loss: 1.0086, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [79/16140], Loss: 1.0791, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [80/16140], Loss: 0.9793, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [81/16140], Loss: 1.1055, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [82/16140], Loss: 1.2049, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [83/16140], Loss: 1.3555, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [84/16140], Loss: 0.9284, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [85/16140], Loss: 1.0438, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [86/16140], Loss: 1.0094, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [87/16140], Loss: 0.5408, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [88/16140], Loss: 0.6537, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [89/16140], Loss: 0.9306, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [90/16140], Loss: 1.1831, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [91/16140], Loss: 0.8937, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [92/16140], Loss: 1.2869, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [93/16140], Loss: 1.2673, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [94/16140], Loss: 1.0178, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [95/16140], Loss: 0.8889, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [96/16140], Loss: 0.5178, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [97/16140], Loss: 1.2528, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [98/16140], Loss: 1.2790, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [99/16140], Loss: 1.2042, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [100/16140], Loss: 1.4644, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [101/16140], Loss: 0.9110, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [102/16140], Loss: 1.0518, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [103/16140], Loss: 1.3039, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [104/16140], Loss: 0.8414, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [105/16140], Loss: 0.9537, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [106/16140], Loss: 1.2107, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [107/16140], Loss: 0.8135, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [108/16140], Loss: 1.1889, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [109/16140], Loss: 1.1274, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [110/16140], Loss: 0.9530, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [111/16140], Loss: 1.1265, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [112/16140], Loss: 1.0573, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [113/16140], Loss: 0.8539, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [114/16140], Loss: 0.8801, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [115/16140], Loss: 0.8523, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [116/16140], Loss: 0.9753, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [117/16140], Loss: 1.1206, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [118/16140], Loss: 0.9896, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [119/16140], Loss: 0.9602, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [120/16140], Loss: 0.8420, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [121/16140], Loss: 0.9833, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [122/16140], Loss: 1.1727, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [123/16140], Loss: 1.0921, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [124/16140], Loss: 0.7572, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [125/16140], Loss: 0.9363, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [126/16140], Loss: 0.8472, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [127/16140], Loss: 1.3221, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [128/16140], Loss: 1.3391, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [129/16140], Loss: 1.0216, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [130/16140], Loss: 1.1265, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [131/16140], Loss: 1.2980, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [132/16140], Loss: 0.9089, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [133/16140], Loss: 1.0732, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [134/16140], Loss: 0.7589, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [135/16140], Loss: 1.1794, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [136/16140], Loss: 0.9773, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [137/16140], Loss: 1.2693, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [138/16140], Loss: 1.1438, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [139/16140], Loss: 1.0926, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [140/16140], Loss: 1.3148, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [141/16140], Loss: 1.0116, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [142/16140], Loss: 1.3391, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [143/16140], Loss: 0.8311, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [144/16140], Loss: 1.0321, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [145/16140], Loss: 0.9509, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [146/16140], Loss: 1.3788, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [147/16140], Loss: 0.9972, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [148/16140], Loss: 1.1372, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [149/16140], Loss: 0.7674, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [150/16140], Loss: 1.0060, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [151/16140], Loss: 0.8685, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [152/16140], Loss: 1.0030, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [153/16140], Loss: 1.2098, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [154/16140], Loss: 1.0393, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [155/16140], Loss: 1.2843, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [156/16140], Loss: 0.9747, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [157/16140], Loss: 1.1369, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [158/16140], Loss: 1.3977, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [159/16140], Loss: 1.0039, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [160/16140], Loss: 0.7101, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [161/16140], Loss: 0.8497, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [162/16140], Loss: 1.8071, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [163/16140], Loss: 0.9451, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [164/16140], Loss: 0.7141, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [165/16140], Loss: 1.3332, Accuracy: 36.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [166/16140], Loss: 0.8834, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [167/16140], Loss: 1.0540, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [168/16140], Loss: 1.2248, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [169/16140], Loss: 0.8832, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [170/16140], Loss: 0.9078, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [171/16140], Loss: 1.1929, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [172/16140], Loss: 1.0794, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [173/16140], Loss: 1.1888, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [174/16140], Loss: 1.1909, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [175/16140], Loss: 1.0217, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [176/16140], Loss: 0.9833, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [177/16140], Loss: 0.9937, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [178/16140], Loss: 0.7282, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [179/16140], Loss: 1.0428, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [180/16140], Loss: 1.5821, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [181/16140], Loss: 0.5859, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [182/16140], Loss: 1.1047, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [183/16140], Loss: 1.3479, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [184/16140], Loss: 0.9739, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [185/16140], Loss: 1.0834, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [186/16140], Loss: 1.0379, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [187/16140], Loss: 0.9398, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [188/16140], Loss: 0.5881, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [189/16140], Loss: 0.9332, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [190/16140], Loss: 1.1388, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [191/16140], Loss: 1.0314, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [192/16140], Loss: 1.1566, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [193/16140], Loss: 1.3642, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [194/16140], Loss: 1.3762, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [195/16140], Loss: 1.4285, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [196/16140], Loss: 1.1444, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [197/16140], Loss: 1.2807, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [198/16140], Loss: 1.1210, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [199/16140], Loss: 1.1406, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [200/16140], Loss: 0.9843, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [201/16140], Loss: 0.9824, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [202/16140], Loss: 0.7097, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [203/16140], Loss: 1.1550, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [204/16140], Loss: 0.9109, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [205/16140], Loss: 1.0800, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [206/16140], Loss: 1.2441, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [207/16140], Loss: 0.9803, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [208/16140], Loss: 1.1874, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [209/16140], Loss: 0.9142, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [210/16140], Loss: 0.9096, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [211/16140], Loss: 0.7783, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [212/16140], Loss: 1.0504, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [213/16140], Loss: 0.7362, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [214/16140], Loss: 0.9389, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [215/16140], Loss: 1.5043, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [216/16140], Loss: 0.8744, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [217/16140], Loss: 0.9222, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [218/16140], Loss: 0.9622, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [219/16140], Loss: 0.8797, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [220/16140], Loss: 1.4168, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [221/16140], Loss: 0.9066, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [222/16140], Loss: 0.9274, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [223/16140], Loss: 0.9020, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [224/16140], Loss: 0.7660, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [225/16140], Loss: 1.1868, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [226/16140], Loss: 1.1930, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [227/16140], Loss: 0.8193, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [228/16140], Loss: 1.2865, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [229/16140], Loss: 0.8066, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [230/16140], Loss: 0.9865, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [231/16140], Loss: 0.7486, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [232/16140], Loss: 1.1105, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [233/16140], Loss: 0.6764, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [234/16140], Loss: 0.9993, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [235/16140], Loss: 1.0697, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [236/16140], Loss: 1.1063, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [237/16140], Loss: 0.9493, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [238/16140], Loss: 1.2032, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [239/16140], Loss: 0.8802, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [240/16140], Loss: 1.1809, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [241/16140], Loss: 0.8347, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [242/16140], Loss: 1.1859, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [243/16140], Loss: 1.4520, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [244/16140], Loss: 0.9337, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [245/16140], Loss: 1.5809, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [246/16140], Loss: 1.2552, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [247/16140], Loss: 1.2127, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [248/16140], Loss: 1.0342, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [249/16140], Loss: 0.9824, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [250/16140], Loss: 1.3698, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [251/16140], Loss: 1.1841, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [252/16140], Loss: 1.0411, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [253/16140], Loss: 1.0974, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [254/16140], Loss: 1.2979, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [255/16140], Loss: 0.9918, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [256/16140], Loss: 1.2766, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [257/16140], Loss: 1.0106, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [258/16140], Loss: 1.1687, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [259/16140], Loss: 1.0103, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [260/16140], Loss: 0.7223, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [261/16140], Loss: 0.7214, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [262/16140], Loss: 1.1349, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [263/16140], Loss: 0.8284, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [264/16140], Loss: 1.2947, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [265/16140], Loss: 1.0220, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [266/16140], Loss: 0.7594, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [267/16140], Loss: 0.9917, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [268/16140], Loss: 1.0186, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [269/16140], Loss: 1.2413, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [270/16140], Loss: 0.9579, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [271/16140], Loss: 1.0300, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [272/16140], Loss: 0.8867, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [273/16140], Loss: 1.2076, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [274/16140], Loss: 1.1682, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [275/16140], Loss: 1.3425, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [276/16140], Loss: 0.8593, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [277/16140], Loss: 1.4277, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [278/16140], Loss: 1.4243, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [279/16140], Loss: 1.1809, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [280/16140], Loss: 1.3391, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [281/16140], Loss: 1.2199, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [282/16140], Loss: 1.1300, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [283/16140], Loss: 1.3851, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [284/16140], Loss: 1.3585, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [285/16140], Loss: 1.0793, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [286/16140], Loss: 0.9531, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [287/16140], Loss: 1.0745, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [288/16140], Loss: 1.0815, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [289/16140], Loss: 1.0109, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [290/16140], Loss: 1.4089, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [291/16140], Loss: 1.1562, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [292/16140], Loss: 0.8609, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [293/16140], Loss: 0.9834, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [294/16140], Loss: 1.4191, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [295/16140], Loss: 1.0220, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [296/16140], Loss: 1.2977, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [297/16140], Loss: 0.8021, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [298/16140], Loss: 1.2197, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [299/16140], Loss: 1.2576, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [300/16140], Loss: 1.1373, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [301/16140], Loss: 1.0146, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [302/16140], Loss: 1.1848, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [303/16140], Loss: 1.1532, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [304/16140], Loss: 0.7959, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [305/16140], Loss: 1.0251, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [306/16140], Loss: 1.1849, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [307/16140], Loss: 1.1565, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [308/16140], Loss: 1.3643, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [309/16140], Loss: 1.1031, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [310/16140], Loss: 0.9374, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [311/16140], Loss: 1.0269, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [312/16140], Loss: 1.0946, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [313/16140], Loss: 1.1914, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [314/16140], Loss: 1.0233, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [315/16140], Loss: 1.0224, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [316/16140], Loss: 1.0237, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [317/16140], Loss: 1.1257, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [318/16140], Loss: 1.0997, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [319/16140], Loss: 0.9972, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [320/16140], Loss: 0.7480, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [321/16140], Loss: 1.3079, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [322/16140], Loss: 1.2668, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [323/16140], Loss: 0.8814, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [324/16140], Loss: 0.9675, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [325/16140], Loss: 1.1673, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [326/16140], Loss: 0.8686, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [327/16140], Loss: 1.3045, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [328/16140], Loss: 1.3279, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [329/16140], Loss: 1.1222, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [330/16140], Loss: 1.1179, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [331/16140], Loss: 0.8363, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [332/16140], Loss: 0.9374, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [333/16140], Loss: 0.7897, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [334/16140], Loss: 1.1094, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [335/16140], Loss: 1.4872, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [336/16140], Loss: 1.1639, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [337/16140], Loss: 0.8920, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [338/16140], Loss: 0.9317, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [339/16140], Loss: 1.0492, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [340/16140], Loss: 1.2248, Accuracy: 36.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [341/16140], Loss: 0.6692, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [342/16140], Loss: 1.1577, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [343/16140], Loss: 0.7871, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [344/16140], Loss: 1.2272, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [345/16140], Loss: 0.8728, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [346/16140], Loss: 1.1067, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [347/16140], Loss: 0.8241, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [348/16140], Loss: 1.1053, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [349/16140], Loss: 1.5252, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [350/16140], Loss: 0.9160, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [351/16140], Loss: 0.8855, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [352/16140], Loss: 0.8238, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [353/16140], Loss: 0.8475, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [354/16140], Loss: 0.8351, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [355/16140], Loss: 1.3823, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [356/16140], Loss: 1.2381, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [357/16140], Loss: 0.8807, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [358/16140], Loss: 0.7825, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [359/16140], Loss: 1.1236, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [360/16140], Loss: 1.3388, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [361/16140], Loss: 1.3736, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [362/16140], Loss: 0.8997, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [363/16140], Loss: 0.7194, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [364/16140], Loss: 0.9265, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [365/16140], Loss: 1.0195, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [366/16140], Loss: 1.1429, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [367/16140], Loss: 1.2500, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [368/16140], Loss: 0.9313, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [369/16140], Loss: 0.7529, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [370/16140], Loss: 0.8001, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [371/16140], Loss: 1.1108, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [372/16140], Loss: 1.0824, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [373/16140], Loss: 0.9117, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [374/16140], Loss: 1.0878, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [375/16140], Loss: 0.8302, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [376/16140], Loss: 1.0972, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [377/16140], Loss: 1.0511, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [378/16140], Loss: 1.4041, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [379/16140], Loss: 1.1355, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [380/16140], Loss: 1.0028, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [381/16140], Loss: 1.0798, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [382/16140], Loss: 1.3555, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [383/16140], Loss: 1.2127, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [384/16140], Loss: 1.0035, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [385/16140], Loss: 1.3370, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [386/16140], Loss: 1.2973, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [387/16140], Loss: 0.8605, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [388/16140], Loss: 1.0624, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [389/16140], Loss: 1.0039, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [390/16140], Loss: 0.7184, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [391/16140], Loss: 0.8414, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [392/16140], Loss: 1.0943, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [393/16140], Loss: 0.9738, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [394/16140], Loss: 1.4531, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [395/16140], Loss: 0.8618, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [396/16140], Loss: 1.0592, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [397/16140], Loss: 1.0578, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [398/16140], Loss: 1.0627, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [399/16140], Loss: 1.4012, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [400/16140], Loss: 1.1110, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [401/16140], Loss: 1.1522, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [402/16140], Loss: 0.9877, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [403/16140], Loss: 1.1909, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [404/16140], Loss: 0.8747, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [405/16140], Loss: 1.0793, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [406/16140], Loss: 1.1606, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [407/16140], Loss: 0.9735, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [408/16140], Loss: 0.8719, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [409/16140], Loss: 1.2384, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [410/16140], Loss: 0.8813, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [411/16140], Loss: 1.1358, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [412/16140], Loss: 1.0391, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [413/16140], Loss: 1.2855, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [414/16140], Loss: 0.8841, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [415/16140], Loss: 0.7588, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [416/16140], Loss: 1.3815, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [417/16140], Loss: 1.2219, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [418/16140], Loss: 0.9723, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [419/16140], Loss: 1.1102, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [420/16140], Loss: 1.2847, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [421/16140], Loss: 1.5989, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [422/16140], Loss: 0.8722, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [423/16140], Loss: 0.9546, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [424/16140], Loss: 1.2714, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [425/16140], Loss: 0.8659, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [426/16140], Loss: 1.4536, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [427/16140], Loss: 0.9216, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [428/16140], Loss: 1.2514, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [429/16140], Loss: 1.2809, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [430/16140], Loss: 0.8909, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [431/16140], Loss: 1.2033, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [432/16140], Loss: 0.8597, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [433/16140], Loss: 0.9593, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [434/16140], Loss: 1.0193, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [435/16140], Loss: 1.2937, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [436/16140], Loss: 1.2009, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [437/16140], Loss: 0.9004, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [438/16140], Loss: 1.1197, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [439/16140], Loss: 0.8332, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [440/16140], Loss: 1.0625, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [441/16140], Loss: 1.6078, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [442/16140], Loss: 1.0888, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [443/16140], Loss: 1.0126, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [444/16140], Loss: 1.0472, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [445/16140], Loss: 1.0442, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [446/16140], Loss: 1.0346, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [447/16140], Loss: 0.8766, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [448/16140], Loss: 0.8348, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [449/16140], Loss: 0.8209, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [450/16140], Loss: 0.9920, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [451/16140], Loss: 0.9697, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [452/16140], Loss: 1.4775, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [453/16140], Loss: 1.0018, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [454/16140], Loss: 1.0111, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [455/16140], Loss: 1.0003, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [456/16140], Loss: 1.1175, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [457/16140], Loss: 1.2009, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [458/16140], Loss: 0.9224, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [459/16140], Loss: 0.8316, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [460/16140], Loss: 0.8151, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [461/16140], Loss: 1.2076, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [462/16140], Loss: 0.9729, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [463/16140], Loss: 0.8483, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [464/16140], Loss: 1.0591, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [465/16140], Loss: 1.0745, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [466/16140], Loss: 1.0393, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [467/16140], Loss: 1.3209, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [468/16140], Loss: 1.4893, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [469/16140], Loss: 1.0149, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [470/16140], Loss: 1.1757, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [471/16140], Loss: 0.9992, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [472/16140], Loss: 0.8280, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [473/16140], Loss: 1.1288, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [474/16140], Loss: 1.4354, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [475/16140], Loss: 1.2258, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [476/16140], Loss: 1.3114, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [477/16140], Loss: 1.0059, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [478/16140], Loss: 1.1135, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [479/16140], Loss: 0.9491, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [480/16140], Loss: 1.2265, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [481/16140], Loss: 1.0489, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [482/16140], Loss: 0.8962, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [483/16140], Loss: 0.9917, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [484/16140], Loss: 1.1998, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [485/16140], Loss: 1.0271, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [486/16140], Loss: 0.6957, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [487/16140], Loss: 0.9373, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [488/16140], Loss: 1.1147, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [489/16140], Loss: 1.1508, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [490/16140], Loss: 1.3052, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [491/16140], Loss: 1.0131, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [492/16140], Loss: 1.4119, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [493/16140], Loss: 0.8596, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [494/16140], Loss: 1.1530, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [495/16140], Loss: 0.8284, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [496/16140], Loss: 0.9482, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [497/16140], Loss: 1.2457, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [498/16140], Loss: 0.7889, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [499/16140], Loss: 1.0546, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [500/16140], Loss: 1.3471, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [501/16140], Loss: 1.0333, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [502/16140], Loss: 1.0474, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [503/16140], Loss: 1.0780, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [504/16140], Loss: 1.1310, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [505/16140], Loss: 1.3985, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [506/16140], Loss: 1.0363, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [507/16140], Loss: 0.9805, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [508/16140], Loss: 1.0736, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [509/16140], Loss: 1.2624, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [510/16140], Loss: 1.1661, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [511/16140], Loss: 1.0917, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [512/16140], Loss: 0.7796, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [513/16140], Loss: 0.6866, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [514/16140], Loss: 1.0201, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [515/16140], Loss: 0.9502, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [516/16140], Loss: 1.4973, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [517/16140], Loss: 0.8933, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [518/16140], Loss: 1.0785, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [519/16140], Loss: 0.7858, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [520/16140], Loss: 1.0429, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [521/16140], Loss: 1.4142, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [522/16140], Loss: 0.9812, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [523/16140], Loss: 1.0173, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [524/16140], Loss: 1.2425, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [525/16140], Loss: 0.9203, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [526/16140], Loss: 0.8710, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [527/16140], Loss: 0.9859, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [528/16140], Loss: 1.1921, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [529/16140], Loss: 0.9145, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [530/16140], Loss: 1.1240, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [531/16140], Loss: 1.0491, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [532/16140], Loss: 1.2529, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [533/16140], Loss: 0.8447, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [534/16140], Loss: 1.0326, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [535/16140], Loss: 1.0629, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [536/16140], Loss: 1.2337, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [537/16140], Loss: 0.5942, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [538/16140], Loss: 1.0278, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [539/16140], Loss: 1.1899, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [540/16140], Loss: 1.0934, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [541/16140], Loss: 0.9541, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [542/16140], Loss: 0.9426, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [543/16140], Loss: 1.2187, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [544/16140], Loss: 0.9991, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [545/16140], Loss: 1.0714, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [546/16140], Loss: 1.2889, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [547/16140], Loss: 0.9024, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [548/16140], Loss: 0.9130, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [549/16140], Loss: 0.9793, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [550/16140], Loss: 0.7436, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [551/16140], Loss: 1.1703, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [552/16140], Loss: 1.0175, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [553/16140], Loss: 1.0014, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [554/16140], Loss: 1.1244, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [555/16140], Loss: 0.9545, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [556/16140], Loss: 0.9653, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [557/16140], Loss: 1.0107, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [558/16140], Loss: 1.0501, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [559/16140], Loss: 1.3488, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [560/16140], Loss: 0.9438, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [561/16140], Loss: 1.3817, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [562/16140], Loss: 0.7059, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [563/16140], Loss: 1.2228, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [564/16140], Loss: 1.1404, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [565/16140], Loss: 0.9800, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [566/16140], Loss: 1.3973, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [567/16140], Loss: 1.1972, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [568/16140], Loss: 1.0798, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [569/16140], Loss: 1.0309, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [570/16140], Loss: 0.7289, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [571/16140], Loss: 1.4542, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [572/16140], Loss: 0.9766, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [573/16140], Loss: 1.4201, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [574/16140], Loss: 1.3813, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [575/16140], Loss: 1.4690, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [576/16140], Loss: 1.3001, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [577/16140], Loss: 1.5550, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [578/16140], Loss: 0.8006, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [579/16140], Loss: 0.8083, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [580/16140], Loss: 1.5117, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [581/16140], Loss: 1.2204, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [582/16140], Loss: 1.2015, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [583/16140], Loss: 1.2171, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [584/16140], Loss: 1.0063, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [585/16140], Loss: 1.3816, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [586/16140], Loss: 0.8969, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [587/16140], Loss: 1.2922, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [588/16140], Loss: 1.2759, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [589/16140], Loss: 1.3864, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [590/16140], Loss: 1.2271, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [591/16140], Loss: 1.0660, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [592/16140], Loss: 1.3372, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [593/16140], Loss: 1.4872, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [594/16140], Loss: 1.1619, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [595/16140], Loss: 0.8373, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [596/16140], Loss: 1.1169, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [597/16140], Loss: 1.6612, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [598/16140], Loss: 1.3275, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [599/16140], Loss: 1.3217, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [600/16140], Loss: 0.9552, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [601/16140], Loss: 1.0569, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [602/16140], Loss: 1.3930, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [603/16140], Loss: 1.5087, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [604/16140], Loss: 0.7183, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [605/16140], Loss: 1.5218, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [606/16140], Loss: 1.2569, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [607/16140], Loss: 1.0439, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [608/16140], Loss: 0.9087, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [609/16140], Loss: 1.0966, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [610/16140], Loss: 0.6963, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [611/16140], Loss: 1.0641, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [612/16140], Loss: 1.2565, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [613/16140], Loss: 1.2212, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [614/16140], Loss: 1.2750, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [615/16140], Loss: 1.1588, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [616/16140], Loss: 1.0198, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [617/16140], Loss: 0.8673, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [618/16140], Loss: 0.9423, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [619/16140], Loss: 0.7555, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [620/16140], Loss: 1.3172, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [621/16140], Loss: 1.1312, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [622/16140], Loss: 1.1740, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [623/16140], Loss: 0.7237, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [624/16140], Loss: 1.1629, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [625/16140], Loss: 1.0129, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [626/16140], Loss: 0.9048, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [627/16140], Loss: 0.8247, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [628/16140], Loss: 0.8282, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [629/16140], Loss: 1.1091, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [630/16140], Loss: 1.1693, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [631/16140], Loss: 1.2556, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [632/16140], Loss: 0.7888, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [633/16140], Loss: 0.7732, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [634/16140], Loss: 0.8865, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [635/16140], Loss: 0.8321, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [636/16140], Loss: 1.1376, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [637/16140], Loss: 0.7385, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [638/16140], Loss: 0.6830, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [639/16140], Loss: 0.9522, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [640/16140], Loss: 0.8361, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [641/16140], Loss: 1.2433, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [642/16140], Loss: 1.2595, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [643/16140], Loss: 1.2327, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [644/16140], Loss: 1.1899, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [645/16140], Loss: 0.9281, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [646/16140], Loss: 0.8099, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [647/16140], Loss: 1.5176, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [648/16140], Loss: 0.8364, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [649/16140], Loss: 0.9550, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [650/16140], Loss: 1.3941, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [651/16140], Loss: 1.3017, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [652/16140], Loss: 0.9141, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [653/16140], Loss: 1.1951, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [654/16140], Loss: 0.7849, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [655/16140], Loss: 1.2653, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [656/16140], Loss: 1.3110, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [657/16140], Loss: 0.8220, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [658/16140], Loss: 1.1976, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [659/16140], Loss: 1.0171, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [660/16140], Loss: 1.2635, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [661/16140], Loss: 0.9544, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [662/16140], Loss: 1.2014, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [663/16140], Loss: 0.8291, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [664/16140], Loss: 0.9541, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [665/16140], Loss: 1.1872, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [666/16140], Loss: 1.2397, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [667/16140], Loss: 1.3999, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [668/16140], Loss: 0.8028, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [669/16140], Loss: 1.4735, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [670/16140], Loss: 0.8783, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [671/16140], Loss: 0.7693, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [672/16140], Loss: 1.1289, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [673/16140], Loss: 0.9109, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [674/16140], Loss: 0.9428, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [675/16140], Loss: 1.1640, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [676/16140], Loss: 1.0622, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [677/16140], Loss: 1.2581, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [678/16140], Loss: 1.0977, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [679/16140], Loss: 0.9574, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [680/16140], Loss: 0.9184, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [681/16140], Loss: 0.8451, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [682/16140], Loss: 0.9353, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [683/16140], Loss: 1.3974, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [684/16140], Loss: 1.3149, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [685/16140], Loss: 1.2878, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [686/16140], Loss: 1.4289, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [687/16140], Loss: 1.1427, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [688/16140], Loss: 1.0563, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [689/16140], Loss: 1.0989, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [690/16140], Loss: 0.7913, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [691/16140], Loss: 1.3383, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [692/16140], Loss: 1.0637, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [693/16140], Loss: 0.9864, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [694/16140], Loss: 0.9749, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [695/16140], Loss: 0.7314, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [696/16140], Loss: 0.9135, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [697/16140], Loss: 1.2186, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [698/16140], Loss: 0.9749, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [699/16140], Loss: 1.1722, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [700/16140], Loss: 1.1907, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [701/16140], Loss: 1.4173, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [702/16140], Loss: 1.4803, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [703/16140], Loss: 0.7920, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [704/16140], Loss: 0.9695, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [705/16140], Loss: 1.0429, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [706/16140], Loss: 1.1647, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [707/16140], Loss: 0.8448, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [708/16140], Loss: 0.9332, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [709/16140], Loss: 1.3271, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [710/16140], Loss: 1.2077, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [711/16140], Loss: 1.3695, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [712/16140], Loss: 1.1112, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [713/16140], Loss: 0.9503, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [714/16140], Loss: 1.0890, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [715/16140], Loss: 0.9039, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [716/16140], Loss: 1.1598, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [717/16140], Loss: 0.8450, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [718/16140], Loss: 0.7909, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [719/16140], Loss: 1.2801, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [720/16140], Loss: 0.8965, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [721/16140], Loss: 0.6807, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [722/16140], Loss: 1.1614, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [723/16140], Loss: 0.8162, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [724/16140], Loss: 0.6634, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [725/16140], Loss: 1.1834, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [726/16140], Loss: 0.8750, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [727/16140], Loss: 0.7557, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [728/16140], Loss: 0.8418, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [729/16140], Loss: 1.7461, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [730/16140], Loss: 1.1994, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [731/16140], Loss: 0.6539, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [732/16140], Loss: 0.6838, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [733/16140], Loss: 0.9587, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [734/16140], Loss: 0.9714, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [735/16140], Loss: 1.1150, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [736/16140], Loss: 1.2927, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [737/16140], Loss: 1.2470, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [738/16140], Loss: 1.3925, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [739/16140], Loss: 1.2346, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [740/16140], Loss: 1.2093, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [741/16140], Loss: 1.1162, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [742/16140], Loss: 1.2665, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [743/16140], Loss: 1.2793, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [744/16140], Loss: 1.3027, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [745/16140], Loss: 1.0422, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [746/16140], Loss: 0.9241, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [747/16140], Loss: 1.0002, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [748/16140], Loss: 1.2246, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [749/16140], Loss: 1.1123, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [750/16140], Loss: 1.2203, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [751/16140], Loss: 0.9337, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [752/16140], Loss: 0.8190, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [753/16140], Loss: 1.0103, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [754/16140], Loss: 1.3169, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [755/16140], Loss: 1.0923, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [756/16140], Loss: 0.8244, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [757/16140], Loss: 1.0070, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [758/16140], Loss: 1.2153, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [759/16140], Loss: 0.9763, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [760/16140], Loss: 0.7530, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [761/16140], Loss: 1.5155, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [762/16140], Loss: 1.3737, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [763/16140], Loss: 1.0090, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [764/16140], Loss: 0.8541, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [765/16140], Loss: 0.9856, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [766/16140], Loss: 0.8481, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [767/16140], Loss: 0.7297, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [768/16140], Loss: 1.0900, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [769/16140], Loss: 1.1249, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [770/16140], Loss: 1.3344, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [771/16140], Loss: 0.9701, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [772/16140], Loss: 1.2591, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [773/16140], Loss: 0.7194, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [774/16140], Loss: 0.9825, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [775/16140], Loss: 1.1572, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [776/16140], Loss: 1.1883, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [777/16140], Loss: 1.1412, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [778/16140], Loss: 1.0783, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [779/16140], Loss: 1.1655, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [780/16140], Loss: 1.1849, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [781/16140], Loss: 1.0240, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [782/16140], Loss: 0.7513, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [783/16140], Loss: 1.1564, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [784/16140], Loss: 1.1263, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [785/16140], Loss: 1.2370, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [786/16140], Loss: 0.7178, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [787/16140], Loss: 1.0127, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [788/16140], Loss: 1.0113, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [789/16140], Loss: 1.1227, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [790/16140], Loss: 1.0818, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [791/16140], Loss: 0.8241, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [792/16140], Loss: 1.0571, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [793/16140], Loss: 0.8425, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [794/16140], Loss: 1.1632, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [795/16140], Loss: 0.8311, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [796/16140], Loss: 1.0219, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [797/16140], Loss: 0.8958, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [798/16140], Loss: 1.2008, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [799/16140], Loss: 0.9346, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [800/16140], Loss: 1.0566, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [801/16140], Loss: 0.8240, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [802/16140], Loss: 0.9034, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [803/16140], Loss: 1.2170, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [804/16140], Loss: 0.9077, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [805/16140], Loss: 1.0139, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [806/16140], Loss: 1.2187, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [807/16140], Loss: 1.0423, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [808/16140], Loss: 0.8554, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [809/16140], Loss: 1.0206, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [810/16140], Loss: 0.9609, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [811/16140], Loss: 1.3487, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [812/16140], Loss: 0.9667, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [813/16140], Loss: 1.2102, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [814/16140], Loss: 1.0262, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [815/16140], Loss: 1.4113, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [816/16140], Loss: 1.0516, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [817/16140], Loss: 1.0138, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [818/16140], Loss: 0.9823, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [819/16140], Loss: 0.7604, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [820/16140], Loss: 1.4271, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [821/16140], Loss: 1.2440, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [822/16140], Loss: 0.9077, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [823/16140], Loss: 1.0194, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [824/16140], Loss: 0.6541, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [825/16140], Loss: 0.9106, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [826/16140], Loss: 0.8345, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [827/16140], Loss: 1.0249, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [828/16140], Loss: 1.2302, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [829/16140], Loss: 1.1971, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [830/16140], Loss: 0.7595, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [831/16140], Loss: 0.8745, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [832/16140], Loss: 0.7977, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [833/16140], Loss: 0.9026, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [834/16140], Loss: 0.8923, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [835/16140], Loss: 1.0645, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [836/16140], Loss: 0.8855, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [837/16140], Loss: 1.1311, Accuracy: 60.00%, Iteration Time: 0.03 seconds
Epoch [9/10], Step [838/16140], Loss: 1.0285, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [839/16140], Loss: 1.0713, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [840/16140], Loss: 0.8171, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [841/16140], Loss: 0.8709, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [842/16140], Loss: 0.9236, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [843/16140], Loss: 1.3529, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [844/16140], Loss: 1.5144, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [845/16140], Loss: 1.0074, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [846/16140], Loss: 1.2909, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [847/16140], Loss: 0.8164, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [848/16140], Loss: 1.0818, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [849/16140], Loss: 0.9083, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [850/16140], Loss: 1.0379, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [851/16140], Loss: 1.3304, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [852/16140], Loss: 1.3795, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [853/16140], Loss: 1.1945, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [854/16140], Loss: 1.0124, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [855/16140], Loss: 0.7313, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [856/16140], Loss: 1.5193, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [857/16140], Loss: 1.4671, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [858/16140], Loss: 0.9935, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [859/16140], Loss: 1.3162, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [860/16140], Loss: 1.1091, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [861/16140], Loss: 1.3034, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [862/16140], Loss: 1.4421, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [863/16140], Loss: 1.1089, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [864/16140], Loss: 0.9030, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [865/16140], Loss: 1.0201, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [866/16140], Loss: 1.2538, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [867/16140], Loss: 0.9733, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [868/16140], Loss: 0.9943, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [869/16140], Loss: 1.2117, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [870/16140], Loss: 1.4546, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [871/16140], Loss: 1.1338, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [872/16140], Loss: 1.1993, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [873/16140], Loss: 1.4722, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [874/16140], Loss: 0.7662, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [875/16140], Loss: 0.8283, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [876/16140], Loss: 1.1172, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [877/16140], Loss: 0.9038, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [878/16140], Loss: 1.2667, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [879/16140], Loss: 0.8514, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [880/16140], Loss: 0.9558, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [881/16140], Loss: 0.7198, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [882/16140], Loss: 0.5863, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [883/16140], Loss: 1.1591, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [884/16140], Loss: 1.3452, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [885/16140], Loss: 1.2390, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [886/16140], Loss: 0.8006, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [887/16140], Loss: 1.0476, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [888/16140], Loss: 0.9580, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [889/16140], Loss: 1.0787, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [890/16140], Loss: 1.3062, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [891/16140], Loss: 0.7124, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [892/16140], Loss: 1.0404, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [893/16140], Loss: 1.0854, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [894/16140], Loss: 1.0160, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [895/16140], Loss: 1.0621, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [896/16140], Loss: 1.0274, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [897/16140], Loss: 1.0276, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [898/16140], Loss: 0.8133, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [899/16140], Loss: 0.7848, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [900/16140], Loss: 1.0238, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [901/16140], Loss: 0.6560, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [902/16140], Loss: 1.0403, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [903/16140], Loss: 1.4000, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [904/16140], Loss: 1.1382, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [905/16140], Loss: 1.1916, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [906/16140], Loss: 1.4307, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [907/16140], Loss: 1.1293, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [908/16140], Loss: 0.9904, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [909/16140], Loss: 1.1113, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [910/16140], Loss: 1.3519, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [911/16140], Loss: 0.6541, Accuracy: 90.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [912/16140], Loss: 1.1925, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [913/16140], Loss: 0.9522, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [914/16140], Loss: 1.1082, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [915/16140], Loss: 1.1835, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [916/16140], Loss: 0.9119, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [917/16140], Loss: 0.9117, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [918/16140], Loss: 0.9992, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [919/16140], Loss: 1.1038, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [920/16140], Loss: 1.4264, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [921/16140], Loss: 1.2006, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [922/16140], Loss: 0.8204, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [923/16140], Loss: 0.8466, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [924/16140], Loss: 1.5883, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [925/16140], Loss: 1.2525, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [926/16140], Loss: 1.6292, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [927/16140], Loss: 1.1937, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [928/16140], Loss: 1.3528, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [929/16140], Loss: 0.9329, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [930/16140], Loss: 1.2515, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [931/16140], Loss: 1.0493, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [932/16140], Loss: 1.2709, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [933/16140], Loss: 1.2490, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [934/16140], Loss: 1.1304, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [935/16140], Loss: 0.9421, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [936/16140], Loss: 1.3030, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [937/16140], Loss: 1.1349, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [938/16140], Loss: 1.0054, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [939/16140], Loss: 1.0757, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [940/16140], Loss: 0.6249, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [941/16140], Loss: 0.8078, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [942/16140], Loss: 1.6938, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [943/16140], Loss: 1.1675, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [944/16140], Loss: 0.9571, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [945/16140], Loss: 1.1009, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [946/16140], Loss: 1.2794, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [947/16140], Loss: 1.1842, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [948/16140], Loss: 0.8398, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [949/16140], Loss: 1.1303, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [950/16140], Loss: 1.1930, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [951/16140], Loss: 0.8108, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [952/16140], Loss: 0.9358, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [953/16140], Loss: 1.0127, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [954/16140], Loss: 1.4072, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [955/16140], Loss: 1.0714, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [956/16140], Loss: 1.0618, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [957/16140], Loss: 1.3411, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [958/16140], Loss: 0.9990, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [959/16140], Loss: 1.1059, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [960/16140], Loss: 0.9172, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [961/16140], Loss: 1.2292, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [962/16140], Loss: 0.7403, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [963/16140], Loss: 1.1629, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [964/16140], Loss: 1.1465, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [965/16140], Loss: 0.8776, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [966/16140], Loss: 1.0660, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [967/16140], Loss: 1.1400, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [968/16140], Loss: 1.1263, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [969/16140], Loss: 1.0999, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [970/16140], Loss: 1.0870, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [971/16140], Loss: 1.2331, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [972/16140], Loss: 0.9499, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [973/16140], Loss: 1.0685, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [974/16140], Loss: 1.0802, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [975/16140], Loss: 1.4715, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [976/16140], Loss: 1.1897, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [977/16140], Loss: 1.2925, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [978/16140], Loss: 1.0450, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [979/16140], Loss: 1.0103, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [980/16140], Loss: 0.9942, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [981/16140], Loss: 0.6389, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [982/16140], Loss: 0.7920, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [983/16140], Loss: 0.7603, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [984/16140], Loss: 1.0207, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [985/16140], Loss: 0.8059, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [986/16140], Loss: 1.0890, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [987/16140], Loss: 1.1989, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [988/16140], Loss: 0.8136, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [989/16140], Loss: 1.3103, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [990/16140], Loss: 0.8513, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [991/16140], Loss: 1.0739, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [992/16140], Loss: 1.2698, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [993/16140], Loss: 1.0648, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [994/16140], Loss: 1.3459, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [995/16140], Loss: 1.0195, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [996/16140], Loss: 0.8343, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [997/16140], Loss: 0.9423, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [998/16140], Loss: 0.9749, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [999/16140], Loss: 1.3608, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1000/16140], Loss: 1.0739, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1001/16140], Loss: 0.9892, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1002/16140], Loss: 1.2159, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1003/16140], Loss: 0.6482, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1004/16140], Loss: 1.1514, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1005/16140], Loss: 1.4098, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1006/16140], Loss: 0.8865, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1007/16140], Loss: 0.8085, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1008/16140], Loss: 0.7367, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1009/16140], Loss: 0.9291, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1010/16140], Loss: 1.1650, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1011/16140], Loss: 0.9676, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1012/16140], Loss: 1.3469, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1013/16140], Loss: 0.9514, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1014/16140], Loss: 0.8715, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1015/16140], Loss: 1.0742, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1016/16140], Loss: 1.2605, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1017/16140], Loss: 1.0339, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1018/16140], Loss: 0.9323, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1019/16140], Loss: 1.3005, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1020/16140], Loss: 0.8661, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1021/16140], Loss: 0.9104, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1022/16140], Loss: 0.8449, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1023/16140], Loss: 1.3873, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1024/16140], Loss: 0.8981, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1025/16140], Loss: 1.1275, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1026/16140], Loss: 0.9548, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1027/16140], Loss: 1.0468, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1028/16140], Loss: 0.9528, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1029/16140], Loss: 1.0165, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1030/16140], Loss: 0.9258, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1031/16140], Loss: 0.9784, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1032/16140], Loss: 1.2062, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1033/16140], Loss: 1.1628, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1034/16140], Loss: 0.7420, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1035/16140], Loss: 1.0136, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1036/16140], Loss: 1.0961, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1037/16140], Loss: 1.0752, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1038/16140], Loss: 1.1930, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1039/16140], Loss: 1.3888, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1040/16140], Loss: 1.0971, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1041/16140], Loss: 0.9298, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1042/16140], Loss: 1.1389, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1043/16140], Loss: 0.7966, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1044/16140], Loss: 1.3849, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1045/16140], Loss: 1.1949, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1046/16140], Loss: 1.1031, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1047/16140], Loss: 0.9130, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1048/16140], Loss: 0.8518, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1049/16140], Loss: 1.4462, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1050/16140], Loss: 1.0205, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1051/16140], Loss: 0.7812, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1052/16140], Loss: 1.0121, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1053/16140], Loss: 0.9122, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1054/16140], Loss: 0.9271, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1055/16140], Loss: 1.0745, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1056/16140], Loss: 0.6097, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1057/16140], Loss: 1.3157, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1058/16140], Loss: 1.4746, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1059/16140], Loss: 1.1527, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1060/16140], Loss: 1.0322, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1061/16140], Loss: 1.1041, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1062/16140], Loss: 0.8895, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1063/16140], Loss: 1.2036, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1064/16140], Loss: 0.8696, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1065/16140], Loss: 1.0956, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1066/16140], Loss: 0.7318, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1067/16140], Loss: 1.0159, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1068/16140], Loss: 1.0069, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1069/16140], Loss: 1.2677, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1070/16140], Loss: 0.9149, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1071/16140], Loss: 0.9887, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1072/16140], Loss: 0.8348, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1073/16140], Loss: 1.2075, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1074/16140], Loss: 0.9297, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1075/16140], Loss: 0.9806, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1076/16140], Loss: 0.8145, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1077/16140], Loss: 1.0318, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1078/16140], Loss: 1.2364, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1079/16140], Loss: 1.2267, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1080/16140], Loss: 0.8263, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1081/16140], Loss: 1.2130, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1082/16140], Loss: 1.0957, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1083/16140], Loss: 0.7920, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1084/16140], Loss: 0.9765, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1085/16140], Loss: 0.9367, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1086/16140], Loss: 1.0345, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1087/16140], Loss: 0.8589, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1088/16140], Loss: 1.4236, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1089/16140], Loss: 1.1869, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1090/16140], Loss: 1.3708, Accuracy: 36.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1091/16140], Loss: 0.7318, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1092/16140], Loss: 0.9497, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1093/16140], Loss: 1.1119, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1094/16140], Loss: 1.2323, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1095/16140], Loss: 0.6206, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1096/16140], Loss: 1.0768, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1097/16140], Loss: 1.2254, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1098/16140], Loss: 0.9024, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1099/16140], Loss: 1.0241, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1100/16140], Loss: 0.8098, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1101/16140], Loss: 1.0973, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1102/16140], Loss: 1.0327, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1103/16140], Loss: 0.9856, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1104/16140], Loss: 0.9521, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1105/16140], Loss: 1.1035, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1106/16140], Loss: 1.1494, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1107/16140], Loss: 1.0505, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1108/16140], Loss: 1.0317, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1109/16140], Loss: 1.4411, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1110/16140], Loss: 0.9826, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1111/16140], Loss: 1.2115, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1112/16140], Loss: 0.6381, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1113/16140], Loss: 1.3439, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1114/16140], Loss: 0.6701, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1115/16140], Loss: 1.3639, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1116/16140], Loss: 0.9107, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1117/16140], Loss: 0.8723, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1118/16140], Loss: 1.0454, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1119/16140], Loss: 1.2861, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1120/16140], Loss: 0.8992, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1121/16140], Loss: 0.8394, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1122/16140], Loss: 0.7114, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1123/16140], Loss: 0.8241, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1124/16140], Loss: 1.2853, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1125/16140], Loss: 1.0675, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1126/16140], Loss: 1.0261, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1127/16140], Loss: 0.8866, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1128/16140], Loss: 1.4465, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1129/16140], Loss: 1.0566, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1130/16140], Loss: 0.6786, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1131/16140], Loss: 0.9387, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1132/16140], Loss: 1.4190, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1133/16140], Loss: 0.9957, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1134/16140], Loss: 1.3093, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1135/16140], Loss: 1.2261, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1136/16140], Loss: 0.9080, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1137/16140], Loss: 0.8868, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1138/16140], Loss: 0.9358, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1139/16140], Loss: 1.2546, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1140/16140], Loss: 1.5429, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1141/16140], Loss: 1.0062, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1142/16140], Loss: 1.0826, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1143/16140], Loss: 0.9637, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1144/16140], Loss: 0.9252, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1145/16140], Loss: 1.7552, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1146/16140], Loss: 1.1007, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1147/16140], Loss: 1.3906, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1148/16140], Loss: 1.3301, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1149/16140], Loss: 0.8192, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1150/16140], Loss: 0.8660, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1151/16140], Loss: 1.0494, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1152/16140], Loss: 1.1511, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1153/16140], Loss: 0.8246, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1154/16140], Loss: 0.9570, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1155/16140], Loss: 0.9317, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1156/16140], Loss: 1.0165, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1157/16140], Loss: 1.3310, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1158/16140], Loss: 1.2555, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1159/16140], Loss: 0.7064, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1160/16140], Loss: 1.0624, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1161/16140], Loss: 0.8580, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1162/16140], Loss: 0.9645, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1163/16140], Loss: 1.1205, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1164/16140], Loss: 1.4625, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1165/16140], Loss: 0.8568, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1166/16140], Loss: 0.4879, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1167/16140], Loss: 0.7870, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1168/16140], Loss: 0.8404, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1169/16140], Loss: 1.1557, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1170/16140], Loss: 1.3083, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1171/16140], Loss: 1.0058, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1172/16140], Loss: 1.0679, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1173/16140], Loss: 1.3682, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1174/16140], Loss: 1.2623, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1175/16140], Loss: 0.8638, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1176/16140], Loss: 1.2521, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1177/16140], Loss: 0.9390, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1178/16140], Loss: 1.0911, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1179/16140], Loss: 1.0297, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1180/16140], Loss: 1.1339, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1181/16140], Loss: 1.2116, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1182/16140], Loss: 0.5777, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1183/16140], Loss: 1.2493, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1184/16140], Loss: 1.1258, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1185/16140], Loss: 0.8907, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1186/16140], Loss: 1.2596, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1187/16140], Loss: 1.0192, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1188/16140], Loss: 1.1978, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1189/16140], Loss: 1.0196, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1190/16140], Loss: 0.9698, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1191/16140], Loss: 1.5207, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1192/16140], Loss: 1.0002, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1193/16140], Loss: 1.0571, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1194/16140], Loss: 1.3466, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1195/16140], Loss: 1.0841, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1196/16140], Loss: 1.0378, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1197/16140], Loss: 1.0685, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1198/16140], Loss: 1.1110, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1199/16140], Loss: 1.1078, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1200/16140], Loss: 0.9549, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1201/16140], Loss: 0.8776, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1202/16140], Loss: 1.0535, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1203/16140], Loss: 0.8817, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1204/16140], Loss: 1.1103, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1205/16140], Loss: 1.1386, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1206/16140], Loss: 0.7193, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1207/16140], Loss: 0.7935, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1208/16140], Loss: 0.9316, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1209/16140], Loss: 1.1594, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1210/16140], Loss: 0.7939, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1211/16140], Loss: 0.7843, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1212/16140], Loss: 0.6770, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1213/16140], Loss: 0.9743, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1214/16140], Loss: 0.8104, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1215/16140], Loss: 1.2308, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1216/16140], Loss: 0.7539, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1217/16140], Loss: 1.0376, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1218/16140], Loss: 0.9245, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1219/16140], Loss: 0.8796, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1220/16140], Loss: 1.2553, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1221/16140], Loss: 0.8139, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1222/16140], Loss: 0.9251, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1223/16140], Loss: 1.2872, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1224/16140], Loss: 0.7842, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1225/16140], Loss: 0.7816, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1226/16140], Loss: 0.9328, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1227/16140], Loss: 0.8250, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1228/16140], Loss: 0.8419, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1229/16140], Loss: 0.8615, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1230/16140], Loss: 0.9347, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1231/16140], Loss: 0.9507, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1232/16140], Loss: 0.9924, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1233/16140], Loss: 0.6765, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1234/16140], Loss: 1.2212, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1235/16140], Loss: 1.1036, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1236/16140], Loss: 1.1155, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1237/16140], Loss: 0.7721, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1238/16140], Loss: 1.1622, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1239/16140], Loss: 0.7817, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1240/16140], Loss: 0.9925, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1241/16140], Loss: 1.0319, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1242/16140], Loss: 1.1693, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1243/16140], Loss: 0.7892, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1244/16140], Loss: 1.0808, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1245/16140], Loss: 1.0127, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1246/16140], Loss: 1.0450, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1247/16140], Loss: 1.4052, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1248/16140], Loss: 1.3384, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1249/16140], Loss: 1.1142, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1250/16140], Loss: 1.1521, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1251/16140], Loss: 1.1465, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1252/16140], Loss: 1.0637, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1253/16140], Loss: 1.3560, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1254/16140], Loss: 1.1892, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1255/16140], Loss: 1.3143, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1256/16140], Loss: 1.0445, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1257/16140], Loss: 1.2471, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1258/16140], Loss: 0.9052, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1259/16140], Loss: 1.1747, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1260/16140], Loss: 1.2671, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1261/16140], Loss: 1.1726, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1262/16140], Loss: 1.2482, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1263/16140], Loss: 1.0457, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1264/16140], Loss: 0.9442, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1265/16140], Loss: 1.2639, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1266/16140], Loss: 0.9097, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1267/16140], Loss: 1.4822, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1268/16140], Loss: 1.3071, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1269/16140], Loss: 1.1596, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1270/16140], Loss: 1.0079, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1271/16140], Loss: 0.9803, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1272/16140], Loss: 1.3427, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1273/16140], Loss: 1.1262, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1274/16140], Loss: 0.9868, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1275/16140], Loss: 1.0154, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1276/16140], Loss: 1.2417, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1277/16140], Loss: 1.1460, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1278/16140], Loss: 1.3469, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1279/16140], Loss: 0.9420, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1280/16140], Loss: 1.2871, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1281/16140], Loss: 1.1908, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1282/16140], Loss: 1.0464, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1283/16140], Loss: 1.1514, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1284/16140], Loss: 1.1218, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1285/16140], Loss: 1.1068, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1286/16140], Loss: 1.1681, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1287/16140], Loss: 1.1855, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1288/16140], Loss: 0.8644, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1289/16140], Loss: 0.9658, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1290/16140], Loss: 1.1606, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1291/16140], Loss: 0.6352, Accuracy: 90.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1292/16140], Loss: 1.0065, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1293/16140], Loss: 0.7646, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1294/16140], Loss: 1.1741, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1295/16140], Loss: 1.0575, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1296/16140], Loss: 0.7878, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1297/16140], Loss: 0.8951, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1298/16140], Loss: 1.3890, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1299/16140], Loss: 0.9808, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1300/16140], Loss: 0.8489, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1301/16140], Loss: 0.6987, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1302/16140], Loss: 0.8070, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1303/16140], Loss: 1.1266, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1304/16140], Loss: 1.0445, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1305/16140], Loss: 0.8787, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1306/16140], Loss: 0.9898, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1307/16140], Loss: 0.9293, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1308/16140], Loss: 0.9259, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1309/16140], Loss: 0.9725, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1310/16140], Loss: 0.8592, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1311/16140], Loss: 1.4397, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1312/16140], Loss: 0.7596, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1313/16140], Loss: 1.2097, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1314/16140], Loss: 0.8906, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1315/16140], Loss: 0.9579, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1316/16140], Loss: 1.6199, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1317/16140], Loss: 1.1505, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1318/16140], Loss: 1.1989, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1319/16140], Loss: 0.9536, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1320/16140], Loss: 0.6799, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1321/16140], Loss: 1.3509, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1322/16140], Loss: 1.1587, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1323/16140], Loss: 0.8942, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1324/16140], Loss: 1.5763, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1325/16140], Loss: 1.1198, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1326/16140], Loss: 1.0502, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1327/16140], Loss: 0.8217, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1328/16140], Loss: 0.8606, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1329/16140], Loss: 0.9677, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1330/16140], Loss: 1.5520, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1331/16140], Loss: 1.0683, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1332/16140], Loss: 1.0509, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1333/16140], Loss: 1.3212, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1334/16140], Loss: 1.0067, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1335/16140], Loss: 0.9090, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1336/16140], Loss: 0.8867, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1337/16140], Loss: 1.1195, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1338/16140], Loss: 1.4122, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1339/16140], Loss: 1.2894, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1340/16140], Loss: 1.3013, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1341/16140], Loss: 1.0893, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1342/16140], Loss: 0.8392, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1343/16140], Loss: 0.9837, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1344/16140], Loss: 0.7444, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1345/16140], Loss: 1.2024, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1346/16140], Loss: 1.1316, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1347/16140], Loss: 0.8692, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1348/16140], Loss: 1.1669, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1349/16140], Loss: 1.1290, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1350/16140], Loss: 1.0047, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1351/16140], Loss: 1.6618, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1352/16140], Loss: 0.9434, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1353/16140], Loss: 0.9378, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1354/16140], Loss: 1.2997, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1355/16140], Loss: 1.2101, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1356/16140], Loss: 0.9454, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1357/16140], Loss: 1.1463, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1358/16140], Loss: 0.9324, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1359/16140], Loss: 1.0042, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1360/16140], Loss: 0.9343, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1361/16140], Loss: 1.4482, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1362/16140], Loss: 1.4067, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1363/16140], Loss: 1.3933, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1364/16140], Loss: 0.8786, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1365/16140], Loss: 1.3312, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1366/16140], Loss: 0.7881, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1367/16140], Loss: 0.6666, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1368/16140], Loss: 1.1723, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1369/16140], Loss: 1.3667, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1370/16140], Loss: 1.3616, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1371/16140], Loss: 1.0751, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1372/16140], Loss: 0.9102, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1373/16140], Loss: 1.0098, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1374/16140], Loss: 0.8338, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1375/16140], Loss: 0.9680, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1376/16140], Loss: 0.8695, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1377/16140], Loss: 0.9224, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1378/16140], Loss: 1.2053, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1379/16140], Loss: 1.1430, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1380/16140], Loss: 0.9358, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1381/16140], Loss: 0.5733, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1382/16140], Loss: 1.1115, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1383/16140], Loss: 1.1619, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1384/16140], Loss: 0.9917, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1385/16140], Loss: 0.8561, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1386/16140], Loss: 1.1577, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1387/16140], Loss: 1.1691, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1388/16140], Loss: 0.9199, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1389/16140], Loss: 0.9273, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1390/16140], Loss: 0.7953, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1391/16140], Loss: 1.1635, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1392/16140], Loss: 0.9965, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1393/16140], Loss: 1.1195, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1394/16140], Loss: 0.8452, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1395/16140], Loss: 0.8106, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1396/16140], Loss: 0.9679, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1397/16140], Loss: 1.3208, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1398/16140], Loss: 0.7729, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1399/16140], Loss: 1.0064, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1400/16140], Loss: 1.1053, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1401/16140], Loss: 1.1237, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1402/16140], Loss: 1.3916, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1403/16140], Loss: 0.8512, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1404/16140], Loss: 1.3410, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1405/16140], Loss: 1.3110, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1406/16140], Loss: 1.0626, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1407/16140], Loss: 1.5478, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1408/16140], Loss: 1.1584, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1409/16140], Loss: 1.1659, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1410/16140], Loss: 1.0268, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1411/16140], Loss: 1.0737, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1412/16140], Loss: 0.8859, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1413/16140], Loss: 1.0157, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1414/16140], Loss: 1.2192, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1415/16140], Loss: 0.9519, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1416/16140], Loss: 1.0037, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1417/16140], Loss: 1.2765, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1418/16140], Loss: 1.1021, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1419/16140], Loss: 1.2299, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1420/16140], Loss: 0.9094, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1421/16140], Loss: 0.9113, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1422/16140], Loss: 0.9972, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1423/16140], Loss: 0.9758, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1424/16140], Loss: 0.8199, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1425/16140], Loss: 1.0917, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1426/16140], Loss: 0.7250, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1427/16140], Loss: 1.1119, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1428/16140], Loss: 0.7394, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1429/16140], Loss: 1.7087, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1430/16140], Loss: 0.8169, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1431/16140], Loss: 1.3825, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1432/16140], Loss: 1.4273, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1433/16140], Loss: 1.0502, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1434/16140], Loss: 0.8586, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1435/16140], Loss: 0.9816, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1436/16140], Loss: 1.0869, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1437/16140], Loss: 1.3663, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1438/16140], Loss: 0.9417, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1439/16140], Loss: 0.8237, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1440/16140], Loss: 1.0093, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1441/16140], Loss: 0.9547, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1442/16140], Loss: 1.0044, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1443/16140], Loss: 0.8938, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1444/16140], Loss: 1.0216, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1445/16140], Loss: 1.3421, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1446/16140], Loss: 0.7818, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1447/16140], Loss: 1.6957, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1448/16140], Loss: 0.9949, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1449/16140], Loss: 0.7758, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1450/16140], Loss: 0.9356, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1451/16140], Loss: 1.1105, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1452/16140], Loss: 1.0934, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1453/16140], Loss: 1.2714, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1454/16140], Loss: 1.1226, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1455/16140], Loss: 1.3562, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1456/16140], Loss: 1.1141, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1457/16140], Loss: 1.3251, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1458/16140], Loss: 1.2693, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1459/16140], Loss: 0.8573, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1460/16140], Loss: 1.1956, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1461/16140], Loss: 1.0259, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1462/16140], Loss: 0.7822, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1463/16140], Loss: 0.8101, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1464/16140], Loss: 1.1331, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1465/16140], Loss: 1.1969, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1466/16140], Loss: 1.2405, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1467/16140], Loss: 0.9858, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1468/16140], Loss: 0.9505, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1469/16140], Loss: 0.9643, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1470/16140], Loss: 0.8357, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1471/16140], Loss: 1.2170, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1472/16140], Loss: 1.0478, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1473/16140], Loss: 1.2282, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1474/16140], Loss: 1.0927, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1475/16140], Loss: 1.0642, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1476/16140], Loss: 0.7910, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1477/16140], Loss: 0.9020, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1478/16140], Loss: 1.0364, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1479/16140], Loss: 1.0482, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1480/16140], Loss: 0.6714, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1481/16140], Loss: 1.2154, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1482/16140], Loss: 0.9253, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1483/16140], Loss: 1.1683, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1484/16140], Loss: 1.1837, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1485/16140], Loss: 0.9442, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1486/16140], Loss: 1.0742, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1487/16140], Loss: 1.2645, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1488/16140], Loss: 0.9110, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1489/16140], Loss: 0.9449, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1490/16140], Loss: 1.2382, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1491/16140], Loss: 1.1472, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1492/16140], Loss: 0.9774, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1493/16140], Loss: 1.5682, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1494/16140], Loss: 1.2057, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1495/16140], Loss: 0.9266, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1496/16140], Loss: 1.5487, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1497/16140], Loss: 1.0216, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1498/16140], Loss: 0.9411, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1499/16140], Loss: 1.3398, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1500/16140], Loss: 1.0034, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1501/16140], Loss: 0.7165, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1502/16140], Loss: 1.1441, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1503/16140], Loss: 0.9036, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1504/16140], Loss: 1.1651, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1505/16140], Loss: 0.9099, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1506/16140], Loss: 0.9912, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1507/16140], Loss: 1.2446, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1508/16140], Loss: 0.9780, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1509/16140], Loss: 1.6142, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1510/16140], Loss: 1.1664, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1511/16140], Loss: 1.3214, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1512/16140], Loss: 1.0893, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1513/16140], Loss: 0.9706, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1514/16140], Loss: 1.8038, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1515/16140], Loss: 1.3420, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1516/16140], Loss: 1.2778, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1517/16140], Loss: 0.8478, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1518/16140], Loss: 1.0343, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1519/16140], Loss: 1.2425, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1520/16140], Loss: 0.9844, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1521/16140], Loss: 1.0223, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1522/16140], Loss: 1.0567, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1523/16140], Loss: 1.6136, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1524/16140], Loss: 0.5771, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1525/16140], Loss: 0.8456, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1526/16140], Loss: 1.1823, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1527/16140], Loss: 0.9377, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1528/16140], Loss: 0.6742, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1529/16140], Loss: 1.1013, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1530/16140], Loss: 1.1581, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1531/16140], Loss: 0.8389, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1532/16140], Loss: 1.0268, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1533/16140], Loss: 1.5059, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1534/16140], Loss: 1.0536, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1535/16140], Loss: 1.5548, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1536/16140], Loss: 0.9137, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1537/16140], Loss: 1.1883, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1538/16140], Loss: 0.7195, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1539/16140], Loss: 0.9769, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1540/16140], Loss: 1.4686, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1541/16140], Loss: 1.5506, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1542/16140], Loss: 1.1885, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1543/16140], Loss: 1.0249, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1544/16140], Loss: 1.0090, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1545/16140], Loss: 1.1322, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1546/16140], Loss: 1.1483, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1547/16140], Loss: 1.0354, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1548/16140], Loss: 1.1857, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1549/16140], Loss: 1.1059, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1550/16140], Loss: 0.9244, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1551/16140], Loss: 1.0260, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1552/16140], Loss: 1.0609, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1553/16140], Loss: 0.8149, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1554/16140], Loss: 0.9878, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1555/16140], Loss: 1.0399, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1556/16140], Loss: 1.3567, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1557/16140], Loss: 0.9656, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1558/16140], Loss: 1.0740, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1559/16140], Loss: 1.0601, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1560/16140], Loss: 1.1649, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1561/16140], Loss: 0.8793, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1562/16140], Loss: 0.9840, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1563/16140], Loss: 0.9341, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1564/16140], Loss: 0.5880, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1565/16140], Loss: 1.0256, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1566/16140], Loss: 1.1840, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1567/16140], Loss: 1.1083, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1568/16140], Loss: 0.8674, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1569/16140], Loss: 0.7688, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1570/16140], Loss: 0.7209, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1571/16140], Loss: 1.3769, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1572/16140], Loss: 0.8233, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1573/16140], Loss: 1.0577, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1574/16140], Loss: 1.2140, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1575/16140], Loss: 1.4551, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1576/16140], Loss: 1.5404, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1577/16140], Loss: 1.0890, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1578/16140], Loss: 0.8875, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1579/16140], Loss: 0.8544, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1580/16140], Loss: 0.9026, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1581/16140], Loss: 1.4510, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1582/16140], Loss: 1.6261, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1583/16140], Loss: 0.9881, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1584/16140], Loss: 1.2511, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1585/16140], Loss: 1.3457, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1586/16140], Loss: 1.3275, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1587/16140], Loss: 1.3516, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1588/16140], Loss: 0.8303, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1589/16140], Loss: 1.4741, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1590/16140], Loss: 1.4545, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1591/16140], Loss: 0.8817, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1592/16140], Loss: 0.9436, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1593/16140], Loss: 0.7786, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1594/16140], Loss: 1.0113, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1595/16140], Loss: 0.7548, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1596/16140], Loss: 1.0782, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1597/16140], Loss: 0.9552, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1598/16140], Loss: 1.2267, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1599/16140], Loss: 1.3426, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1600/16140], Loss: 1.1413, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1601/16140], Loss: 1.3786, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1602/16140], Loss: 1.1422, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1603/16140], Loss: 1.2424, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1604/16140], Loss: 1.4425, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1605/16140], Loss: 1.3306, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1606/16140], Loss: 0.9249, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1607/16140], Loss: 1.2876, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1608/16140], Loss: 0.7513, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1609/16140], Loss: 1.2234, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1610/16140], Loss: 0.9864, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1611/16140], Loss: 1.1047, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1612/16140], Loss: 0.8954, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1613/16140], Loss: 1.4093, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [9/10], Step [1614/16140], Loss: 1.0625, Accuracy: 71.43%, Iteration Time: 0.02 seconds
Epoch [9/10], Loss: 1.0658, Accuracy: 75.66%
Training finished!
Epoch [10/10], Step [1/16140], Loss: 1.0228, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [2/16140], Loss: 1.1054, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [3/16140], Loss: 0.9806, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [4/16140], Loss: 0.9273, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [5/16140], Loss: 1.0812, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [6/16140], Loss: 1.2033, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [7/16140], Loss: 0.8388, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [8/16140], Loss: 0.6780, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [9/16140], Loss: 0.9644, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [10/16140], Loss: 0.9367, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [11/16140], Loss: 0.5708, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [12/16140], Loss: 1.2257, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [13/16140], Loss: 1.0632, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [14/16140], Loss: 0.8780, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [15/16140], Loss: 1.0116, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [16/16140], Loss: 0.7833, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [17/16140], Loss: 0.9290, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [18/16140], Loss: 0.7834, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [19/16140], Loss: 1.0059, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [20/16140], Loss: 1.0262, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [21/16140], Loss: 1.2464, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [22/16140], Loss: 1.0659, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [23/16140], Loss: 0.9625, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [24/16140], Loss: 0.7753, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [25/16140], Loss: 0.9957, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [26/16140], Loss: 0.8518, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [27/16140], Loss: 1.0636, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [28/16140], Loss: 0.9457, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [29/16140], Loss: 1.0641, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [30/16140], Loss: 1.3690, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [31/16140], Loss: 0.8777, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [32/16140], Loss: 0.9505, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [33/16140], Loss: 0.6451, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [34/16140], Loss: 1.0420, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [35/16140], Loss: 0.7784, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [36/16140], Loss: 0.8007, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [37/16140], Loss: 0.8884, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [38/16140], Loss: 1.2741, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [39/16140], Loss: 0.9316, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [40/16140], Loss: 1.2402, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [41/16140], Loss: 0.9833, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [42/16140], Loss: 1.1813, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [43/16140], Loss: 1.1294, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [44/16140], Loss: 1.1663, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [45/16140], Loss: 0.6739, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [46/16140], Loss: 1.0669, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [47/16140], Loss: 1.1877, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [48/16140], Loss: 1.4453, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [49/16140], Loss: 0.8950, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [50/16140], Loss: 1.1141, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [51/16140], Loss: 0.8849, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [52/16140], Loss: 0.9313, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [53/16140], Loss: 1.5242, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [54/16140], Loss: 0.9988, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [55/16140], Loss: 1.3668, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [56/16140], Loss: 1.0072, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [57/16140], Loss: 1.1141, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [58/16140], Loss: 1.1350, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [59/16140], Loss: 0.8725, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [60/16140], Loss: 1.0001, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [61/16140], Loss: 1.2338, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [62/16140], Loss: 1.2499, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [63/16140], Loss: 0.8445, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [64/16140], Loss: 0.9095, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [65/16140], Loss: 1.0698, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [66/16140], Loss: 1.0631, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [67/16140], Loss: 0.9620, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [68/16140], Loss: 0.9820, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [69/16140], Loss: 0.7974, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [70/16140], Loss: 1.0279, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [71/16140], Loss: 0.9389, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [72/16140], Loss: 1.0783, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [73/16140], Loss: 1.3914, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [74/16140], Loss: 1.0024, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [75/16140], Loss: 1.0687, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [76/16140], Loss: 0.7827, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [77/16140], Loss: 1.0546, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [78/16140], Loss: 1.1493, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [79/16140], Loss: 1.1110, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [80/16140], Loss: 1.2870, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [81/16140], Loss: 1.4753, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [82/16140], Loss: 0.7658, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [83/16140], Loss: 0.8079, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [84/16140], Loss: 1.1341, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [85/16140], Loss: 0.7416, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [86/16140], Loss: 0.7825, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [87/16140], Loss: 1.4013, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [88/16140], Loss: 0.7774, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [89/16140], Loss: 1.2294, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [90/16140], Loss: 1.1103, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [91/16140], Loss: 0.9649, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [92/16140], Loss: 1.7385, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [93/16140], Loss: 1.1265, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [94/16140], Loss: 0.9445, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [95/16140], Loss: 0.9655, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [96/16140], Loss: 0.8537, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [97/16140], Loss: 1.0910, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [98/16140], Loss: 1.1458, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [99/16140], Loss: 0.9675, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [100/16140], Loss: 1.1323, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [101/16140], Loss: 0.8888, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [102/16140], Loss: 0.8727, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [103/16140], Loss: 0.9907, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [104/16140], Loss: 0.9679, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [105/16140], Loss: 1.1775, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [106/16140], Loss: 0.7892, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [107/16140], Loss: 1.1239, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [108/16140], Loss: 1.0430, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [109/16140], Loss: 1.0364, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [110/16140], Loss: 0.9748, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [111/16140], Loss: 0.9295, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [112/16140], Loss: 0.9045, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [113/16140], Loss: 1.0076, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [114/16140], Loss: 1.1643, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [115/16140], Loss: 1.1978, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [116/16140], Loss: 0.9094, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [117/16140], Loss: 0.9860, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [118/16140], Loss: 1.0685, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [119/16140], Loss: 0.8293, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [120/16140], Loss: 1.1746, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [121/16140], Loss: 1.3032, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [122/16140], Loss: 1.3198, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [123/16140], Loss: 1.6062, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [124/16140], Loss: 0.9613, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [125/16140], Loss: 1.3298, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [126/16140], Loss: 0.6161, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [127/16140], Loss: 0.8927, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [128/16140], Loss: 0.9562, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [129/16140], Loss: 1.0364, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [130/16140], Loss: 0.9296, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [131/16140], Loss: 0.9394, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [132/16140], Loss: 1.5010, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [133/16140], Loss: 0.8472, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [134/16140], Loss: 0.8582, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [135/16140], Loss: 0.8553, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [136/16140], Loss: 1.0480, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [137/16140], Loss: 0.9720, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [138/16140], Loss: 0.9484, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [139/16140], Loss: 1.2923, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [140/16140], Loss: 0.9208, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [141/16140], Loss: 1.4064, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [142/16140], Loss: 1.1138, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [143/16140], Loss: 1.1493, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [144/16140], Loss: 0.7971, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [145/16140], Loss: 1.1159, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [146/16140], Loss: 0.7090, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [147/16140], Loss: 1.0614, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [148/16140], Loss: 0.6058, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [149/16140], Loss: 1.2999, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [150/16140], Loss: 0.8174, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [151/16140], Loss: 1.1520, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [152/16140], Loss: 1.0289, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [153/16140], Loss: 1.0627, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [154/16140], Loss: 0.6715, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [155/16140], Loss: 1.0231, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [156/16140], Loss: 0.8762, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [157/16140], Loss: 0.7215, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [158/16140], Loss: 0.8043, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [159/16140], Loss: 0.6520, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [160/16140], Loss: 0.9275, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [161/16140], Loss: 0.9971, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [162/16140], Loss: 0.7458, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [163/16140], Loss: 0.9375, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [164/16140], Loss: 0.8920, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [165/16140], Loss: 0.5047, Accuracy: 90.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [166/16140], Loss: 0.6703, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [167/16140], Loss: 1.3268, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [168/16140], Loss: 0.8011, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [169/16140], Loss: 1.0395, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [170/16140], Loss: 0.6034, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [171/16140], Loss: 0.9958, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [172/16140], Loss: 1.2340, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [173/16140], Loss: 0.8644, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [174/16140], Loss: 1.2105, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [175/16140], Loss: 0.9831, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [176/16140], Loss: 1.3282, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [177/16140], Loss: 0.7906, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [178/16140], Loss: 0.8300, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [179/16140], Loss: 1.2260, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [180/16140], Loss: 1.0985, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [181/16140], Loss: 0.9823, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [182/16140], Loss: 0.8225, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [183/16140], Loss: 0.9652, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [184/16140], Loss: 0.8900, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [185/16140], Loss: 0.9791, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [186/16140], Loss: 0.8839, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [187/16140], Loss: 0.8789, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [188/16140], Loss: 1.1113, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [189/16140], Loss: 1.2181, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [190/16140], Loss: 1.0432, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [191/16140], Loss: 1.3546, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [192/16140], Loss: 1.0193, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [193/16140], Loss: 1.2103, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [194/16140], Loss: 1.0194, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [195/16140], Loss: 1.2211, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [196/16140], Loss: 1.2491, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [197/16140], Loss: 0.9189, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [198/16140], Loss: 1.1994, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [199/16140], Loss: 1.0939, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [200/16140], Loss: 0.8990, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [201/16140], Loss: 1.0683, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [202/16140], Loss: 0.8282, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [203/16140], Loss: 0.9120, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [204/16140], Loss: 0.9019, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [205/16140], Loss: 0.9413, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [206/16140], Loss: 1.0545, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [207/16140], Loss: 1.1365, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [208/16140], Loss: 1.3077, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [209/16140], Loss: 1.0871, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [210/16140], Loss: 0.9658, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [211/16140], Loss: 0.7809, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [212/16140], Loss: 1.1493, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [213/16140], Loss: 0.8198, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [214/16140], Loss: 1.2681, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [215/16140], Loss: 1.2700, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [216/16140], Loss: 0.7711, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [217/16140], Loss: 1.1581, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [218/16140], Loss: 0.8772, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [219/16140], Loss: 0.8507, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [220/16140], Loss: 1.0792, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [221/16140], Loss: 0.7563, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [222/16140], Loss: 1.2218, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [223/16140], Loss: 0.7497, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [224/16140], Loss: 1.2115, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [225/16140], Loss: 0.7004, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [226/16140], Loss: 0.8988, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [227/16140], Loss: 0.7450, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [228/16140], Loss: 0.9861, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [229/16140], Loss: 1.0909, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [230/16140], Loss: 0.8876, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [231/16140], Loss: 1.0130, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [232/16140], Loss: 0.6091, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [233/16140], Loss: 1.1786, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [234/16140], Loss: 1.0197, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [235/16140], Loss: 1.1105, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [236/16140], Loss: 0.9210, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [237/16140], Loss: 0.8592, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [238/16140], Loss: 0.8186, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [239/16140], Loss: 1.1548, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [240/16140], Loss: 0.9253, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [241/16140], Loss: 1.2537, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [242/16140], Loss: 1.1364, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [243/16140], Loss: 1.2641, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [244/16140], Loss: 0.9641, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [245/16140], Loss: 1.5400, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [246/16140], Loss: 1.5583, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [247/16140], Loss: 1.1171, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [248/16140], Loss: 1.2193, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [249/16140], Loss: 1.0500, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [250/16140], Loss: 1.0187, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [251/16140], Loss: 0.9959, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [252/16140], Loss: 0.8607, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [253/16140], Loss: 1.3126, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [254/16140], Loss: 1.2112, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [255/16140], Loss: 1.0964, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [256/16140], Loss: 1.2044, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [257/16140], Loss: 1.1226, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [258/16140], Loss: 1.3256, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [259/16140], Loss: 1.2931, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [260/16140], Loss: 1.0514, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [261/16140], Loss: 1.0813, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [262/16140], Loss: 0.9915, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [263/16140], Loss: 0.9759, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [264/16140], Loss: 1.1284, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [265/16140], Loss: 0.9637, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [266/16140], Loss: 1.1068, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [267/16140], Loss: 0.9199, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [268/16140], Loss: 0.8259, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [269/16140], Loss: 0.9342, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [270/16140], Loss: 1.1450, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [271/16140], Loss: 1.0011, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [272/16140], Loss: 0.9986, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [273/16140], Loss: 0.9588, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [274/16140], Loss: 0.8644, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [275/16140], Loss: 0.7526, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [276/16140], Loss: 0.9284, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [277/16140], Loss: 0.7645, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [278/16140], Loss: 1.2621, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [279/16140], Loss: 1.0347, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [280/16140], Loss: 0.8958, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [281/16140], Loss: 1.2158, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [282/16140], Loss: 0.7915, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [283/16140], Loss: 0.7163, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [284/16140], Loss: 0.7510, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [285/16140], Loss: 1.0937, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [286/16140], Loss: 0.8179, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [287/16140], Loss: 0.9629, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [288/16140], Loss: 0.9546, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [289/16140], Loss: 1.0449, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [290/16140], Loss: 1.1625, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [291/16140], Loss: 0.8047, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [292/16140], Loss: 1.0740, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [293/16140], Loss: 1.0320, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [294/16140], Loss: 1.1172, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [295/16140], Loss: 0.8297, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [296/16140], Loss: 0.9930, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [297/16140], Loss: 0.8132, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [298/16140], Loss: 0.9147, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [299/16140], Loss: 1.0888, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [300/16140], Loss: 0.8927, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [301/16140], Loss: 1.4633, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [302/16140], Loss: 0.5803, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [303/16140], Loss: 0.7334, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [304/16140], Loss: 0.8017, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [305/16140], Loss: 1.3943, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [306/16140], Loss: 0.7329, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [307/16140], Loss: 0.7953, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [308/16140], Loss: 0.8382, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [309/16140], Loss: 0.9957, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [310/16140], Loss: 1.0869, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [311/16140], Loss: 0.9466, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [312/16140], Loss: 0.9326, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [313/16140], Loss: 0.7980, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [314/16140], Loss: 1.1607, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [315/16140], Loss: 1.0965, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [316/16140], Loss: 0.9335, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [317/16140], Loss: 0.9057, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [318/16140], Loss: 1.1259, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [319/16140], Loss: 1.0758, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [320/16140], Loss: 1.2063, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [321/16140], Loss: 1.1991, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [322/16140], Loss: 0.9904, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [323/16140], Loss: 1.2148, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [324/16140], Loss: 0.4909, Accuracy: 90.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [325/16140], Loss: 0.9688, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [326/16140], Loss: 1.4041, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [327/16140], Loss: 1.1117, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [328/16140], Loss: 1.0946, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [329/16140], Loss: 1.2318, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [330/16140], Loss: 1.1080, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [331/16140], Loss: 1.0767, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [332/16140], Loss: 0.9985, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [333/16140], Loss: 1.1529, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [334/16140], Loss: 0.6754, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [335/16140], Loss: 0.9019, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [336/16140], Loss: 1.0589, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [337/16140], Loss: 1.0991, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [338/16140], Loss: 1.0440, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [339/16140], Loss: 0.8432, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [340/16140], Loss: 1.0041, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [341/16140], Loss: 1.3600, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [342/16140], Loss: 0.6747, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [343/16140], Loss: 0.9268, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [344/16140], Loss: 1.0892, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [345/16140], Loss: 1.0116, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [346/16140], Loss: 1.1354, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [347/16140], Loss: 1.3079, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [348/16140], Loss: 1.1361, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [349/16140], Loss: 0.9226, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [350/16140], Loss: 1.0345, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [351/16140], Loss: 0.7740, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [352/16140], Loss: 1.1630, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [353/16140], Loss: 0.8754, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [354/16140], Loss: 1.3318, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [355/16140], Loss: 0.5997, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [356/16140], Loss: 0.6778, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [357/16140], Loss: 0.8673, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [358/16140], Loss: 1.2425, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [359/16140], Loss: 1.0551, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [360/16140], Loss: 1.3562, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [361/16140], Loss: 0.6220, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [362/16140], Loss: 1.2823, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [363/16140], Loss: 1.1111, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [364/16140], Loss: 0.8534, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [365/16140], Loss: 0.9179, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [366/16140], Loss: 0.7760, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [367/16140], Loss: 0.9092, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [368/16140], Loss: 1.4071, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [369/16140], Loss: 1.3802, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [370/16140], Loss: 0.9238, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [371/16140], Loss: 0.6990, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [372/16140], Loss: 0.7224, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [373/16140], Loss: 1.0767, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [374/16140], Loss: 1.3780, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [375/16140], Loss: 0.8624, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [376/16140], Loss: 0.8322, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [377/16140], Loss: 0.8598, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [378/16140], Loss: 0.9914, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [379/16140], Loss: 0.7702, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [380/16140], Loss: 1.1700, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [381/16140], Loss: 1.1010, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [382/16140], Loss: 1.0815, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [383/16140], Loss: 1.1717, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [384/16140], Loss: 0.9909, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [385/16140], Loss: 0.6810, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [386/16140], Loss: 0.6828, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [387/16140], Loss: 1.2336, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [388/16140], Loss: 1.0098, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [389/16140], Loss: 0.6698, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [390/16140], Loss: 0.9147, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [391/16140], Loss: 1.0374, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [392/16140], Loss: 1.1004, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [393/16140], Loss: 0.7770, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [394/16140], Loss: 0.8990, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [395/16140], Loss: 1.0818, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [396/16140], Loss: 1.0820, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [397/16140], Loss: 0.8234, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [398/16140], Loss: 1.0880, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [399/16140], Loss: 0.9586, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [400/16140], Loss: 0.9693, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [401/16140], Loss: 0.9310, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [402/16140], Loss: 1.3111, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [403/16140], Loss: 1.2194, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [404/16140], Loss: 1.1643, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [405/16140], Loss: 1.1606, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [406/16140], Loss: 1.2107, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [407/16140], Loss: 0.8825, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [408/16140], Loss: 1.0635, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [409/16140], Loss: 1.0593, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [410/16140], Loss: 1.2443, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [411/16140], Loss: 1.1775, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [412/16140], Loss: 1.0019, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [413/16140], Loss: 1.2014, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [414/16140], Loss: 1.1911, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [415/16140], Loss: 0.9484, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [416/16140], Loss: 0.9384, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [417/16140], Loss: 1.0978, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [418/16140], Loss: 1.3452, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [419/16140], Loss: 1.0600, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [420/16140], Loss: 0.9016, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [421/16140], Loss: 0.9705, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [422/16140], Loss: 0.8914, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [423/16140], Loss: 0.9161, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [424/16140], Loss: 0.8974, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [425/16140], Loss: 0.9235, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [426/16140], Loss: 1.2532, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [427/16140], Loss: 1.1976, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [428/16140], Loss: 1.0103, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [429/16140], Loss: 0.9804, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [430/16140], Loss: 1.3780, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [431/16140], Loss: 1.0771, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [432/16140], Loss: 1.0323, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [433/16140], Loss: 1.0874, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [434/16140], Loss: 0.8325, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [435/16140], Loss: 0.9846, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [436/16140], Loss: 1.2346, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [437/16140], Loss: 1.2334, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [438/16140], Loss: 0.9992, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [439/16140], Loss: 1.0218, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [440/16140], Loss: 0.7332, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [441/16140], Loss: 1.1489, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [442/16140], Loss: 1.2665, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [443/16140], Loss: 1.0937, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [444/16140], Loss: 1.3150, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [445/16140], Loss: 0.7851, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [446/16140], Loss: 0.5634, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [447/16140], Loss: 1.0515, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [448/16140], Loss: 1.6021, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [449/16140], Loss: 1.0675, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [450/16140], Loss: 1.2498, Accuracy: 60.00%, Iteration Time: 0.03 seconds
Epoch [10/10], Step [451/16140], Loss: 1.2893, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [452/16140], Loss: 1.2978, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [453/16140], Loss: 1.2303, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [454/16140], Loss: 1.4717, Accuracy: 36.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [455/16140], Loss: 0.8926, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [456/16140], Loss: 0.9080, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [457/16140], Loss: 0.7921, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [458/16140], Loss: 1.3911, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [459/16140], Loss: 0.9211, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [460/16140], Loss: 0.7955, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [461/16140], Loss: 1.1883, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [462/16140], Loss: 0.9286, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [463/16140], Loss: 0.8975, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [464/16140], Loss: 0.9928, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [465/16140], Loss: 1.2934, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [466/16140], Loss: 1.0168, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [467/16140], Loss: 0.9162, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [468/16140], Loss: 1.1291, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [469/16140], Loss: 0.9755, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [470/16140], Loss: 1.0275, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [471/16140], Loss: 0.8643, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [472/16140], Loss: 1.0138, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [473/16140], Loss: 0.9356, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [474/16140], Loss: 1.1010, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [475/16140], Loss: 1.1812, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [476/16140], Loss: 1.0438, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [477/16140], Loss: 1.0737, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [478/16140], Loss: 0.8613, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [479/16140], Loss: 1.0422, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [480/16140], Loss: 1.4025, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [481/16140], Loss: 0.9999, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [482/16140], Loss: 1.0609, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [483/16140], Loss: 0.8738, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [484/16140], Loss: 1.2451, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [485/16140], Loss: 0.8758, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [486/16140], Loss: 0.9351, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [487/16140], Loss: 1.0758, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [488/16140], Loss: 1.3500, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [489/16140], Loss: 1.1625, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [490/16140], Loss: 1.0349, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [491/16140], Loss: 1.4550, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [492/16140], Loss: 1.2030, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [493/16140], Loss: 1.0392, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [494/16140], Loss: 1.2899, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [495/16140], Loss: 1.2095, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [496/16140], Loss: 1.2933, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [497/16140], Loss: 0.9191, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [498/16140], Loss: 1.0309, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [499/16140], Loss: 0.8098, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [500/16140], Loss: 1.0066, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [501/16140], Loss: 0.8853, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [502/16140], Loss: 0.7382, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [503/16140], Loss: 0.9353, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [504/16140], Loss: 0.7371, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [505/16140], Loss: 1.2388, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [506/16140], Loss: 0.9618, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [507/16140], Loss: 1.4363, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [508/16140], Loss: 1.3354, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [509/16140], Loss: 1.1856, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [510/16140], Loss: 0.8245, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [511/16140], Loss: 0.7042, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [512/16140], Loss: 0.6861, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [513/16140], Loss: 0.9080, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [514/16140], Loss: 0.7495, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [515/16140], Loss: 0.9339, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [516/16140], Loss: 0.9710, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [517/16140], Loss: 0.6920, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [518/16140], Loss: 0.8530, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [519/16140], Loss: 0.9421, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [520/16140], Loss: 0.6849, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [521/16140], Loss: 1.0213, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [522/16140], Loss: 0.9600, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [523/16140], Loss: 0.6347, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [524/16140], Loss: 0.8781, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [525/16140], Loss: 0.8229, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [526/16140], Loss: 1.2063, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [527/16140], Loss: 1.2481, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [528/16140], Loss: 1.0251, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [529/16140], Loss: 0.9251, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [530/16140], Loss: 1.0179, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [531/16140], Loss: 1.0847, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [532/16140], Loss: 0.9964, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [533/16140], Loss: 0.7268, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [534/16140], Loss: 0.8667, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [535/16140], Loss: 0.7584, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [536/16140], Loss: 0.9146, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [537/16140], Loss: 1.1985, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [538/16140], Loss: 0.7357, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [539/16140], Loss: 0.7741, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [540/16140], Loss: 0.8425, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [541/16140], Loss: 0.7334, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [542/16140], Loss: 1.0960, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [543/16140], Loss: 1.1644, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [544/16140], Loss: 0.9055, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [545/16140], Loss: 1.4121, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [546/16140], Loss: 1.4212, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [547/16140], Loss: 1.0467, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [548/16140], Loss: 1.0742, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [549/16140], Loss: 1.1484, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [550/16140], Loss: 0.9287, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [551/16140], Loss: 0.8327, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [552/16140], Loss: 1.0519, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [553/16140], Loss: 1.1282, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [554/16140], Loss: 0.7967, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [555/16140], Loss: 1.5010, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [556/16140], Loss: 1.3971, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [557/16140], Loss: 1.1267, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [558/16140], Loss: 1.0865, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [559/16140], Loss: 1.0248, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [560/16140], Loss: 0.9961, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [561/16140], Loss: 1.0920, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [562/16140], Loss: 1.0176, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [563/16140], Loss: 1.0827, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [564/16140], Loss: 0.9707, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [565/16140], Loss: 0.8196, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [566/16140], Loss: 1.0996, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [567/16140], Loss: 0.6345, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [568/16140], Loss: 1.2151, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [569/16140], Loss: 1.0133, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [570/16140], Loss: 0.8179, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [571/16140], Loss: 1.0550, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [572/16140], Loss: 0.9117, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [573/16140], Loss: 1.6698, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [574/16140], Loss: 0.9598, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [575/16140], Loss: 0.8946, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [576/16140], Loss: 1.0752, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [577/16140], Loss: 1.1761, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [578/16140], Loss: 1.0062, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [579/16140], Loss: 0.8231, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [580/16140], Loss: 0.7801, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [581/16140], Loss: 0.9497, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [582/16140], Loss: 0.7814, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [583/16140], Loss: 1.1435, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [584/16140], Loss: 0.8242, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [585/16140], Loss: 0.9791, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [586/16140], Loss: 0.8594, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [587/16140], Loss: 1.0549, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [588/16140], Loss: 0.9090, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [589/16140], Loss: 0.5150, Accuracy: 90.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [590/16140], Loss: 1.0485, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [591/16140], Loss: 0.7684, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [592/16140], Loss: 0.9829, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [593/16140], Loss: 1.3189, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [594/16140], Loss: 0.7411, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [595/16140], Loss: 1.3503, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [596/16140], Loss: 1.2691, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [597/16140], Loss: 0.9793, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [598/16140], Loss: 1.0506, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [599/16140], Loss: 0.7954, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [600/16140], Loss: 0.9588, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [601/16140], Loss: 0.7077, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [602/16140], Loss: 1.0361, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [603/16140], Loss: 0.8237, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [604/16140], Loss: 1.1937, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [605/16140], Loss: 0.9625, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [606/16140], Loss: 1.1993, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [607/16140], Loss: 0.9341, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [608/16140], Loss: 1.1394, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [609/16140], Loss: 1.1245, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [610/16140], Loss: 1.2043, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [611/16140], Loss: 1.0148, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [612/16140], Loss: 1.3459, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [613/16140], Loss: 1.0751, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [614/16140], Loss: 1.1847, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [615/16140], Loss: 1.3806, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [616/16140], Loss: 1.1295, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [617/16140], Loss: 0.9514, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [618/16140], Loss: 1.1054, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [619/16140], Loss: 0.9269, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [620/16140], Loss: 0.9854, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [621/16140], Loss: 1.0025, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [622/16140], Loss: 0.7708, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [623/16140], Loss: 0.9560, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [624/16140], Loss: 0.8045, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [625/16140], Loss: 1.6399, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [626/16140], Loss: 1.0260, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [627/16140], Loss: 0.9863, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [628/16140], Loss: 1.2121, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [629/16140], Loss: 0.9319, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [630/16140], Loss: 1.0629, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [631/16140], Loss: 0.7363, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [632/16140], Loss: 1.0746, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [633/16140], Loss: 1.1950, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [634/16140], Loss: 1.2935, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [635/16140], Loss: 1.0313, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [636/16140], Loss: 0.7828, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [637/16140], Loss: 0.8494, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [638/16140], Loss: 0.7399, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [639/16140], Loss: 1.3970, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [640/16140], Loss: 1.1320, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [641/16140], Loss: 1.0056, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [642/16140], Loss: 1.0327, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [643/16140], Loss: 1.1637, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [644/16140], Loss: 0.9925, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [645/16140], Loss: 1.3107, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [646/16140], Loss: 1.2005, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [647/16140], Loss: 1.0174, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [648/16140], Loss: 1.0080, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [649/16140], Loss: 0.8373, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [650/16140], Loss: 0.8294, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [651/16140], Loss: 0.8484, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [652/16140], Loss: 1.3063, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [653/16140], Loss: 0.9532, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [654/16140], Loss: 1.2284, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [655/16140], Loss: 1.0353, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [656/16140], Loss: 1.1096, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [657/16140], Loss: 0.9049, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [658/16140], Loss: 1.1283, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [659/16140], Loss: 1.3941, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [660/16140], Loss: 0.7823, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [661/16140], Loss: 1.3817, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [662/16140], Loss: 0.7302, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [663/16140], Loss: 1.0557, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [664/16140], Loss: 0.8333, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [665/16140], Loss: 1.0459, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [666/16140], Loss: 1.0028, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [667/16140], Loss: 1.0726, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [668/16140], Loss: 0.9940, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [669/16140], Loss: 1.3595, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [670/16140], Loss: 1.1278, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [671/16140], Loss: 1.0804, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [672/16140], Loss: 0.7605, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [673/16140], Loss: 1.0988, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [674/16140], Loss: 0.9951, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [675/16140], Loss: 0.8196, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [676/16140], Loss: 0.7658, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [677/16140], Loss: 0.9692, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [678/16140], Loss: 1.1479, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [679/16140], Loss: 1.2404, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [680/16140], Loss: 1.0658, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [681/16140], Loss: 0.8965, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [682/16140], Loss: 0.6552, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [683/16140], Loss: 1.2320, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [684/16140], Loss: 0.8956, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [685/16140], Loss: 1.0794, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [686/16140], Loss: 0.7973, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [687/16140], Loss: 0.9035, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [688/16140], Loss: 1.0489, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [689/16140], Loss: 0.8117, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [690/16140], Loss: 0.9020, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [691/16140], Loss: 0.9333, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [692/16140], Loss: 1.2823, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [693/16140], Loss: 1.2687, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [694/16140], Loss: 0.7234, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [695/16140], Loss: 0.9978, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [696/16140], Loss: 0.9088, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [697/16140], Loss: 0.8600, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [698/16140], Loss: 0.7707, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [699/16140], Loss: 1.0579, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [700/16140], Loss: 0.8731, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [701/16140], Loss: 1.0014, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [702/16140], Loss: 0.8926, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [703/16140], Loss: 1.1642, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [704/16140], Loss: 1.2128, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [705/16140], Loss: 0.9618, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [706/16140], Loss: 0.9512, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [707/16140], Loss: 0.9997, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [708/16140], Loss: 1.1263, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [709/16140], Loss: 1.0745, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [710/16140], Loss: 1.4444, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [711/16140], Loss: 1.3169, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [712/16140], Loss: 0.7613, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [713/16140], Loss: 1.0244, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [714/16140], Loss: 0.9204, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [715/16140], Loss: 1.0141, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [716/16140], Loss: 1.3348, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [717/16140], Loss: 1.2118, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [718/16140], Loss: 1.2545, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [719/16140], Loss: 0.9088, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [720/16140], Loss: 1.1587, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [721/16140], Loss: 0.8637, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [722/16140], Loss: 1.2797, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [723/16140], Loss: 1.2529, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [724/16140], Loss: 1.0899, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [725/16140], Loss: 1.2657, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [726/16140], Loss: 0.6992, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [727/16140], Loss: 1.5156, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [728/16140], Loss: 0.9593, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [729/16140], Loss: 1.3772, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [730/16140], Loss: 1.3622, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [731/16140], Loss: 0.9031, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [732/16140], Loss: 1.1750, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [733/16140], Loss: 0.9503, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [734/16140], Loss: 1.2638, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [735/16140], Loss: 1.2444, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [736/16140], Loss: 0.9645, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [737/16140], Loss: 0.9943, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [738/16140], Loss: 1.1375, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [739/16140], Loss: 0.7036, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [740/16140], Loss: 1.4152, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [741/16140], Loss: 0.9862, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [742/16140], Loss: 0.8164, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [743/16140], Loss: 0.9637, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [744/16140], Loss: 1.3392, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [745/16140], Loss: 0.8574, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [746/16140], Loss: 1.1954, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [747/16140], Loss: 1.1308, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [748/16140], Loss: 1.0903, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [749/16140], Loss: 1.0767, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [750/16140], Loss: 1.0977, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [751/16140], Loss: 1.3025, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [752/16140], Loss: 0.9188, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [753/16140], Loss: 1.0601, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [754/16140], Loss: 1.0900, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [755/16140], Loss: 0.9803, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [756/16140], Loss: 0.7238, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [757/16140], Loss: 0.9249, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [758/16140], Loss: 0.9829, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [759/16140], Loss: 1.0889, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [760/16140], Loss: 1.4635, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [761/16140], Loss: 0.9709, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [762/16140], Loss: 0.8243, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [763/16140], Loss: 1.1938, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [764/16140], Loss: 0.9643, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [765/16140], Loss: 1.2254, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [766/16140], Loss: 0.9114, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [767/16140], Loss: 1.0532, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [768/16140], Loss: 0.8406, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [769/16140], Loss: 1.1046, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [770/16140], Loss: 1.3752, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [771/16140], Loss: 0.6588, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [772/16140], Loss: 1.3209, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [773/16140], Loss: 1.0901, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [774/16140], Loss: 0.9286, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [775/16140], Loss: 1.1140, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [776/16140], Loss: 0.9678, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [777/16140], Loss: 0.7545, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [778/16140], Loss: 1.6605, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [779/16140], Loss: 1.4047, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [780/16140], Loss: 1.1728, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [781/16140], Loss: 1.0210, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [782/16140], Loss: 0.8739, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [783/16140], Loss: 1.1687, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [784/16140], Loss: 0.9794, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [785/16140], Loss: 0.7003, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [786/16140], Loss: 0.8992, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [787/16140], Loss: 0.9791, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [788/16140], Loss: 0.7733, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [789/16140], Loss: 0.8954, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [790/16140], Loss: 0.9269, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [791/16140], Loss: 0.8261, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [792/16140], Loss: 1.1841, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [793/16140], Loss: 0.7747, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [794/16140], Loss: 0.7690, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [795/16140], Loss: 1.1294, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [796/16140], Loss: 0.8921, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [797/16140], Loss: 1.0127, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [798/16140], Loss: 1.1901, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [799/16140], Loss: 0.7145, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [800/16140], Loss: 0.8909, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [801/16140], Loss: 1.1217, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [802/16140], Loss: 0.8029, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [803/16140], Loss: 1.0287, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [804/16140], Loss: 1.0078, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [805/16140], Loss: 1.2193, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [806/16140], Loss: 1.1448, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [807/16140], Loss: 1.1813, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [808/16140], Loss: 1.1945, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [809/16140], Loss: 0.6938, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [810/16140], Loss: 1.6973, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [811/16140], Loss: 1.3661, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [812/16140], Loss: 1.0756, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [813/16140], Loss: 1.5150, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [814/16140], Loss: 1.0491, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [815/16140], Loss: 1.2104, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [816/16140], Loss: 1.1939, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [817/16140], Loss: 1.6324, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [818/16140], Loss: 1.2959, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [819/16140], Loss: 1.2879, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [820/16140], Loss: 1.0979, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [821/16140], Loss: 1.2476, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [822/16140], Loss: 0.7505, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [823/16140], Loss: 1.0820, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [824/16140], Loss: 1.2146, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [825/16140], Loss: 1.1083, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [826/16140], Loss: 0.9002, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [827/16140], Loss: 0.9719, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [828/16140], Loss: 1.1315, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [829/16140], Loss: 1.2182, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [830/16140], Loss: 0.8582, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [831/16140], Loss: 1.0362, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [832/16140], Loss: 0.9643, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [833/16140], Loss: 0.9253, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [834/16140], Loss: 1.2493, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [835/16140], Loss: 1.2138, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [836/16140], Loss: 1.0842, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [837/16140], Loss: 0.9686, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [838/16140], Loss: 1.2373, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [839/16140], Loss: 1.0311, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [840/16140], Loss: 0.9861, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [841/16140], Loss: 0.9071, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [842/16140], Loss: 1.2309, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [843/16140], Loss: 1.0769, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [844/16140], Loss: 0.9712, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [845/16140], Loss: 0.7780, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [846/16140], Loss: 1.0546, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [847/16140], Loss: 1.2649, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [848/16140], Loss: 1.2095, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [849/16140], Loss: 1.3792, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [850/16140], Loss: 1.1998, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [851/16140], Loss: 1.3815, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [852/16140], Loss: 1.0079, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [853/16140], Loss: 1.0851, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [854/16140], Loss: 1.3294, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [855/16140], Loss: 1.0306, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [856/16140], Loss: 0.8697, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [857/16140], Loss: 1.0433, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [858/16140], Loss: 1.3248, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [859/16140], Loss: 1.0573, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [860/16140], Loss: 0.8894, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [861/16140], Loss: 1.0202, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [862/16140], Loss: 0.7444, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [863/16140], Loss: 0.8789, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [864/16140], Loss: 1.3527, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [865/16140], Loss: 1.4246, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [866/16140], Loss: 0.8762, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [867/16140], Loss: 0.9848, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [868/16140], Loss: 0.7951, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [869/16140], Loss: 1.1057, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [870/16140], Loss: 1.0594, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [871/16140], Loss: 0.7893, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [872/16140], Loss: 0.9222, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [873/16140], Loss: 0.9081, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [874/16140], Loss: 0.9028, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [875/16140], Loss: 0.6405, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [876/16140], Loss: 1.0824, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [877/16140], Loss: 1.1231, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [878/16140], Loss: 0.9253, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [879/16140], Loss: 0.9910, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [880/16140], Loss: 1.1923, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [881/16140], Loss: 0.8488, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [882/16140], Loss: 0.7142, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [883/16140], Loss: 1.0549, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [884/16140], Loss: 1.3418, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [885/16140], Loss: 1.0219, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [886/16140], Loss: 1.0236, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [887/16140], Loss: 1.0474, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [888/16140], Loss: 0.9129, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [889/16140], Loss: 1.4687, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [890/16140], Loss: 1.0444, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [891/16140], Loss: 1.1312, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [892/16140], Loss: 0.7043, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [893/16140], Loss: 0.8941, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [894/16140], Loss: 1.1913, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [895/16140], Loss: 0.9239, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [896/16140], Loss: 1.2856, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [897/16140], Loss: 0.8604, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [898/16140], Loss: 0.9798, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [899/16140], Loss: 1.0411, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [900/16140], Loss: 1.0436, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [901/16140], Loss: 1.0492, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [902/16140], Loss: 1.1454, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [903/16140], Loss: 0.9422, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [904/16140], Loss: 0.9219, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [905/16140], Loss: 1.2409, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [906/16140], Loss: 0.9866, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [907/16140], Loss: 1.1108, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [908/16140], Loss: 1.0767, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [909/16140], Loss: 1.1180, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [910/16140], Loss: 1.1816, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [911/16140], Loss: 1.2625, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [912/16140], Loss: 1.1024, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [913/16140], Loss: 1.3822, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [914/16140], Loss: 0.8769, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [915/16140], Loss: 1.2274, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [916/16140], Loss: 1.1123, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [917/16140], Loss: 1.1404, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [918/16140], Loss: 1.0847, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [919/16140], Loss: 0.8955, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [920/16140], Loss: 1.2180, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [921/16140], Loss: 1.0918, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [922/16140], Loss: 1.1169, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [923/16140], Loss: 1.1936, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [924/16140], Loss: 1.3914, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [925/16140], Loss: 0.9490, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [926/16140], Loss: 0.8337, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [927/16140], Loss: 0.8769, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [928/16140], Loss: 0.9485, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [929/16140], Loss: 1.1766, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [930/16140], Loss: 1.0646, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [931/16140], Loss: 1.2763, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [932/16140], Loss: 0.8894, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [933/16140], Loss: 1.2069, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [934/16140], Loss: 1.1659, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [935/16140], Loss: 1.2973, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [936/16140], Loss: 1.4019, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [937/16140], Loss: 0.9807, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [938/16140], Loss: 1.2088, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [939/16140], Loss: 1.2814, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [940/16140], Loss: 1.0461, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [941/16140], Loss: 0.6828, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [942/16140], Loss: 1.2635, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [943/16140], Loss: 0.9237, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [944/16140], Loss: 1.0012, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [945/16140], Loss: 1.1914, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [946/16140], Loss: 0.9248, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [947/16140], Loss: 0.9497, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [948/16140], Loss: 1.0209, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [949/16140], Loss: 0.8535, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [950/16140], Loss: 1.1457, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [951/16140], Loss: 1.0944, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [952/16140], Loss: 0.7972, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [953/16140], Loss: 1.0769, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [954/16140], Loss: 0.8233, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [955/16140], Loss: 0.9846, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [956/16140], Loss: 1.5448, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [957/16140], Loss: 0.6079, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [958/16140], Loss: 1.0170, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [959/16140], Loss: 0.8806, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [960/16140], Loss: 1.0815, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [961/16140], Loss: 0.8668, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [962/16140], Loss: 1.1388, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [963/16140], Loss: 0.9555, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [964/16140], Loss: 1.4170, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [965/16140], Loss: 1.4163, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [966/16140], Loss: 1.5149, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [967/16140], Loss: 1.1239, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [968/16140], Loss: 1.2072, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [969/16140], Loss: 1.1133, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [970/16140], Loss: 1.0898, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [971/16140], Loss: 1.1343, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [972/16140], Loss: 0.8823, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [973/16140], Loss: 1.0167, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [974/16140], Loss: 1.2276, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [975/16140], Loss: 1.0489, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [976/16140], Loss: 0.9286, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [977/16140], Loss: 1.2230, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [978/16140], Loss: 1.1147, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [979/16140], Loss: 1.1235, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [980/16140], Loss: 1.2966, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [981/16140], Loss: 0.8697, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [982/16140], Loss: 0.9923, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [983/16140], Loss: 1.0255, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [984/16140], Loss: 0.9805, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [985/16140], Loss: 1.3416, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [986/16140], Loss: 0.7849, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [987/16140], Loss: 1.3181, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [988/16140], Loss: 1.1117, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [989/16140], Loss: 1.0236, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [990/16140], Loss: 1.0819, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [991/16140], Loss: 0.7774, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [992/16140], Loss: 1.1304, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [993/16140], Loss: 1.1652, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [994/16140], Loss: 0.9717, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [995/16140], Loss: 1.0480, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [996/16140], Loss: 1.2004, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [997/16140], Loss: 0.8880, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [998/16140], Loss: 1.1107, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [999/16140], Loss: 1.1475, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1000/16140], Loss: 0.9568, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1001/16140], Loss: 1.3079, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1002/16140], Loss: 0.9858, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1003/16140], Loss: 1.0903, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1004/16140], Loss: 1.1155, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1005/16140], Loss: 0.9085, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1006/16140], Loss: 1.1420, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1007/16140], Loss: 0.7826, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1008/16140], Loss: 1.0783, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1009/16140], Loss: 1.1695, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1010/16140], Loss: 0.8676, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1011/16140], Loss: 0.9685, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1012/16140], Loss: 1.2614, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1013/16140], Loss: 1.0259, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1014/16140], Loss: 1.3746, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1015/16140], Loss: 1.1934, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1016/16140], Loss: 0.7929, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1017/16140], Loss: 1.0544, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1018/16140], Loss: 1.0622, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1019/16140], Loss: 0.9847, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1020/16140], Loss: 1.2174, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1021/16140], Loss: 0.9755, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1022/16140], Loss: 1.1016, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1023/16140], Loss: 0.7907, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1024/16140], Loss: 1.1910, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1025/16140], Loss: 0.9485, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1026/16140], Loss: 1.1320, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1027/16140], Loss: 1.0708, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1028/16140], Loss: 1.0496, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1029/16140], Loss: 1.5950, Accuracy: 36.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1030/16140], Loss: 0.7361, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1031/16140], Loss: 1.4290, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1032/16140], Loss: 1.2894, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1033/16140], Loss: 1.5069, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1034/16140], Loss: 0.9498, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1035/16140], Loss: 1.3229, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1036/16140], Loss: 1.1549, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1037/16140], Loss: 1.8325, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1038/16140], Loss: 1.1890, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1039/16140], Loss: 1.0355, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1040/16140], Loss: 0.7056, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1041/16140], Loss: 1.4275, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1042/16140], Loss: 1.1323, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1043/16140], Loss: 1.0280, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1044/16140], Loss: 0.7686, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1045/16140], Loss: 0.9240, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1046/16140], Loss: 0.8115, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1047/16140], Loss: 1.2678, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1048/16140], Loss: 1.2324, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1049/16140], Loss: 1.1889, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1050/16140], Loss: 1.0299, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1051/16140], Loss: 1.0935, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1052/16140], Loss: 1.1272, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1053/16140], Loss: 1.0797, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1054/16140], Loss: 0.9939, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1055/16140], Loss: 0.9207, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1056/16140], Loss: 0.9298, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1057/16140], Loss: 1.5606, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1058/16140], Loss: 1.1372, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1059/16140], Loss: 1.3363, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1060/16140], Loss: 1.0616, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1061/16140], Loss: 1.1577, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1062/16140], Loss: 1.0697, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1063/16140], Loss: 1.0701, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1064/16140], Loss: 1.1856, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1065/16140], Loss: 0.8828, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1066/16140], Loss: 0.8337, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1067/16140], Loss: 1.0669, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1068/16140], Loss: 0.9848, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1069/16140], Loss: 0.8381, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1070/16140], Loss: 1.0594, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1071/16140], Loss: 1.3237, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1072/16140], Loss: 1.2290, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1073/16140], Loss: 1.3340, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1074/16140], Loss: 0.9860, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1075/16140], Loss: 1.1513, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1076/16140], Loss: 0.8140, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1077/16140], Loss: 0.7925, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1078/16140], Loss: 0.8354, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1079/16140], Loss: 1.1845, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1080/16140], Loss: 1.4622, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1081/16140], Loss: 1.3433, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1082/16140], Loss: 1.4266, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1083/16140], Loss: 1.0806, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1084/16140], Loss: 1.0074, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1085/16140], Loss: 0.6290, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1086/16140], Loss: 1.2430, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1087/16140], Loss: 0.6367, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1088/16140], Loss: 1.0195, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1089/16140], Loss: 0.9529, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1090/16140], Loss: 0.7153, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1091/16140], Loss: 0.9806, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1092/16140], Loss: 1.3455, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1093/16140], Loss: 0.9437, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1094/16140], Loss: 1.3261, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1095/16140], Loss: 0.9343, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1096/16140], Loss: 1.0389, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1097/16140], Loss: 0.8468, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1098/16140], Loss: 1.1219, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1099/16140], Loss: 1.2177, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1100/16140], Loss: 1.1393, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1101/16140], Loss: 0.8948, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1102/16140], Loss: 1.0472, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1103/16140], Loss: 1.0611, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1104/16140], Loss: 1.0998, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1105/16140], Loss: 0.8233, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1106/16140], Loss: 0.7547, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1107/16140], Loss: 1.0395, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1108/16140], Loss: 1.1220, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1109/16140], Loss: 1.0748, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1110/16140], Loss: 1.0069, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1111/16140], Loss: 0.9441, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1112/16140], Loss: 0.7841, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1113/16140], Loss: 0.9166, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1114/16140], Loss: 0.8111, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1115/16140], Loss: 1.0283, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1116/16140], Loss: 1.1515, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1117/16140], Loss: 1.0284, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1118/16140], Loss: 0.8868, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1119/16140], Loss: 1.2856, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1120/16140], Loss: 0.8274, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1121/16140], Loss: 1.0878, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1122/16140], Loss: 1.0733, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1123/16140], Loss: 0.7466, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1124/16140], Loss: 1.3383, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1125/16140], Loss: 1.5299, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1126/16140], Loss: 0.8300, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1127/16140], Loss: 1.0503, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1128/16140], Loss: 0.8504, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1129/16140], Loss: 1.4295, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1130/16140], Loss: 0.9491, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1131/16140], Loss: 1.2632, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1132/16140], Loss: 0.7429, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1133/16140], Loss: 1.0048, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1134/16140], Loss: 0.9597, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1135/16140], Loss: 1.1172, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1136/16140], Loss: 1.3134, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1137/16140], Loss: 0.7489, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1138/16140], Loss: 0.9742, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1139/16140], Loss: 1.2608, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1140/16140], Loss: 1.0770, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1141/16140], Loss: 1.1298, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1142/16140], Loss: 1.2011, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1143/16140], Loss: 0.9974, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1144/16140], Loss: 0.9848, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1145/16140], Loss: 1.1243, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1146/16140], Loss: 1.3158, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1147/16140], Loss: 1.0262, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1148/16140], Loss: 0.8920, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1149/16140], Loss: 1.1539, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1150/16140], Loss: 1.0084, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1151/16140], Loss: 1.2094, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1152/16140], Loss: 1.0563, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1153/16140], Loss: 0.9290, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1154/16140], Loss: 0.9374, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1155/16140], Loss: 0.9434, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1156/16140], Loss: 1.3160, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1157/16140], Loss: 0.9655, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1158/16140], Loss: 0.8832, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1159/16140], Loss: 1.2563, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1160/16140], Loss: 1.4233, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1161/16140], Loss: 1.4446, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1162/16140], Loss: 0.8884, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1163/16140], Loss: 1.0939, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1164/16140], Loss: 1.1668, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1165/16140], Loss: 1.0883, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1166/16140], Loss: 1.0137, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1167/16140], Loss: 1.1924, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1168/16140], Loss: 1.3358, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1169/16140], Loss: 0.9531, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1170/16140], Loss: 0.9336, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1171/16140], Loss: 1.0394, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1172/16140], Loss: 1.0900, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1173/16140], Loss: 0.8022, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1174/16140], Loss: 1.3251, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1175/16140], Loss: 1.0100, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1176/16140], Loss: 0.9728, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1177/16140], Loss: 1.3775, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1178/16140], Loss: 1.2805, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1179/16140], Loss: 0.9438, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1180/16140], Loss: 0.8376, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1181/16140], Loss: 0.7649, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1182/16140], Loss: 1.0931, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1183/16140], Loss: 1.1895, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1184/16140], Loss: 0.7835, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1185/16140], Loss: 0.7594, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1186/16140], Loss: 1.1545, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1187/16140], Loss: 1.0128, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1188/16140], Loss: 0.7734, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1189/16140], Loss: 1.1428, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1190/16140], Loss: 1.4090, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1191/16140], Loss: 1.0250, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1192/16140], Loss: 0.9400, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1193/16140], Loss: 1.2245, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1194/16140], Loss: 1.1717, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1195/16140], Loss: 0.9168, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1196/16140], Loss: 1.1035, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1197/16140], Loss: 1.0998, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1198/16140], Loss: 1.1026, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1199/16140], Loss: 0.8920, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1200/16140], Loss: 0.9571, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1201/16140], Loss: 0.9642, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1202/16140], Loss: 1.1944, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1203/16140], Loss: 1.1867, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1204/16140], Loss: 1.2826, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1205/16140], Loss: 0.9121, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1206/16140], Loss: 1.0113, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1207/16140], Loss: 0.8437, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1208/16140], Loss: 1.2614, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1209/16140], Loss: 0.6942, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1210/16140], Loss: 1.3352, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1211/16140], Loss: 1.4092, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1212/16140], Loss: 0.9960, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1213/16140], Loss: 1.2459, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1214/16140], Loss: 0.8650, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1215/16140], Loss: 1.0309, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1216/16140], Loss: 1.2200, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1217/16140], Loss: 0.6841, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1218/16140], Loss: 0.8404, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1219/16140], Loss: 1.4351, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1220/16140], Loss: 0.9019, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1221/16140], Loss: 1.0318, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1222/16140], Loss: 1.1797, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1223/16140], Loss: 0.7773, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1224/16140], Loss: 0.8172, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1225/16140], Loss: 1.1105, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1226/16140], Loss: 1.3757, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1227/16140], Loss: 1.1310, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1228/16140], Loss: 0.8632, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1229/16140], Loss: 0.5820, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1230/16140], Loss: 1.1779, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1231/16140], Loss: 0.8253, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1232/16140], Loss: 0.6847, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1233/16140], Loss: 0.6549, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1234/16140], Loss: 1.0944, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1235/16140], Loss: 1.3426, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1236/16140], Loss: 0.9375, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1237/16140], Loss: 0.9388, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1238/16140], Loss: 1.1810, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1239/16140], Loss: 0.9227, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1240/16140], Loss: 1.0172, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1241/16140], Loss: 1.0559, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1242/16140], Loss: 1.3812, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1243/16140], Loss: 1.4180, Accuracy: 36.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1244/16140], Loss: 0.9142, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1245/16140], Loss: 1.3512, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1246/16140], Loss: 1.0068, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1247/16140], Loss: 1.0456, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1248/16140], Loss: 1.0267, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1249/16140], Loss: 1.0373, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1250/16140], Loss: 0.9217, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1251/16140], Loss: 1.4168, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1252/16140], Loss: 1.4914, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1253/16140], Loss: 1.1668, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1254/16140], Loss: 1.2334, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1255/16140], Loss: 1.4797, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1256/16140], Loss: 0.8922, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1257/16140], Loss: 1.2914, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1258/16140], Loss: 1.0236, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1259/16140], Loss: 1.0245, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1260/16140], Loss: 1.2903, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1261/16140], Loss: 1.0943, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1262/16140], Loss: 1.0664, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1263/16140], Loss: 1.0741, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1264/16140], Loss: 1.4183, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1265/16140], Loss: 1.2384, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1266/16140], Loss: 0.8436, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1267/16140], Loss: 1.1641, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1268/16140], Loss: 1.0566, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1269/16140], Loss: 0.9377, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1270/16140], Loss: 0.8020, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1271/16140], Loss: 1.0203, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1272/16140], Loss: 1.3882, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1273/16140], Loss: 1.1128, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1274/16140], Loss: 0.6973, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1275/16140], Loss: 0.7658, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1276/16140], Loss: 1.0938, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1277/16140], Loss: 1.2607, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1278/16140], Loss: 1.0801, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1279/16140], Loss: 1.0163, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1280/16140], Loss: 1.2723, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1281/16140], Loss: 0.9217, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1282/16140], Loss: 1.0024, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1283/16140], Loss: 1.2037, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1284/16140], Loss: 1.1597, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1285/16140], Loss: 1.0315, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1286/16140], Loss: 0.6490, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1287/16140], Loss: 1.0140, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1288/16140], Loss: 1.5505, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1289/16140], Loss: 0.7773, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1290/16140], Loss: 0.8890, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1291/16140], Loss: 1.2425, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1292/16140], Loss: 1.2395, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1293/16140], Loss: 0.8566, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1294/16140], Loss: 0.7961, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1295/16140], Loss: 0.9591, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1296/16140], Loss: 0.7840, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1297/16140], Loss: 0.9456, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1298/16140], Loss: 0.9631, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1299/16140], Loss: 1.1549, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1300/16140], Loss: 0.8754, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1301/16140], Loss: 0.9090, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1302/16140], Loss: 0.9323, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1303/16140], Loss: 0.8202, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1304/16140], Loss: 1.1312, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1305/16140], Loss: 0.6332, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1306/16140], Loss: 1.4333, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1307/16140], Loss: 0.6919, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1308/16140], Loss: 1.9122, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1309/16140], Loss: 0.7190, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1310/16140], Loss: 0.9928, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1311/16140], Loss: 1.2139, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1312/16140], Loss: 1.2484, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1313/16140], Loss: 1.0012, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1314/16140], Loss: 0.8107, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1315/16140], Loss: 1.2766, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1316/16140], Loss: 1.3295, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1317/16140], Loss: 0.9018, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1318/16140], Loss: 1.4088, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1319/16140], Loss: 1.5647, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1320/16140], Loss: 0.7999, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1321/16140], Loss: 0.9511, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1322/16140], Loss: 1.1941, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1323/16140], Loss: 0.9821, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1324/16140], Loss: 1.1133, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1325/16140], Loss: 1.4680, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1326/16140], Loss: 0.9837, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1327/16140], Loss: 1.3041, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1328/16140], Loss: 1.0782, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1329/16140], Loss: 0.9943, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1330/16140], Loss: 1.2433, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1331/16140], Loss: 0.7797, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1332/16140], Loss: 0.9327, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1333/16140], Loss: 1.0267, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1334/16140], Loss: 1.2084, Accuracy: 36.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1335/16140], Loss: 1.0113, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1336/16140], Loss: 0.8499, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1337/16140], Loss: 1.1030, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1338/16140], Loss: 1.1728, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1339/16140], Loss: 1.3280, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1340/16140], Loss: 0.6660, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1341/16140], Loss: 0.8985, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1342/16140], Loss: 1.2839, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1343/16140], Loss: 0.7191, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1344/16140], Loss: 1.1143, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1345/16140], Loss: 0.8190, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1346/16140], Loss: 0.9657, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1347/16140], Loss: 0.9546, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1348/16140], Loss: 1.0083, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1349/16140], Loss: 1.1951, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1350/16140], Loss: 1.2281, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1351/16140], Loss: 1.2517, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1352/16140], Loss: 1.3923, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1353/16140], Loss: 1.4346, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1354/16140], Loss: 0.7305, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1355/16140], Loss: 0.7525, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1356/16140], Loss: 1.1116, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1357/16140], Loss: 0.6046, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1358/16140], Loss: 1.0664, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1359/16140], Loss: 1.1510, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1360/16140], Loss: 1.2299, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1361/16140], Loss: 0.8484, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1362/16140], Loss: 0.8771, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1363/16140], Loss: 0.9474, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1364/16140], Loss: 1.3376, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1365/16140], Loss: 0.8991, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1366/16140], Loss: 1.1948, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1367/16140], Loss: 0.6517, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1368/16140], Loss: 1.1345, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1369/16140], Loss: 0.9540, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1370/16140], Loss: 0.6218, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1371/16140], Loss: 0.8127, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1372/16140], Loss: 0.7947, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1373/16140], Loss: 1.1623, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1374/16140], Loss: 0.7812, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1375/16140], Loss: 1.1012, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1376/16140], Loss: 1.2165, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1377/16140], Loss: 1.0316, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1378/16140], Loss: 0.5599, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1379/16140], Loss: 1.1480, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1380/16140], Loss: 0.9311, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1381/16140], Loss: 1.0486, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1382/16140], Loss: 1.2918, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1383/16140], Loss: 0.7488, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1384/16140], Loss: 0.6469, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1385/16140], Loss: 1.3179, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1386/16140], Loss: 1.2520, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1387/16140], Loss: 1.0446, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1388/16140], Loss: 1.3280, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1389/16140], Loss: 0.9841, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1390/16140], Loss: 1.2165, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1391/16140], Loss: 1.2359, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1392/16140], Loss: 1.0305, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1393/16140], Loss: 1.1495, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1394/16140], Loss: 0.9381, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1395/16140], Loss: 1.1076, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1396/16140], Loss: 1.4765, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1397/16140], Loss: 1.2042, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1398/16140], Loss: 0.9619, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1399/16140], Loss: 1.3048, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1400/16140], Loss: 1.2247, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1401/16140], Loss: 1.4134, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1402/16140], Loss: 0.8923, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1403/16140], Loss: 1.1490, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1404/16140], Loss: 1.1239, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1405/16140], Loss: 1.2759, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1406/16140], Loss: 0.8411, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1407/16140], Loss: 0.9453, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1408/16140], Loss: 1.0009, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1409/16140], Loss: 1.0018, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1410/16140], Loss: 1.1487, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1411/16140], Loss: 1.0855, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1412/16140], Loss: 1.0651, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1413/16140], Loss: 1.1320, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1414/16140], Loss: 0.9031, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1415/16140], Loss: 1.0367, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1416/16140], Loss: 0.8558, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1417/16140], Loss: 1.0064, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1418/16140], Loss: 0.9702, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1419/16140], Loss: 0.7638, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1420/16140], Loss: 1.3361, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1421/16140], Loss: 0.8387, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1422/16140], Loss: 0.7779, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1423/16140], Loss: 1.1005, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1424/16140], Loss: 1.2164, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1425/16140], Loss: 0.9515, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1426/16140], Loss: 1.2436, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1427/16140], Loss: 0.8203, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1428/16140], Loss: 0.7926, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1429/16140], Loss: 0.9505, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1430/16140], Loss: 1.3569, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1431/16140], Loss: 1.2292, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1432/16140], Loss: 1.2770, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1433/16140], Loss: 0.7724, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1434/16140], Loss: 0.8095, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1435/16140], Loss: 0.8987, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1436/16140], Loss: 0.7008, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1437/16140], Loss: 1.0064, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1438/16140], Loss: 0.7622, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1439/16140], Loss: 0.6296, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1440/16140], Loss: 1.2958, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1441/16140], Loss: 1.3271, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1442/16140], Loss: 0.8117, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1443/16140], Loss: 0.8945, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1444/16140], Loss: 1.0368, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1445/16140], Loss: 0.6642, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1446/16140], Loss: 0.9460, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1447/16140], Loss: 1.3790, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1448/16140], Loss: 1.0948, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1449/16140], Loss: 0.8744, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1450/16140], Loss: 1.1080, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1451/16140], Loss: 1.1674, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1452/16140], Loss: 1.0543, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1453/16140], Loss: 1.3582, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1454/16140], Loss: 0.7514, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1455/16140], Loss: 0.9830, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1456/16140], Loss: 0.8891, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1457/16140], Loss: 1.0259, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1458/16140], Loss: 1.2986, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1459/16140], Loss: 1.2787, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1460/16140], Loss: 1.1526, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1461/16140], Loss: 1.0421, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1462/16140], Loss: 1.1930, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1463/16140], Loss: 1.0700, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1464/16140], Loss: 1.2670, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1465/16140], Loss: 1.2251, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1466/16140], Loss: 1.1421, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1467/16140], Loss: 1.1220, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1468/16140], Loss: 1.2312, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1469/16140], Loss: 0.9928, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1470/16140], Loss: 1.0415, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1471/16140], Loss: 1.0005, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1472/16140], Loss: 1.2903, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1473/16140], Loss: 1.3248, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1474/16140], Loss: 0.9330, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1475/16140], Loss: 1.2842, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1476/16140], Loss: 0.8772, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1477/16140], Loss: 1.2482, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1478/16140], Loss: 1.3755, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1479/16140], Loss: 1.0522, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1480/16140], Loss: 0.9743, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1481/16140], Loss: 1.0526, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1482/16140], Loss: 0.8328, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1483/16140], Loss: 0.8505, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1484/16140], Loss: 0.7694, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1485/16140], Loss: 1.1001, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1486/16140], Loss: 0.9041, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1487/16140], Loss: 0.9251, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1488/16140], Loss: 0.7571, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1489/16140], Loss: 1.2975, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1490/16140], Loss: 1.1001, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1491/16140], Loss: 1.3672, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1492/16140], Loss: 0.9322, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1493/16140], Loss: 1.0632, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1494/16140], Loss: 1.6276, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1495/16140], Loss: 1.2904, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1496/16140], Loss: 0.9280, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1497/16140], Loss: 0.7539, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1498/16140], Loss: 1.2295, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1499/16140], Loss: 1.0442, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1500/16140], Loss: 1.3831, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1501/16140], Loss: 0.8606, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1502/16140], Loss: 0.9742, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1503/16140], Loss: 1.0060, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1504/16140], Loss: 0.8066, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1505/16140], Loss: 0.9246, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1506/16140], Loss: 0.9812, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1507/16140], Loss: 1.4254, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1508/16140], Loss: 1.0524, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1509/16140], Loss: 1.2222, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1510/16140], Loss: 0.8685, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1511/16140], Loss: 1.1264, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1512/16140], Loss: 0.8988, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1513/16140], Loss: 0.9970, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1514/16140], Loss: 1.0720, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1515/16140], Loss: 1.2969, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1516/16140], Loss: 0.9613, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1517/16140], Loss: 1.5182, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1518/16140], Loss: 0.6779, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1519/16140], Loss: 1.4836, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1520/16140], Loss: 1.2925, Accuracy: 43.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1521/16140], Loss: 0.9926, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1522/16140], Loss: 0.5934, Accuracy: 86.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1523/16140], Loss: 1.2553, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1524/16140], Loss: 1.0526, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1525/16140], Loss: 1.4224, Accuracy: 36.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1526/16140], Loss: 1.3365, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1527/16140], Loss: 0.5925, Accuracy: 90.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1528/16140], Loss: 0.9269, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1529/16140], Loss: 0.9867, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1530/16140], Loss: 1.0632, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1531/16140], Loss: 0.8377, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1532/16140], Loss: 1.3548, Accuracy: 40.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1533/16140], Loss: 0.8478, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1534/16140], Loss: 1.0043, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1535/16140], Loss: 0.9464, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1536/16140], Loss: 0.9343, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1537/16140], Loss: 0.8713, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1538/16140], Loss: 1.1077, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1539/16140], Loss: 0.9842, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1540/16140], Loss: 1.0062, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1541/16140], Loss: 1.1362, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1542/16140], Loss: 1.1353, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1543/16140], Loss: 1.1637, Accuracy: 50.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1544/16140], Loss: 0.9645, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1545/16140], Loss: 1.0946, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1546/16140], Loss: 1.5924, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1547/16140], Loss: 1.2082, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1548/16140], Loss: 1.2099, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1549/16140], Loss: 0.8798, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1550/16140], Loss: 1.2726, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1551/16140], Loss: 1.1548, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1552/16140], Loss: 1.1773, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1553/16140], Loss: 0.7701, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1554/16140], Loss: 1.2494, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1555/16140], Loss: 1.5487, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1556/16140], Loss: 1.2969, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1557/16140], Loss: 1.3206, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1558/16140], Loss: 1.0526, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1559/16140], Loss: 1.0792, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1560/16140], Loss: 1.2485, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1561/16140], Loss: 0.9333, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1562/16140], Loss: 1.0548, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1563/16140], Loss: 1.0918, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1564/16140], Loss: 0.9829, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1565/16140], Loss: 1.0930, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1566/16140], Loss: 0.8378, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1567/16140], Loss: 1.0772, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1568/16140], Loss: 1.0592, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1569/16140], Loss: 0.9053, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1570/16140], Loss: 1.1297, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1571/16140], Loss: 0.9566, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1572/16140], Loss: 0.9389, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1573/16140], Loss: 1.3148, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1574/16140], Loss: 0.9376, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1575/16140], Loss: 1.0416, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1576/16140], Loss: 0.7406, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1577/16140], Loss: 1.1687, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1578/16140], Loss: 0.7966, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1579/16140], Loss: 1.0512, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1580/16140], Loss: 1.3827, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1581/16140], Loss: 0.8561, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1582/16140], Loss: 0.9975, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1583/16140], Loss: 0.7709, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1584/16140], Loss: 1.2871, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1585/16140], Loss: 1.5476, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1586/16140], Loss: 1.0642, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1587/16140], Loss: 0.9825, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1588/16140], Loss: 0.9348, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1589/16140], Loss: 1.4004, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1590/16140], Loss: 0.8749, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1591/16140], Loss: 0.7607, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1592/16140], Loss: 0.9505, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1593/16140], Loss: 0.9472, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1594/16140], Loss: 0.9581, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1595/16140], Loss: 0.8646, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1596/16140], Loss: 1.0537, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1597/16140], Loss: 1.0146, Accuracy: 63.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1598/16140], Loss: 0.6963, Accuracy: 76.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1599/16140], Loss: 0.6772, Accuracy: 83.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1600/16140], Loss: 0.9016, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1601/16140], Loss: 0.8444, Accuracy: 66.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1602/16140], Loss: 1.4882, Accuracy: 53.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1603/16140], Loss: 1.0148, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1604/16140], Loss: 1.4420, Accuracy: 46.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1605/16140], Loss: 0.8787, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1606/16140], Loss: 1.0906, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1607/16140], Loss: 0.7373, Accuracy: 80.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1608/16140], Loss: 0.9522, Accuracy: 56.67%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1609/16140], Loss: 0.8026, Accuracy: 70.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1610/16140], Loss: 1.1938, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1611/16140], Loss: 0.9941, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1612/16140], Loss: 1.0776, Accuracy: 60.00%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1613/16140], Loss: 0.8778, Accuracy: 73.33%, Iteration Time: 0.02 seconds
Epoch [10/10], Step [1614/16140], Loss: 1.3895, Accuracy: 57.14%, Iteration Time: 0.02 seconds
Epoch [10/10], Loss: 1.0431, Accuracy: 76.76%
Training finished!

